In [ ]:
from latest_model.ContentAnalysisPipeline import ContentAnalysisPipeline
from latest_model.mistralModel import OpenRouterMistralClient
import os

def print_section_header(title):
    """Print a formatted section header for better readability"""
    print("\n" + "="*50)
    print(f" {title} ".center(50, "="))
    print("="*50 + "\n")

def main():
    """
    Main function to test the ContentAnalysisPipeline functionality without command-line arguments
    """
    # Create test directory
    output_dir = "test_output"
    os.makedirs(output_dir, exist_ok=True)
    
    print_section_header("TESTING CONTENT ANALYSIS PIPELINE")
    print("This script demonstrates the functionality of ContentAnalysisPipeline")
    
    # Test with different domain configurations
    domains_to_test = ["literature"]#None, ,"literature","science", "math" ]
    
    for domain in domains_to_test:
        print_section_header(f"TESTING WITH DOMAIN: {domain if domain else 'Generic (None)'}")
        
        # Initialize pipeline with domain
        print(f"Initializing ContentAnalysisPipeline with domain: {domain}")
        api_token = "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"
        lm_client = OpenRouterMistralClient(api_key=api_token)

        pipeline = ContentAnalysisPipeline(domain=domain,lm_client=lm_client)
        
        # Sample PDF path (doesn't need to exist for this test)
        pdf_path = "input.pdf"
        
        # Process the document
        print(f"Processing document: {pdf_path}")
        result = pipeline.process_document(
            pdf_path=pdf_path,
            output_dir=output_dir,
            extract_images=True,
            extract_tables=True,
            respect_heading_hierarchy=True
        )
        
        # Save results
        output_path = os.path.join(output_dir, f"analysis_result_{domain if domain else 'generic'}.json")
        saved_files = pipeline.save_results(result, output_path, formats=['json', 'markdown','text'])
        
        # Display a sample of the results
        print_section_header("SAMPLE OF RESULTS")
        print("Document metadata:")
        print(result.keys())
        for key, value in result['document_metadata'].items():
            print(f"  - {key}: {value}")
        
        # Updated to use the correct path for keywords
        print("\nKeywords and concepts extracted:")
        keywords_concepts = result['semantic_analysis'].get('keywords_and_concepts', [])
        if keywords_concepts:
            print(f"  - {', '.join(keywords_concepts)}")
        else:
            print("  - No keywords found")
        
        print("\nDomain-specific data:")
        for key in result['domain_specific']:
            print(f"  - {key}")
        
        # Check file existence
        print("\nOutput files created:")
        for file_path in saved_files:
            if os.path.exists(file_path):
                print(f"  - {file_path} ({os.path.getsize(file_path)} bytes)")
            else:
                print(f"  - Failed to create {file_path}")
        
        print("\n" + "-"*50 + "\n")
    
    print_section_header("TEST COMPLETED")
    print("The ContentAnalysisPipeline test has completed successfully.")
    print(f"All output files can be found in the '{output_dir}' directory.")

if __name__ == "__main__":
    main()

In [2]:
"""
Advanced PDF Content Extractor
------------------------------
Extracts content from PDFs with support for:
- Structured document understanding
- Mathematical expressions and equations
- Chemical formulas and reactions
- Tables and figures
- Document metadata
"""

import os
import re
import json
import numpy as np
import fitz  # PyMuPDF
import cv2
from PIL import Image
import io
import pytesseract
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTFigure, LTImage, LTTextBoxHorizontal
import camelot  # For table extraction
from sympy.parsing.latex import parse_latex
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class PDFContentExtractor:
    def __init__(self, tesseract_path=None, math_recognition=True, chem_recognition=True):
        """
        Initialize the PDF content extractor
        
        Args:
            tesseract_path: Path to Tesseract OCR executable
            math_recognition: Boolean to enable mathematical expression recognition
            chem_recognition: Boolean to enable chemical formula recognition
        """
        self.math_recognition = math_recognition
        self.chem_recognition = chem_recognition
        
        # Configure Tesseract if path provided
        if tesseract_path:
            pytesseract.pytesseract.tesseract_cmd = tesseract_path
            
        # Patterns for identifying mathematical content
        self.math_patterns = {
            'latex_delimiters': re.compile(r'\$(.*?)\$|\\\[(.*?)\\\]|\\\((.*?)\\\)'),
            'subscripts': re.compile(r'[a-zA-Z0-9]_\{[^}]+\}|[a-zA-Z0-9]_[0-9]'),
            'fractions': re.compile(r'\\frac\{[^}]+\}\{[^}]+\}'),
            'integrals': re.compile(r'\\int'),
            'summations': re.compile(r'\\sum'),
            'square_roots': re.compile(r'\\sqrt')
        }
        
        # Patterns for identifying chemical content
        self.chem_patterns = {
            'chemical_formulas': re.compile(r'[A-Z][a-z]?[0-9]*'),
            'reaction_arrows': re.compile(r'->|→|⟶|⇌|⇄|⇆|⇋|↔'),
            'subscripts': re.compile(r'_\{[^}]+\}|_[0-9]'),
            'superscripts': re.compile(r'\^\{[^}]+\}|\^[0-9\+\-]')
        }
        
    def extract_metadata(self, pdf_doc):
        """Extract document metadata"""
        metadata = {
            "title": pdf_doc.metadata.get("title", ""),
            "author": pdf_doc.metadata.get("author", ""),
            "subject": pdf_doc.metadata.get("subject", ""),
            "keywords": pdf_doc.metadata.get("keywords", ""),
            "creator": pdf_doc.metadata.get("creator", ""),
            "producer": pdf_doc.metadata.get("producer", ""),
            "creation_date": pdf_doc.metadata.get("creationDate", ""),
            "modification_date": pdf_doc.metadata.get("modDate", ""),
            "page_count": pdf_doc.page_count
        }
        
        # Extract document structure (TOC/outline if available)
        toc = pdf_doc.get_toc()
        if toc:
            metadata["table_of_contents"] = toc
            
        return metadata
        
    def is_math_content(self, text):
        """Check if text contains mathematical expressions"""
        if not self.math_recognition:
            return False
            
        for pattern_name, pattern in self.math_patterns.items():
            if pattern.search(text):
                return True
        return False
        
    def is_chemical_content(self, text):
        """Check if text contains chemical formulas or reactions"""
        if not self.chem_recognition:
            return False
            
        # Count matches for chemical patterns
        matches = 0
        for pattern_name, pattern in self.chem_patterns.items():
            if pattern.search(text):
                matches += 1
        
        # If multiple patterns match, it's likely chemical content
        return matches >= 2
        
    def process_math_content(self, text):
        """Process and normalize mathematical content"""
        # Convert LaTeX math to a structured representation
        try:
            # Extract LaTeX expressions
            math_expressions = []
            for delim_pattern in [r'\$(.*?)\$', r'\\\[(.*?)\\\]', r'\\\((.*?)\\\)']:
                for match in re.finditer(delim_pattern, text, re.DOTALL):
                    expr = match.group(1)
                    if expr:
                        math_expressions.append({
                            'type': 'math_expression',
                            'latex': expr,
                            'position': match.span()
                        })
                        
            return math_expressions
        except Exception as e:
            logger.warning(f"Error processing math content: {e}")
            return text
            
    def process_chemical_content(self, text):
        """Process chemical formulas and reactions"""
        try:
            # Find reaction arrows
            reactions = []
            for match in self.chem_patterns['reaction_arrows'].finditer(text):
                # Look for chemical components before and after the arrow
                reaction_text = text[max(0, match.start() - 20):min(len(text), match.end() + 20)]
                reactions.append({
                    'type': 'chemical_reaction',
                    'text': reaction_text,
                    'position': match.span()
                })
                
            # Find standalone chemical formulas
            formulas = []
            for match in self.chem_patterns['chemical_formulas'].finditer(text):
                # Only consider it a formula if it matches certain patterns
                formula = match.group(0)
                if len(formula) > 1 and re.search(r'[A-Z][a-z]?[0-9]+', formula):
                    formulas.append({
                        'type': 'chemical_formula',
                        'text': formula,
                        'position': match.span()
                    })
                    
            return {'reactions': reactions, 'formulas': formulas}
        except Exception as e:
            logger.warning(f"Error processing chemical content: {e}")
            return text
            
    def extract_images(self, pdf_doc, output_dir=None):
        """Extract images from PDF with position information"""
        images = []

        for page_idx, page in enumerate(pdf_doc):
            image_list = page.get_images(full=True)

            for img_idx, img_info in enumerate(image_list):
                xref = img_info[0]
                base_image = pdf_doc.extract_image(xref)

                if not base_image:
                    continue

                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                # Get image position on page - Corrected method usage
                position = None
                for img_rect in page.get_image_rects(img_info):  #  Passing the image info as parameter
                    position = {
                        "x0": img_rect.x0,
                        "y0": img_rect.y0,
                        "x1": img_rect.x1,
                        "y1": img_rect.y1
                    }
                    break
                
                # Save image if output directory provided
                image_filename = None
                if output_dir:
                    os.makedirs(output_dir, exist_ok=True)
                    image_filename = f"page{page_idx+1}_img{img_idx +1}.{image_ext}"
                    image_path = os.path.join(output_dir,   image_filename)

                    with open(image_path, "wb") as img_file:
                        img_file.write(image_bytes)

                # Use OCR to extract any text in the image
                image_text = None
                try:
                    pil_image = Image.open(io.BytesIO(image_bytes))
                    image_text = pytesseract.image_to_string    (pil_image)
                    if not image_text.strip():
                        image_text = None
                except Exception as e:
                    logger.warning(f"OCR failed for image: {e}")

                images.append({
                    "page": page_idx + 1,
                    "index": img_idx + 1,
                    "position": position,
                    "file_name": image_filename,
                    "format": image_ext,
                    "ocr_text": image_text
                })

        return images
    
    def extract_tables(self, pdf_path):
        """Extract tables from PDF using Camelot"""
        tables = []
        
        try:
            # Use Camelot to extract tables
            table_pages = camelot.read_pdf(pdf_path, pages='all', flavor='lattice')
            
            for i, table in enumerate(table_pages):
                tables.append({
                    "page": table.page,
                    "index": i + 1,
                    "data": table.df.to_dict('records'),
                    "accuracy": table.accuracy,
                    "whitespace": table.whitespace,
                    "positions": {
                        "top": table._bbox[1],
                        "bottom": table._bbox[3],
                        "left": table._bbox[0],
                        "right": table._bbox[2]
                    }
                })
        except Exception as e:
            logger.warning(f"Table extraction failed: {e}")
            
        return tables
        
    def extract_document_structure(self, pdf_path):
        """Extract document structure using pdfminer to understand layout"""
        structure = []
        
        for page_layout in extract_pages(pdf_path):
            page_struct = {"type": "page", "elements": []}
            
            # Sort elements by their vertical position (y-coordinate)
            elements = sorted(
                [element for element in page_layout],
                key=lambda e: -e.y0
            )
            
            for element in elements:
                if isinstance(element, LTTextBoxHorizontal):
                    text = element.get_text().strip()
                    if not text:
                        continue
                    
                    # Analyze font size to detect headers
                    font_sizes = []
                    for text_line in element:
                        for char in text_line:
                            if hasattr(char, "size"):
                                font_sizes.append(char.size)
                    
                    avg_font_size = sum(font_sizes) / len(font_sizes) if font_sizes else 0
                    
                    # Detect if this is likely a heading
                    is_heading = False
                    if avg_font_size > 12 and len(text) < 100:  # Simplified heuristic
                        is_heading = True
                    
                    page_struct["elements"].append({
                        "type": "heading" if is_heading else "text",
                        "text": text,
                        "position": {
                            "x0": element.x0,
                            "y0": element.y0,
                            "x1": element.x1,
                            "y1": element.y1
                        },
                        "font_size": avg_font_size if font_sizes else None
                    })
                    
                elif isinstance(element, (LTFigure, LTImage)):
                    page_struct["elements"].append({
                        "type": "figure",
                        "position": {
                            "x0": element.x0,
                            "y0": element.y0,
                            "x1": element.x1,
                            "y1": element.y1
                        }
                    })
            
            structure.append(page_struct)
            
        return structure
                
    def extract_content(self, pdf_path, output_dir=None, extract_images=True, extract_tables=True):
        """
        Main method to extract all content from PDF
        
        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to save extracted images (optional)
            extract_images: Boolean indicating whether to extract images
            extract_tables: Boolean indicating whether to extract tables
            
        Returns:
            Dictionary containing all extracted content
        """
        result = {
            "metadata": {},
            "structure": [],
            "pages": [],
            "images": [],
            "tables": [],
            "math_expressions": [],
            "chemical_content": []
        }
        
        try:
            # Open PDF document
            pdf_doc = fitz.open(pdf_path)
            
            # Extract metadata
            result["metadata"] = self.extract_metadata(pdf_doc)
            
            # Extract document structure
            result["structure"] = self.extract_document_structure(pdf_path)
            
            # Extract images if requested
            if extract_images:
                result["images"] = self.extract_images(pdf_doc, output_dir)
                
            # Extract tables if requested
            if extract_tables:
                result["tables"] = self.extract_tables(pdf_path)
                
            # Process each page for text content
            all_math_expressions = []
            all_chemical_content = []
            
            for page_idx, page in enumerate(pdf_doc):
                page_text = page.get_text()
                page_content = {
                    "page_number": page_idx + 1,
                    "text": page_text,
                    "blocks": []
                }
                
                # Extract text blocks with position information
                blocks = page.get_text("blocks")
                for block_idx, block in enumerate(blocks):
                    x0, y0, x1, y1, text, block_no, block_type = block
                    
                    block_content = {
                        "block_number": block_idx + 1,
                        "text": text,
                        "position": {"x0": x0, "y0": y0, "x1": x1, "y1": y1},
                        "type": "text"
                    }
                    
                    # Check for special content (math or chemical)
                    if self.is_math_content(text):
                        block_content["type"] = "math"
                        math_exprs = self.process_math_content(text)
                        block_content["math_expressions"] = math_exprs
                        all_math_expressions.extend(math_exprs)
                        
                    elif self.is_chemical_content(text):
                        block_content["type"] = "chemical"
                        chem_content = self.process_chemical_content(text)
                        block_content["chemical_content"] = chem_content
                        all_chemical_content.append(chem_content)
                        
                    page_content["blocks"].append(block_content)
                    
                result["pages"].append(page_content)
            
            result["math_expressions"] = all_math_expressions
            result["chemical_content"] = all_chemical_content
            
            # Close the document
            pdf_doc.close()
            
            return result
            
        except Exception as e:
            logger.error(f"Error extracting PDF content: {e}")
            return {"error": str(e)}
                    
    @staticmethod
    def preprocess_pdf_for_extraction(pdf_path, output_path=None):
        """Preprocess PDF to improve extraction quality (optional)"""
        if output_path is None:
            output_path = pdf_path.replace(".pdf", "_processed.pdf")
            
        doc = fitz.open(pdf_path)
        for page in doc:
            # Apply OCR to pages with little text
            if len(page.get_text().strip()) < 100:
                pix = page.get_pixmap()
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                text = pytesseract.image_to_string(img)
                
                # If OCR found text, add it to the page
                if text.strip():
                    page.insert_text((50, 50), text)
                    
        doc.save(output_path)
        doc.close()
        return output_path


c:\Users\adity\envs\4thyearproject\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [3]:

    # # Add to PDFContentExtractor class
    # def segment_text(self, text):
    #     """
    #     Basic text segmentation - splits text into blocks by paragraphs
        
    #     Args:
    #         text: Text content to segment
            
    #     Returns:
    #         List of text blocks
    #     """
    #     # Simple paragraph-based segmentation
    #     paragraphs = text.split('\n\n')
    #     return [p.strip() for p in paragraphs if p.strip()]
        
    # def identify_block_type(self, text, position=None, font_info=None):
    #     """
    #     Basic block type identification based on text content
        
    #     Args:
    #         text: Text content to analyze
    #         position: Optional position information
    #         font_info: Optional font information
            
    #     Returns:
    #         String indicating block type ('text', 'heading', etc.)
    #     """
    #     text = text.strip()
        
    #     # Basic heading detection
    #     if len(text) < 100 and text.endswith(':'):
    #         return "heading"
            
    #     # Check for mathematical content
    #     if self.is_math_content(text):
    #         return "math"
            
    #     # Check for chemical content
    #     if self.is_chemical_content(text):
    #         return "chemical"
            
    #     # Default
    #     return "text"
    
    # def save_extraction_result(self, result, output_path):
    #         """Save extraction result to a JSON file"""
    #         with open(output_path, 'w', encoding='utf-8') as f:
    #             json.dump(result, f, indent=2, ensure_ascii=False)

    # def segment_text(self, text, min_length=10, min_words=3):
    #     """
    #     Core text segmentation logic to break text into meaningful blocks

    #     Args:
    #         text: Text to segment
    #         min_length: Minimum character length for a meaningful segment
    #         min_words: Minimum word count for a meaningful segment

    #     Returns:
    #         List of text segments
    #     """
    #     if not text or not text.strip():
    #         return []

    #     # Simple paragraph-based segmentation
    #     paragraphs = text.replace('\r\n', '\n').split('\n\n')
    #     segments = []

    #     for para in paragraphs:
    #         para = para.strip()
    #         if not para:
    #             continue
            
    #         # Check if the paragraph meets minimum requirements
    #         if len(para) < min_length:
    #             continue
            
    #         words = para.split()
    #         if len(words) < min_words:
    #             continue
            
    #         segments.append(para)

    #     return segments

    # def classify_block(self, text):
    #     """
    #     Basic block classification for the base extractor

    #     Args:
    #         text: Text block to classify

    #     Returns:
    #         Dictionary with classification details
    #     """
    #     block_type = "text"  # Default type

    #     # Check for special content types already in PDFContentExtractor
    #     if self.is_math_content(text):
    #         block_type = "math"
    #         content = self.process_math_content(text)
    #     elif self.is_chemical_content(text):
    #         block_type = "chemical"
    #         content = self.process_chemical_content(text)
    #     else:
    #         content = text

    #     return {
    #         "type": block_type,
    #         "content": content
    #     }
    
    # def identify_structure(self, blocks, page_idx=0):
    #     """
    #     Basic structure identification (base implementation)
        
    #     Args:
    #         blocks: List of text blocks
    #         page_idx: Page index
            
    #     Returns:
    #         List of structured content with basic type identification
    #     """
    #     structured_content = []
        
    #     for idx, block in enumerate(blocks):
    #         text = block.get("text", "")
    #         if not text:
    #             continue
                
    #         # Basic classification
    #         classification = self.classify_block(text)
            
    #         # Create a structured block
    #         structured_block = {
    #             "page": page_idx + 1,
    #             "block_index": idx,
    #             "text": text,
    #             "type": classification["type"],
    #             "position": block.get("position")
    #         }
            
    #         # Add any special content
    #         if classification["type"] == "math":
    #             structured_block["math_expressions"] = classification["content"]
    #         elif classification["type"] == "chemical":
    #             structured_block["chemical_content"] = classification["content"]
                
    #         structured_content.append(structured_block)
            
    #     return structured_content

In [4]:
# def convert_to_markdown(extraction_result, image_output_dir=None):
#     """
#     Convert the extracted PDF content to a markdown document
    
#     Args:
#         extraction_result: The dictionary returned by extract_content_with_cleaning
#         image_output_dir: Directory where images are stored (to create proper paths)
        
#     Returns:
#         String containing the formatted markdown content
#     """
#     markdown_lines = []
    
#     # Add document title if available
#     title = extraction_result.get("metadata", {}).get("title")
#     if title:
#         markdown_lines.append(f"# {title}")
#         markdown_lines.append("")
    
#     # Process each page's structured content
#     for page in extraction_result.get("pages", []):
#         structured_content = page.get("structured_content", [])
        
#         for section in structured_content:
#             heading_text = section.get("heading")
#             heading_level = section.get("heading_level", 3)  # Default to level 3
#             content = section.get("content", [])
            
#             # Add section heading with proper markdown level
#             if heading_text:
#                 # Ensure heading level is between 1-6 (valid markdown heading levels)
#                 valid_level = max(1, min(heading_level, 6))
#                 heading_marker = "#" * valid_level
#                 markdown_lines.append(f"{heading_marker} {heading_text}")
#                 markdown_lines.append("")
            
#             # Add section content - fix for int vs string issue
#             for text in content:
#                 if text is None:
#                     continue
                    
#                 # Convert to string if not already
#                 if not isinstance(text, str):
#                     text = str(text)
                    
#                 if text.strip():
#                     # Check if this might be a list item
#                     if text.strip().startswith("•") or text.strip().startswith("-") or re.match(r"^\d+\.\s", text.strip()):
#                         # It's likely a list item, preserve it
#                         markdown_lines.append(text)
#                     else:
#                         # Regular paragraph
#                         markdown_lines.append(text)
#                     markdown_lines.append("")  # Add blank line after each content block
    
#     # Process tables if available
#     tables = extraction_result.get("tables", [])
#     if tables:
#         markdown_lines.append("## Tables")
#         markdown_lines.append("")
        
#         for i, table in enumerate(tables):
#             markdown_lines.append(f"### Table {i+1}")
#             markdown_lines.append("")
            
#             # Convert table data to markdown format
#             table_data = table.get("data", [])
#             if table_data:
#                 # Get all keys from all rows to ensure complete header
#                 all_keys = set()
#                 for row in table_data:
#                     all_keys.update(row.keys())
                
#                 # Create header row
#                 header = " | ".join(str(key) for key in all_keys)  # Convert keys to strings
#                 markdown_lines.append(f"| {header} |")
                
#                 # Create separator row
#                 separator = " | ".join(["---" for _ in all_keys])
#                 markdown_lines.append(f"| {separator} |")
                
#                 # Create data rows
#                 for row in table_data:
#                     values = [str(row.get(key, "")) for key in all_keys]  # Ensure all values are strings
#                     data_row = " | ".join(values)
#                     markdown_lines.append(f"| {data_row} |")
                
#                 markdown_lines.append("")
    
#     # Process images if available
#     images = extraction_result.get("images", [])
#     if images and any(img.get("file_name") for img in images):
#         markdown_lines.append("## Images")
#         markdown_lines.append("")
        
#         for i, image in enumerate(images):
#             file_name = image.get("file_name")
#             ocr_text = image.get("ocr_text")
            
#             if file_name:
#                 markdown_lines.append(f"### Image {i+1}")
#                 markdown_lines.append("")
                
#                 # Fix: Create proper relative path to the image
#                 if image_output_dir:
#                     # Use path relative to the markdown file
#                     img_path = os.path.join(os.path.basename(image_output_dir), file_name)
#                     # Replace backslashes with forward slashes for markdown compatibility
#                     img_path = img_path.replace("\\", "/")
#                 else:
#                     img_path = file_name
                    
#                 markdown_lines.append(f"![Image {i+1}]({img_path})")
                
#                 if ocr_text:
#                     markdown_lines.append("")
#                     markdown_lines.append("**OCR Text:**")
#                     markdown_lines.append("")
#                     markdown_lines.append(f"```\n{ocr_text}\n```")
                
#                 markdown_lines.append("")
    
#     # Process mathematical content if available
#     math_expressions = extraction_result.get("math_expressions", [])
#     if math_expressions:
#         markdown_lines.append("## Mathematical Content")
#         markdown_lines.append("")
        
#         for i, expr in enumerate(math_expressions):
#             latex = expr.get("latex")
#             if latex:
#                 markdown_lines.append(f"### Expression {i+1}")
#                 markdown_lines.append("")
#                 # Use double dollar signs for display math in markdown
#                 markdown_lines.append(f"$${latex}$$")
#                 markdown_lines.append("")
    
#     # Process chemical content if available
#     chemical_content = extraction_result.get("chemical_content", [])
#     if chemical_content:
#         markdown_lines.append("## Chemical Content")
#         markdown_lines.append("")
        
#         # Process reactions
#         reactions = []
#         for item in chemical_content:
#             if isinstance(item, dict) and "reactions" in item:
#                 reactions.extend(item["reactions"])
        
#         if reactions:
#             markdown_lines.append("### Chemical Reactions")
#             markdown_lines.append("")
            
#             for i, reaction in enumerate(reactions):
#                 reaction_text = reaction.get("text", "")
#                 if reaction_text:
#                     markdown_lines.append(f"{i+1}. `{reaction_text}`")
#                     markdown_lines.append("")
    
#     # Add metadata section at the end
#     metadata = extraction_result.get("metadata", {})
#     if metadata:
#         markdown_lines.append("## Document Metadata")
#         markdown_lines.append("")
#         markdown_lines.append("| Property | Value |")
#         markdown_lines.append("| --- | --- |")
        
#         for key, value in metadata.items():
#             if key == "table_of_contents":
#                 continue  # Skip TOC for the table
#             # Ensure values are strings
#             markdown_lines.append(f"| {key} | {str(value)} |")
        
#         markdown_lines.append("")
    
#     # Return the complete markdown document
#     return "\n".join(markdown_lines)

In [3]:
"""
Enhanced PDF Content Extractor with Inheritance
----------------------------------------------
Extends the original PDFContentExtractor with improved:
- Text cleaning and noise removal
- Better heading detection
- Content organization
- JSON and TXT output formats
"""
import os
import re
import json
import string
import unicodedata
from collections import defaultdict
import io
from PIL import Image
import logging


class EnhancedPDFExtractor(PDFContentExtractor):
    def __init__(self, tesseract_path=None, math_recognition=True, chem_recognition=True):
        """
        Initialize the enhanced PDF content extractor, inheriting from PDFContentExtractor
        
        Args:
            tesseract_path: Path to Tesseract OCR executable
            math_recognition: Boolean to enable mathematical expression recognition
            chem_recognition: Boolean to enable chemical formula recognition
        """
        # Initialize the parent class
        super().__init__(tesseract_path, math_recognition, chem_recognition)
        
        # Configure logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)
        
        # Additional regex patterns for cleaning
        self.noise_patterns = [
            re.compile(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]'),  # Control characters
            re.compile(r'^\s*$'),  # Empty lines
            re.compile(r'^\s*[.•■□○●◆◇★☆♦♠♣♥→←↑↓⇒⇐⇑⇓⇔⇕⇖⇗⇘⇙]+\s*$'),  # Lines with only bullets/symbols
        ]
        
        # Potential headings detection
        self.heading_patterns = [
            re.compile(r'^[0-9]+\.[0-9.]*\s+[A-Z]'),  # Numbered headings (1.2 Title)
            re.compile(r'^[A-Z][a-z]*\s+[0-9]+\.\s'),  # Chapter/Section headings (Chapter 1.)
            re.compile(r'^[A-Z][A-Z\s]+$'),  # ALL CAPS headings
            re.compile(r'^[A-Z][a-z]+(\s+[A-Z][a-z]+){0,5}$')  # Title Case short lines
        ]
        
        # For filtering out irrelevant short fragments
        self.min_content_length = 10  # Minimum characters for content blocks
        self.min_words_for_content = 3  # Minimum words for content to be considered relevant
        
        # Font analysis parameters
        self.font_heading_threshold = 1.2  # Multiplier above avg font size to be considered heading
        self.font_title_threshold = 1.5   # Multiplier above avg font size to be considered title
        self.font_weight_threshold = 600  # Minimum font weight to be considered bold
        
    def clean_text(self, text):
        """Clean text by removing noise and normalizing content"""
        # Normalize unicode characters
        text = unicodedata.normalize('NFKC', text)
        
        # Replace common problematic characters
        text = text.replace('\u2028', '\n').replace('\u2029', '\n')  # Line/paragraph separators
        
        # Remove control characters and other noise
        for pattern in self.noise_patterns:
            text = pattern.sub('', text)
        
        # Fix spacing issues
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
        text = re.sub(r'\n\s*\n+', '\n\n', text)  # Normalize multiple newlines
        
        # Normalize punctuation spacing
        text = re.sub(r'\s+([.,;:!?)])', r'\1', text)
        text = re.sub(r'([([{])\s+', r'\1', text)
        
        return text.strip()
    
    def analyze_document_fonts(self, pdf_doc):
        """
        Analyze fonts across the document to identify common sizes and styles
        
        Args:
            pdf_doc: The fitz PDF document object
            
        Returns:
            Dictionary with font statistics and hierarchical font information
        """
        font_sizes = []
        font_weights = []
        font_details = []
        
        for page_idx, page in enumerate(pdf_doc):
            # Get detailed text information including fonts
            text_dict = page.get_text("dict", flags=11)
            
            for block in text_dict.get("blocks", []):
                if "lines" not in block:
                    continue
                    
                for line in block.get("lines", []):
                    for span in line.get("spans", []):
                        text = span.get("text", "").strip()
                        if not text:
                            continue
                            
                        # Extract font information
                        font_name = span.get("font", "")
                        font_size = span.get("size", 0)
                        flags = span.get("flags", 0)
                        
                        # Determine if it's bold (bit 1 in flags is often used for bold)
                        is_bold = bool(flags & 1)
                        # Determine if it's italic (bit 2 in flags is often used for italic)
                        is_italic = bool(flags & 2)
                        
                        # Estimate font weight (400=normal, 700=bold)
                        font_weight = 700 if is_bold else 400
                        
                        # Store details
                        if font_size > 0:
                            font_sizes.append(font_size)
                            font_weights.append(font_weight)
                            
                            font_details.append({
                                "text": text,
                                "font_name": font_name,
                                "font_size": font_size,
                                "is_bold": is_bold,
                                "is_italic": is_italic,
                                "font_weight": font_weight,
                                "page": page_idx + 1,
                                "bbox": [span.get("bbox", [0, 0, 0, 0])]
                            })
        
        # Compute statistics
        if not font_sizes:
            return {
                "min_size": 0, 
                "max_size": 0, 
                "avg_size": 0, 
                "common_size": 0,
                "font_histogram": {},
                "heading_candidates": []
            }
        
        # Create a histogram of font sizes rounded to nearest 0.5
        font_counts = defaultdict(int)
        for size in font_sizes:
            rounded_size = round(size * 2) / 2
            font_counts[rounded_size] += 1
            
        # Sort by frequency to get most common sizes
        sorted_sizes = sorted(font_counts.items(), key=lambda x: x[1], reverse=True)
        common_size = sorted_sizes[0][0] if sorted_sizes else 0
        
        # Identify potential heading sizes (significantly larger than common size)
        avg_size = sum(font_sizes) / len(font_sizes)
        heading_sizes = [size for size in set(font_sizes) if size >= avg_size * self.font_heading_threshold]
        heading_sizes.sort(reverse=True)  # Sort from largest to smallest
        
        # Identify text that matches heading criteria
        heading_candidates = []
        for detail in font_details:
            # Consider it a heading candidate if:
            # 1. Font size is in heading_sizes list OR
            # 2. Font is bold and size is at least average
            if (detail["font_size"] in heading_sizes or 
                (detail["is_bold"] and detail["font_size"] >= avg_size)):
                
                # Classify heading level based on size
                if detail["font_size"] >= avg_size * self.font_title_threshold:
                    level = 1  # Title/h1
                elif detail["font_size"] >= avg_size * self.font_heading_threshold:
                    level = 2  # Subtitle/h2
                elif detail["is_bold"] and detail["font_size"] >= avg_size:
                    level = 3  # Section/h3
                else:
                    level = 4  # Subsection/h4
                
                heading_candidates.append({
                    "text": detail["text"],
                    "font_size": detail["font_size"],
                    "is_bold": detail["is_bold"],
                    "heading_level": level,
                    "page": detail["page"],
                    "bbox": detail["bbox"]
                })
        
        return {
            "min_size": min(font_sizes),
            "max_size": max(font_sizes),
            "avg_size": avg_size,
            "common_size": common_size,
            "font_histogram": dict(sorted_sizes),
            "heading_candidates": heading_candidates
        }
    
    def is_likely_heading(self, text, font_size=None, avg_font_size=None, is_bold=False, font_stats=None):
        """
        Determine if a text block is likely a heading based on content, font size and style
        
        Args:
            text: The text to analyze
            font_size: Font size of this text (if available)
            avg_font_size: Average font size in the document (if available)
            is_bold: Whether the text is in bold font (if available)
            font_stats: Full font statistics from analyze_document_fonts (if available)
            
        Returns:
            Boolean indicating if the text is likely a heading and estimated heading level
        """
        # Short text sanity check
        text = text.strip()
        if not text:
            return False, None
            
        if len(text) > 150:  # Headings shouldn't be too long
            return False, None
        
        # Initialize result and heading level
        is_heading = False
        heading_level = None
        
        # Font size check (if available)
        if font_size and avg_font_size:
            size_ratio = font_size / avg_font_size
            
            # Title/H1
            if size_ratio >= self.font_title_threshold:
                is_heading = True
                heading_level = 1
            # Subtitle/H2
            elif size_ratio >= self.font_heading_threshold:
                is_heading = True
                heading_level = 2
            # Potential section heading if bold and above average size
            elif is_bold and size_ratio >= 1.0:
                is_heading = True
                heading_level = 3
                
        # If we haven't determined it's a heading by font, check patterns
        if not is_heading:
            # Pattern matching for heading-like text
            for pattern in self.heading_patterns:
                if pattern.match(text):
                    is_heading = True
                    # Default to level 3 for pattern-matched headings 
                    # unless we have better information
                    heading_level = heading_level or 3
                    break
                    
            # Additional heuristics for headings
            if not is_heading:
                # Label-like headings ending with colon
                if text.endswith(':') and len(text.split()) < 7:
                    is_heading = True
                    heading_level = heading_level or 4
                
                # Title Case with 2-6 words
                elif len(text) < 50 and text[0].isupper() and not text.endswith('.'):
                    words = text.split()
                    if 2 <= len(words) <= 6 and all(w[0].isupper() for w in words if w[0].isalpha()):
                        is_heading = True
                        heading_level = heading_level or 4
        
        # Check if this text matches any heading candidates in font_stats
        if font_stats and "heading_candidates" in font_stats:
            for candidate in font_stats["heading_candidates"]:
                if text == candidate["text"]:
                    is_heading = True
                    # Use the level from font analysis if available
                    heading_level = candidate["heading_level"]
                    break
                    
        return is_heading, heading_level
    
    def is_meaningful_content(self, text, is_heading_candidate=False):
        """Determine if text is meaningful content versus noise"""
        text = text.strip()
        if not text:
            return False

        # Special exception for heading candidates
        if is_heading_candidate:
            # For headings, we just need non-empty text, but still check for problematic characters
            non_ascii = sum(1 for c in text if ord(c) > 127)
            if non_ascii > len(text) * 0.3:  # More than 30% non-ASCII
                return False
            return True

        # For regular content, apply stricter rules
        # Check for minimum length
        if len(text) < self.min_content_length:
            return False

        # Check for minimum words
        words = [w for w in text.split() if w.strip(string.punctuation)]
        if len(words) < self.min_words_for_content:
            return False

        # Check for high proportion of problematic characters
        non_ascii = sum(1 for c in text if ord(c) > 127)
        if non_ascii > len(text) * 0.3:  # More than 30% non-ASCII
            return False

        return True
        
    def _group_content_by_headings(self, page_content, respect_heading_levels=True):
        """
        Group content items under their respective headings
        
        Args:
            page_content: Dictionary containing blocks of content
            respect_heading_levels: Whether to create a hierarchical structure based on heading levels
            
        Returns:
            List of content sections organized by headings
        """
        if not page_content.get("blocks"):
            return []
            
        if not respect_heading_levels:
            # Simple flat structure (original implementation)
            current_heading = None
            structured_content = []
            
            for block in page_content["blocks"]:
                if block.get("type") == "heading":
                    current_heading = {
                        "heading": block["text"],
                        "content": [],
                        "font_size": block.get("font_size"),
                        "position": block.get("position"),
                        "heading_level": block.get("heading_level", 3)  # Default to level 3
                    }
                    structured_content.append(current_heading)
                elif current_heading is not None:
                    current_heading["content"].append(block["text"])
                else:
                    # Content without a heading
                    structured_content.append({
                        "heading": None,
                        "content": [block["text"]],
                        "position": block.get("position")
                    })
                    
            return structured_content
        else:
            # Create hierarchical structure based on heading levels
            content_tree = {
                "heading": "Document",
                "heading_level": 0,
                "content": [],
                "children": []
            }
            
            # Current section pointers at each level
            current_sections = {0: content_tree}
            
            for block in page_content["blocks"]:
                if block.get("type") == "heading":
                    level = block.get("heading_level", 3)  # Default to level 3
                    
                    # Create new section
                    new_section = {
                        "heading": block["text"],
                        "heading_level": level,
                        "content": [],
                        "children": [],
                        "font_size": block.get("font_size"),
                        "position": block.get("position")
                    }
                    
                    # Find parent section (closest section with lower level)
                    parent_level = max([l for l in current_sections.keys() if l < level], default=0)
                    parent = current_sections[parent_level]
                    
                    # Add to parent's children
                    parent["children"].append(new_section)
                    
                    # Update current section for this level
                    current_sections[level] = new_section
                    
                    # Clear any deeper levels
                    for deeper_level in list(current_sections.keys()):
                        if deeper_level > level:
                            del current_sections[deeper_level]
                            
                else:
                    # Regular content - add to deepest current section
                    deepest_level = max(current_sections.keys())
                    current_sections[deepest_level]["content"].append(block["text"])
            
            # Function to flatten the tree for simple output
            def flatten_tree(node, sections=None):
                if sections is None:
                    sections = []
                
                if node["heading"] != "Document" or node["content"]:
                    sections.append({
                        "heading": node["heading"] if node["heading"] != "Document" else None,
                        "content": node["content"],
                        "heading_level": node["heading_level"],
                        "font_size": node.get("font_size"),
                        "position": node.get("position")
                    })
                
                for child in node["children"]:
                    flatten_tree(child, sections)
                    
                return sections
                
            return flatten_tree(content_tree)
            
    def extract_content_with_cleaning(self, pdf_path, output_dir=None, extract_images=True, extract_tables=True, respect_heading_hierarchy=True):
        """
        Enhanced extraction with text cleaning and better structure detection

        This method extends the parent's extract_content method with additional processing:
        - Cleans text to remove noise and unidentified symbols
        - Better detects headings and their relationship to content using font analysis
        - Filters out irrelevant short text fragments
        - Creates hierarchical structure based on heading levels

        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to save extracted images (optional)
            extract_images: Boolean indicating whether to extract images
            extract_tables: Boolean indicating whether to extract tables
            respect_heading_hierarchy: Whether to create hierarchical structure based on heading levels

        Returns:
            Dictionary containing all extracted and cleaned content, formatted for analyze_content
        """
        # First use the parent class to extract the raw content
        raw_result = super().extract_content(pdf_path, output_dir, extract_images, extract_tables)

        # Handle extraction errors
        if "error" in raw_result:
            return raw_result

        # Now enhance and clean the extracted content
        enhanced_result = raw_result.copy()

        try:
            # Reopen the PDF for font analysis
            pdf_doc = fitz.open(pdf_path)

            # Perform detailed font analysis
            self.logger.info("Performing detailed font analysis...")
            font_stats = self.analyze_document_fonts(pdf_doc)
            avg_font_size = font_stats["avg_size"]

            # Add font statistics to the result
            enhanced_result["font_statistics"] = {
                "avg_size": font_stats["avg_size"],
                "common_size": font_stats["common_size"],
                "min_size": font_stats["min_size"],
                "max_size": font_stats["max_size"],
                "size_distribution": font_stats["font_histogram"]
            }

            # Add heading candidates from font analysis
            enhanced_result["detected_headings"] = font_stats["heading_candidates"]

            # Process and clean each page
            enhanced_pages = []
            all_text = []

            # Create content_blocks list that analyze_content expects
            content_blocks = []

            # Create sections dictionary that analyze_content expects
            sections = {}
            current_section_id = None
            current_section_content = []

            for page in raw_result["pages"]:
                page_number = page["page_number"]
                enhanced_page = {
                    "page_number": page_number,
                    "blocks": []
                }

                # Process each text block
                for block in page["blocks"]:
                    # Clean the text
                    cleaned_text = self.clean_text(block["text"])

                    # Extract font information if available
                    font_size = None
                    is_bold = False

                    if "position" in block:
                        # Try to match this block with font information from our analysis
                        block_matches = []
                        block_pos = block["position"]

                        # Look for heading candidates that might match this block
                        for candidate in font_stats["heading_candidates"]:
                            if cleaned_text in candidate["text"] or candidate["text"] in cleaned_text:
                                block_matches.append(candidate)

                        if block_matches:
                            # Use font information from the best match
                            best_match = block_matches[0]  # Simple approach: just take first match
                            font_size = best_match["font_size"]
                            is_bold = best_match["is_bold"]
                        else:
                            # Fallback: estimate font size from position data
                            height = block_pos["y1"] - block_pos["y0"]
                            # Rough heuristic: font size ~ 1/2 of block height for single line
                            font_size = height / 2

                    # First check if this might be a heading
                    is_heading_candidate = self.is_likely_heading(
                        cleaned_text, 
                        font_size, 
                        avg_font_size,
                        is_bold,
                        font_stats
                    )[0]  # Just get the boolean, ignore the heading level for now

                    # Check if it's meaningful content, with special handling for headings
                    if not self.is_meaningful_content(cleaned_text, is_heading_candidate):
                        continue
                    
                    # Now get the full heading info including level
                    is_heading, heading_level = self.is_likely_heading(
                        cleaned_text, 
                        font_size, 
                        avg_font_size,
                        is_bold,
                        font_stats
                    )

                    # Create enhanced block
                    enhanced_block = {
                        "text": cleaned_text,
                        "type": "heading" if is_heading else "content",
                        "position": block.get("position"),
                        "font_size": font_size,
                        "is_bold": is_bold
                    }
    

                    # Add heading level if applicable
                    if is_heading and heading_level:
                        enhanced_block["heading_level"] = heading_level

                    # Keep any special content (math/chemical)
                    if "math_expressions" in block:
                        enhanced_block["math_expressions"] = block["math_expressions"]
                    if "chemical_content" in block:
                        enhanced_block["chemical_content"] = block["chemical_content"]

                    # Add to content_blocks (needed by analyze_content)
                    content_block = {
                        "text": cleaned_text,
                        "type": enhanced_block["type"],
                        "page": page_number,
                        "position": block.get("position")
                    }

                    if is_heading:
                        content_block["heading_level"] = heading_level

                        # Start a new section when we encounter a heading
                        if current_section_id:
                            # Save the previous section
                            sections[current_section_id] = "\n".join(current_section_content)

                        # Create new section ID (simple approach: use heading text)
                        current_section_id = f"section_{len(sections) + 1}_{cleaned_text[:20].replace(' ', '_')}"
                        current_section_content = [cleaned_text]
                    else:
                        # Add content to current section if there is one
                        if current_section_id:
                            current_section_content.append(cleaned_text)

                    # Add special content to content_block if available
                    if "math_expressions" in enhanced_block:
                        content_block["math_expressions"] = enhanced_block["math_expressions"]
                    if "chemical_content" in enhanced_block:
                        content_block["chemical_content"] = enhanced_block["chemical_content"]

                    content_blocks.append(content_block)
                    enhanced_page["blocks"].append(enhanced_block)
                    all_text.append(cleaned_text)

                # Group content by headings
                enhanced_page["structured_content"] = self._group_content_by_headings(
                    enhanced_page, 
                    respect_heading_levels=respect_heading_hierarchy
                )
                enhanced_pages.append(enhanced_page)

            # Save last section if it exists
            if current_section_id and current_section_content:
                sections[current_section_id] = "\n".join(current_section_content)

            # Update the result with enhanced pages
            enhanced_result["pages"] = enhanced_pages
            enhanced_result["text_content"] = "\n\n".join(all_text)

            # Add the expected keys for analyze_content
            enhanced_result["content_blocks"] = content_blocks
            enhanced_result["sections"] = sections

            # Close the document
            pdf_doc.close()

            return enhanced_result

        except Exception as e:
            self.logger.error(f"Error in enhanced extraction: {e}")
            import traceback
            traceback.print_exc()
            # Fall back to the original result if enhancement fails
            return raw_result
    
    # Override in EnhancedPDFExtractor
    def identify_block_type(self, text, position=None, font_info=None):
        """
        Enhanced block type identification using font information and patterns
        
        Args:
            text: Text content to analyze
            position: Optional position information
            font_info: Optional font information dictionary with font size, weight, etc.
            
        Returns:
            Dictionary with block type and additional metadata
        """
        text = text.strip()
        result = {
            "type": "text",
            "heading_level": None
        }
        
        # First clean the text
        cleaned_text = self.clean_text(text)
        
        # Determine if this could be a heading based on text patterns and font
        avg_font_size = font_info.get("avg_size") if font_info else None
        font_size = font_info.get("size") if font_info else None
        is_bold = font_info.get("is_bold", False) if font_info else False
        
        # Check if this might be a heading
        is_heading_candidate, _ = self.is_likely_heading(
            cleaned_text, 
            font_size, 
            avg_font_size,
            is_bold,
            font_info
        )
        
        # Skip if not meaningful after cleaning, with special handling for heading candidates
        if not self.is_meaningful_content(cleaned_text, is_heading_candidate):
            result["type"] = "noise"
            return result
        
        # Now do the full heading determination
        is_heading, heading_level = self.is_likely_heading(
            cleaned_text, 
            font_size, 
            avg_font_size,
            is_bold,
            font_info
        )
        
        if is_heading:
            result["type"] = "heading"
            result["heading_level"] = heading_level
        
        # Check for special content types from parent class
        parent_type = super().identify_block_type(cleaned_text, position)
        if parent_type in ["math", "chemical"]:
            result["type"] = parent_type
        
        return result
    
    def organize_blocks_by_structure(self, blocks, respect_heading_levels=True):
        """
        Organize blocks into a hierarchical structure based on headings
        
        Args:
            blocks: List of content blocks with type and text
            respect_heading_levels: Whether to create a hierarchical structure
            
        Returns:
            Structured content organized by headings
        """
        if not blocks:
            return []
            
        if not respect_heading_levels:
            # Simple flat structure
            current_heading = None
            structured_content = []
            
            for block in blocks:
                if block.get("type") == "heading":
                    current_heading = {
                        "heading": block["text"],
                        "content": [],
                        "font_size": block.get("font_size"),
                        "position": block.get("position"),
                        "heading_level": block.get("heading_level", 3)
                    }
                    structured_content.append(current_heading)
                elif current_heading is not None:
                    current_heading["content"].append(block["text"])
                else:
                    # Content without a heading
                    structured_content.append({
                        "heading": None,
                        "content": [block["text"]],
                        "position": block.get("position")
                    })
                    
            return structured_content
        else:
            # Create hierarchical structure based on heading levels
            # Implementation from original _group_content_by_headings method
            content_tree = {
                "heading": "Document",
                "heading_level": 0,
                "content": [],
                "children": []
            }
            
            # Current section pointers at each level
            current_sections = {0: content_tree}
            
            for block in blocks:
                if block.get("type") == "heading":
                    level = block.get("heading_level", 3)
                    
                    # Create new section
                    new_section = {
                        "heading": block["text"],
                        "heading_level": level,
                        "content": [],
                        "children": [],
                        "font_size": block.get("font_size"),
                        "position": block.get("position")
                    }
                    
                    # Find parent section (closest section with lower level)
                    parent_level = max([l for l in current_sections.keys() if l < level], default=0)
                    parent = current_sections[parent_level]
                    
                    # Add to parent's children
                    parent["children"].append(new_section)
                    
                    # Update current section for this level
                    current_sections[level] = new_section
                    
                    # Clear any deeper levels
                    for deeper_level in list(current_sections.keys()):
                        if deeper_level > level:
                            del current_sections[deeper_level]
                            
                else:
                    # Regular content - add to deepest current section
                    deepest_level = max(current_sections.keys())
                    current_sections[deepest_level]["content"].append(block["text"])
            
            # Function to flatten the tree for simple output
            def flatten_tree(node, sections=None):
                if sections is None:
                    sections = []
                
                if node["heading"] != "Document" or node["content"]:
                    sections.append({
                        "heading": node["heading"] if node["heading"] != "Document" else None,
                        "content": node["content"],
                        "heading_level": node["heading_level"],
                        "font_size": node.get("font_size"),
                        "position": node.get("position")
                    })
                
                for child in node["children"]:
                    flatten_tree(child, sections)
                    
                return sections
                
            return flatten_tree(content_tree)        


In [6]:
#     def segment_text(self, text, min_length=None, min_words=None):
#             """
#             Enhanced text segmentation with cleaning

#             Args:
#                 text: Text to segment
#                 min_length: Minimum length (defaults to self.min_content_length)
#                 min_words: Minimum words (defaults to self.min_words_for_content)

#             Returns:
#                 List of cleaned text segments
#             """
#             # Use class defaults if not specified
#             min_length = min_length or self.min_content_length
#             min_words = min_words or self.min_words_for_content

#             # Clean the text first
#             cleaned_text = self.clean_text(text)

#             # Use parent's segmentation but with our parameters
#             segments = super().segment_text(cleaned_text, min_length, min_words)

#             # Further filter segments to ensure they're meaningful
#             return [seg for seg in segments if self.is_meaningful_content(seg)]

#     def classify_block(self, text, font_info=None, font_stats=None):
#         """
#         Enhanced block classification using font information
#         Args:
#             text: Text to classify
#             font_info: Dictionary with font information
#             font_stats: Document-wide font statistics
#         Returns:
#             Dictionary with classification details
#         """
#         # First check if it's a special content type using parent's method
#         base_classification = super().classify_block(text)
#         if base_classification["type"] in ["math", "chemical"]:
#             return base_classification
#         # Clean the text
#         cleaned_text = self.clean_text(text)
#         # Extract font details
#         font_size = font_info.get("font_size") if font_info else None
#         is_bold = font_info.get("is_bold", False) if font_info else False
#         avg_font_size = font_stats.get("avg_size") if font_stats else None
#         # Check if it's a heading
#         is_heading, heading_level = self.is_likely_heading(
#             cleaned_text, font_size, avg_font_size, is_bold, font_stats
#         )
#         if is_heading:
#             return {
#                 "type": "heading",
#                 "content": cleaned_text,
#                 "heading_level": heading_level,
#                 "font_size": font_size,
#                 "is_bold": is_bold
#             }
#         # Default to content
#         return {
#             "type": "content",
#             "content": cleaned_text,
#             "font_size": font_size,
#             "is_bold": is_bold
#         }
#     def identify_structure(self, blocks, page_idx=0, font_stats=None):
#         """
#         Enhanced structure identification with font analysis
#         Args:
#             blocks: List of text blocks
#             page_idx: Page index
#             font_stats: Document-wide font statistics
#         Returns:
#             List of structured content with hierarchical organization
#         """
#         # First get basic structure from parent
#         basic_structure = super().identify_structure(blocks, page_idx)
#         # Now enhance it with our heading detection and organization
#         enhanced_blocks = []
#         for block in basic_structure:
#             font_info = {
#                 "font_size": block.get("font_size"),
#                 "is_bold": block.get("is_bold", False)
#             }
#             # Reclassify blocks with our enhanced logic
#             classification = self.classify_block(block["text"], font_info, font_stats)
#             enhanced_block = block.copy()
#             enhanced_block["type"] = classification["type"]
#             # Add heading level if applicable
#             if classification["type"] == "heading" and "heading_level" in classification:
#                 enhanced_block["heading_level"] = classification["heading_level"]
#             enhanced_blocks.append(enhanced_block)
#         # Group content by headings
#         structured_content = self._group_content_by_headings({
#             "blocks": enhanced_blocks
#         })
#         return structured_content

#     def save_extraction_result(self, result, output_path_base, formats=None):
#             """
#             Save extraction result in multiple formats with a single call

#             Args:
#                 result: The extraction result dictionary
#                 output_path_base: Base path for output files (without extension)
#                 formats: List of formats to save ('json', 'text', 'markdown') or None for all

#             Returns:
#                 Dictionary with paths to all generated files
#             """
#             if formats is None:
#                 formats = ['json', 'text', 'markdown']

#             output_files = {}

#             # Ensure formats is a list
#             if isinstance(formats, str):
#                 formats = [formats]

#             # Save in each requested format
#             for fmt in formats:
#                 if fmt.lower() == 'json':
#                     json_path = f"{output_path_base}.json"
#                     try:
#                         with open(json_path, 'w', encoding='utf-8') as f:
#                             json.dump(result, f, indent=2, ensure_ascii=False)
#                         output_files['json'] = json_path
#                         self.logger.info(f"JSON result saved to {json_path}")
#                     except Exception as e:
#                         self.logger.error(f"Error saving JSON: {e}")

#                 elif fmt.lower() == 'text':
#                     text_path = f"{output_path_base}.txt"
#                     try:
#                         # Use full_text if available, otherwise compile from pages
#                         if "full_text" in result:
#                             full_text = result["full_text"]
#                         else:
#                             # Generate full text from pages
#                             text_blocks = []
#                             for page in result.get("pages", []):
#                                 for block in page.get("blocks", []):
#                                     if "text" in block:
#                                         text_blocks.append(block["text"])
#                             full_text = "\n\n".join(text_blocks)

#                         with open(text_path, 'w', encoding='utf-8') as f:
#                             f.write(full_text)
#                         output_files['text'] = text_path
#                         self.logger.info(f"Plain text saved to {text_path}")
#                     except Exception as e:
#                         self.logger.error(f"Error saving text: {e}")

#                 elif fmt.lower() == 'markdown':
#                     markdown_path = f"{output_path_base}.md"
#                     try:
#                         markdown_content = convert_to_markdown(result,image_output_dir="output_markdown/")
#                         with open(markdown_path, 'w', encoding='utf-8') as f:
#                             f.write(markdown_content)
#                         output_files['markdown'] = markdown_path
#                         self.logger.info(f"Markdown content saved to {markdown_path}")
#                     except Exception as e:
#                         self.logger.error(f"Error saving markdown: {e}")

#             return output_files

#         def extract_and_save(self, pdf_path, output_path_base, output_dir=None, 
#                             extract_images=True, extract_tables=True, formats=None):
#             """
#             Extract PDF content and save in multiple formats with a single call

#             Args:
#                 pdf_path: Path to the PDF file
#                 output_path_base: Base path for output files (without extension)
#                 output_dir: Directory to save extracted images (optional)
#                 extract_images: Boolean indicating whether to extract images
#                 extract_tables: Boolean indicating whether to extract tables
#                 formats: List of formats to save ('json', 'text', 'markdown') or None for all

#             Returns:
#                 Dictionary containing extraction result and output file paths
#             """
#             # Extract content with cleaning
#             result = self.extract_content_with_cleaning(
#                 pdf_path, output_dir, extract_images, extract_tables
#             )

#             # Save in all requested formats
#             output_files = self.save_extraction_result(result, output_path_base, formats)

#             # Add output files to result
#             result['output_files'] = output_files

#             return result
# # Example usage

In [4]:
class LiteraryWorkAnalyzer:
    """Support class for analyzing literary works including Shakespeare plays"""
    def __init__(self, work_subtype=None):
        """
        Initialize with optional work subtype specification
        
        Args:
            work_subtype: Subtype of literary work (e.g., 'play:shakespeare', 'novel', 'poem')
        """
        self.work_type = work_subtype.split(':')[0] if work_subtype and ':' in work_subtype else work_subtype
        self.work_subtype = work_subtype.split(':')[1] if work_subtype and ':' in work_subtype else None
        self.character_map = {}
        self.scene_breakdown = {}
        self.literary_devices = {
            'metaphor': [],
            'simile': [],
            'alliteration': [],
            'personification': [],
            'irony': [],
            'foreshadowing': []
        }
        
    def analyze_characters(self, text):
        """
        Identify and analyze characters in the text
        
        Args:
            text: Full text content to analyze
            
        Returns:
            Dictionary with character information
        """
        import re
        
        # First pass: identify potential character names from speech patterns
        # Look for patterns like "HAMLET:" or "ROMEO."
        character_pattern = r'([A-Z]{2,}[A-Z\s]*)[\.:]\s'
        potential_characters = set(re.findall(character_pattern, text))
        
        # Filter out stage directions often in all caps
        filtered_characters = [char for char in potential_characters 
                              if char not in ['EXIT', 'ENTER', 'EXEUNT', 'ALL']]
        
        character_data = {}
        for character in filtered_characters:
            # Count lines spoken
            line_count = len(re.findall(f"{character}[\.:]", text))
            
            # Find all speech by this character
            speech_pattern = f"{character}[\.:]\\s*(.*?)(?=([A-Z]{{2,}}[A-Z\\s]*[\\.:]|$))"
            speeches = re.findall(speech_pattern, text, re.DOTALL)
            speech_text = ' '.join([s[0] for s in speeches if s[0]])
            
            # Calculate word count and average speech length
            words = speech_text.split()
            word_count = len(words)
            avg_speech_length = word_count / max(1, line_count)
            
            character_data[character] = {
                'line_count': line_count,
                'word_count': word_count,
                'avg_speech_length': avg_speech_length,
                'sample_speech': speech_text[:200] + '...' if len(speech_text) > 200 else speech_text
            }
        
        self.character_map = character_data
        return character_data
        
    def identify_scenes(self, text):
        """
        Break down text into acts and scenes for plays
        
        Args:
            text: Full text content
            
        Returns:
            Dictionary with act/scene structure
        """
        import re
        
        acts_scenes = {}
        
        # For Shakespeare and similar structured plays
        if self.work_type == 'play':
            # Find act divisions
            act_pattern = r'ACT\s+([IVXLCDM]+|[0-9]+)(.+?)(?=ACT\s+[IVXLCDM]+|[0-9]+|$)'
            acts = re.findall(act_pattern, text, re.DOTALL)
            
            for act_num, act_content in acts:
                # Find scene divisions within this act
                scene_pattern = r'SCENE\s+([IVXLCDM]+|[0-9]+)(.+?)(?=SCENE\s+[IVXLCDM]+|[0-9]+|$)'
                scenes = re.findall(scene_pattern, act_content, re.DOTALL)
                
                act_key = f"ACT {act_num}"
                acts_scenes[act_key] = {}
                
                for scene_num, scene_content in scenes:
                    scene_key = f"SCENE {scene_num}"
                    
                    # Try to extract setting information
                    setting_match = re.search(r'\[(.*?)\]', scene_content)
                    setting = setting_match.group(1) if setting_match else "Unknown setting"
                    
                    # Get characters in scene
                    character_pattern = r'([A-Z]{2,}[A-Z\s]*)[\.:]\s'
                    scene_characters = set(re.findall(character_pattern, scene_content))
                    
                    acts_scenes[act_key][scene_key] = {
                        'setting': setting,
                        'characters': list(scene_characters),
                        'length': len(scene_content),
                        'word_count': len(scene_content.split())
                    }
        
        # For novels and other chapter-based works
        elif self.work_type == 'novel':
            # Find chapter divisions
            chapter_pattern = r'CHAPTER\s+([IVXLCDM]+|[0-9]+)(.+?)(?=CHAPTER\s+[IVXLCDM]+|[0-9]+|$)'
            chapters = re.findall(chapter_pattern, text, re.DOTALL)
            
            for chapter_num, chapter_content in chapters:
                chapter_key = f"CHAPTER {chapter_num}"
                
                # Try to extract title if available
                title_match = re.search(r'CHAPTER\s+[IVXLCDM0-9]+\.\s+(.*?)[\n\r]', chapter_content)
                title = title_match.group(1) if title_match else "Untitled"
                
                acts_scenes[chapter_key] = {
                    'title': title,
                    'length': len(chapter_content),
                    'word_count': len(chapter_content.split())
                }
        
        self.scene_breakdown = acts_scenes
        return acts_scenes
    
    def detect_literary_devices(self, text):
        """
        Identify common literary devices in the text
        
        Args:
            text: Text to analyze
            
        Returns:
            Dictionary with literary devices found
        """
        import re
        
        devices = {
            'metaphor': [],
            'simile': [],
            'alliteration': [],
            'personification': []
        }
        
        # Break text into sentences for analysis
        sentences = re.split(r'[.!?]', text)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        # Find similes (like/as comparisons)
        simile_patterns = [
            r'\b\w+\s+like\s+\w+',  # "white like snow"
            r'\b\w+\s+as\s+\w+\s+as\s+\w+'  # "cold as winter night"
        ]
        
        for sentence in sentences:
            # Check for similes
            for pattern in simile_patterns:
                matches = re.findall(pattern, sentence, re.IGNORECASE)
                if matches:
                    devices['simile'].extend(matches)
            
            # Check for alliteration (3+ words starting with same consonant)
            words = sentence.split()
            if len(words) >= 3:
                for i in range(len(words) - 2):
                    if (words[i] and words[i+1] and words[i+2] and
                        len(words[i]) > 0 and len(words[i+1]) > 0 and len(words[i+2]) > 0):
                        first_chars = [w[0].lower() for w in [words[i], words[i+1], words[i+2]]]
                        if (first_chars[0] == first_chars[1] == first_chars[2] and 
                            first_chars[0] not in 'aeiou'):
                            devices['alliteration'].append(' '.join([words[i], words[i+1], words[i+2]]))
            
            # Basic personification detection (non-human things doing human actions)
            personification_verbs = ['spoke', 'whispered', 'thought', 'laughed', 'cried', 'smiled', 'frowned']
            personification_subjects = ['sky', 'moon', 'sun', 'wind', 'tree', 'forest', 'sea', 'ocean', 'mountain']
            
            for subject in personification_subjects:
                for verb in personification_verbs:
                    if f"{subject} {verb}" in sentence.lower():
                        devices['personification'].append(f"{subject} {verb}")
        
        self.literary_devices = devices
        return devices
        
    def analyze_verse_patterns(self, text):
        """
        Analyze verse patterns in poetic works including Shakespeare
        
        Args:
            text: Text to analyze
            
        Returns:
            Dictionary with verse analysis
        """
        # Break text into lines
        lines = text.split('\n')
        lines = [line.strip() for line in lines if line.strip()]
        
        verse_analysis = {
            'iambic_pentameter_count': 0,
            'rhyming_couplets': 0,
            'blank_verse_sections': 0,
            'sonnets': 0,
            'average_syllables_per_line': 0
        }
        
        total_syllables = 0
        
        # Count syllables for each line
        for i, line in enumerate(lines):
            # Simple syllable counting - can be improved
            syllable_count = self._count_syllables(line)
            total_syllables += syllable_count
            
            # Check for potential iambic pentameter (10 syllables)
            if 9 <= syllable_count <= 11:
                verse_analysis['iambic_pentameter_count'] += 1
                
            # Check for potential rhyming couplets
            if i > 0 and len(lines[i-1]) > 2 and len(line) > 2:
                if lines[i-1][-2:].lower() == line[-2:].lower():
                    verse_analysis['rhyming_couplets'] += 1
        
        # Calculate average syllables per line
        if lines:
            verse_analysis['average_syllables_per_line'] = total_syllables / len(lines)
            
        return verse_analysis
    
    def _count_syllables(self, word):
        """
        Simple syllable counter for English text
        
        Args:
            word: Word to count syllables for
            
        Returns:
            Estimated syllable count
        """
        # Remove punctuation
        word = ''.join(c for c in word if c.isalpha() or c.isspace())
        
        # Split into individual words
        words = word.split()
        count = 0
        
        for w in words:
            w = w.lower()
            
            # Count vowel groups
            vowels = "aeiouy"
            on_vowel = False
            in_word = False
            syllables = 0
            
            # Count syllables in the word
            for char in w:
                if char in vowels:
                    if not on_vowel:
                        syllables += 1
                        on_vowel = True
                else:
                    on_vowel = False
                    
            # Adjust for endings
            if w.endswith('e') and not w.endswith('le'):
                syllables -= 1
                
            if syllables == 0:
                syllables = 1
                
            count += syllables
            
        return count
    
    def detect_shakespearean_themes(self, text):
        """
        Identify common Shakespearean themes in the text
        
        Args:
            text: Text to analyze
            
        Returns:
            Dictionary with theme analysis
        """
        # Define common Shakespearean themes and related keywords
        themes = {
            'power_corruption': ['crown', 'throne', 'king', 'queen', 'rule', 'power', 'tyrant', 'ambition'],
            'love_romance': ['love', 'heart', 'passion', 'desire', 'kiss', 'marry', 'marriage', 'lovers'],
            'betrayal': ['betray', 'trust', 'loyal', 'faithful', 'deceive', 'traitor', 'backstab'],
            'appearance_reality': ['seem', 'appear', 'mask', 'disguise', 'truth', 'deception', 'reality'],
            'supernatural': ['ghost', 'witch', 'fairy', 'spirit', 'magic', 'dream', 'fate', 'omen'],
            'order_chaos': ['order', 'chaos', 'balance', 'harmony', 'discord', 'peace', 'war', 'tumult'],
            'revenge': ['revenge', 'vengeance', 'avenge', 'wrong', 'justice', 'blood', 'death']
        }
        
        text_lower = text.lower()
        theme_scores = {}
        
        # Score each theme based on keyword presence
        for theme, keywords in themes.items():
            # Count occurrences of theme keywords
            score = sum(text_lower.count(' ' + keyword + ' ') for keyword in keywords)
            theme_scores[theme] = score
        
        # Normalize scores by text length (per 10,000 words)
        word_count = len(text_lower.split())
        for theme in theme_scores:
            theme_scores[theme] = (theme_scores[theme] * 10000) / max(1, word_count)
        
        # Find the dominant themes (scores above average)
        avg_score = sum(theme_scores.values()) / len(theme_scores)
        dominant_themes = {theme: score for theme, score in theme_scores.items() if score > avg_score}
        
        return {
            'all_themes': theme_scores,
            'dominant_themes': dominant_themes
        }
    

In [5]:
class DomainRulesManager:
    """Centralized manager for domain-specific rules and patterns"""
    
    @staticmethod
    def get_domain_rules(domain):
        """
        Load domain-specific rules and patterns for content analysis.
        
        Args:
            domain: The domain to load rules for
            
        Returns:
            Dictionary of rules and patterns specific to the domain
        """
        import re
        
        if domain is None:
            return {}
            
        # Consolidated domain rules
        domain_rules = {
            'science': {
                'terminologies': ['experiment', 'hypothesis', 'theory', 'data'],
                'section_types': ['methodology', 'results', 'discussion', 'conclusion'],
                'entity_patterns': [r'[A-Z][a-z]+ [a-z]+ium', r'\d+\.\d+ [a-z]+'],
                'method_pattern': re.compile(r'(?i)(methods?|procedure|protocol)[:]\s*(.*?)(?=\n\n|\Z)'),
                'result_pattern': re.compile(r'(?i)(results?|findings)[:]\s*(.*?)(?=\n\n|\Z)'),
                'experiment_pattern': re.compile(r'(?i)(experiment|study|investigation)[\s\d]*[:]\s*(.*?)(?=\n\n|\Z)')
            },
            'math': {
                'terminologies': ['theorem', 'proof', 'lemma', 'corollary'],
                'section_types': ['definition', 'theorem', 'proof', 'example'],
                'entity_patterns': [r'[A-Z][a-z]+ (theorem|formula|equation)', r'equation \(\d+\)'],
                'theorem_pattern': re.compile(r'(?i)(theorem|lemma|corollary|proposition)\s*(\d+|[\w\s]+)[:]\s*(.*?)(?=\n\n|\Z|proof)'),
                'proof_pattern': re.compile(r'(?i)proof[:]\s*(.*?)(?=\n\n|\Z|q\.e\.d\.|□)'),
                'equation_pattern': re.compile(r'(?i)(?:\((\d+)\)|equation\s*(\d+))[:]\s*(.*?)(?=\n\n|\Z)')
            },
            'literature': {
                'terminologies': ['symbolism', 'metaphor', 'character', 'plot', 'theme', 'motif', 'foreshadowing', 'irony', 'allegory', 'setting'],
                'section_types': ['exposition', 'climax', 'resolution', 'analysis', 'chapter', 'prologue', 'epilogue', 'act', 'scene', 'soliloquy'],
                'entity_patterns': [
                    r'[A-Z][a-z]+ (symbolizes|represents)', 
                    r'theme of [a-z]+', 
                    r'Chapter [IVXLCDM]+', 
                    r'Chapter \d+',
                    r'Act [IVXLCDM]+', 
                    r'Scene [IVXLCDM]+', 
                    r'Enter [A-Z][a-z]+', 
                    r'\[Exit[^\]]*\]', 
                    r'\[Aside\]'
                ],
                'character_pattern': re.compile(r'(?i)([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)[:]\s*["\'](.+?)["\']'),
                'theme_pattern': re.compile(r'(?i)(theme|motif|symbol)[\s\w]*[:]\s*(.*?)(?=\n\n|\Z)'),
                'plot_pattern': re.compile(r'(?i)(exposition|rising action|climax|falling action|resolution)[:]\s*(.*?)(?=\n\n|\Z)'),
                'subtypes': {
                    'play': {
                        'shakespeare': {
                            'terminologies': ['soliloquy', 'aside', 'iambic pentameter', 'sonnet', 'blank verse', 'couplet', 'quatrain', 'folio', 'quarto'],
                            'themes': ['power_corruption', 'love_romance', 'betrayal', 'appearance_reality', 'supernatural', 'order_chaos', 'revenge']
                        },
                        'modern': {
                            'terminologies': ['dialogue', 'stage direction', 'monologue', 'act', 'scene']
                        }
                    },
                    'novel': {
                        'terminologies': ['narrator', 'protagonist', 'antagonist', 'setting', 'plot', 'chapter']
                    },
                    'poem': {
                        'terminologies': ['stanza', 'verse', 'rhyme', 'meter', 'rhythm', 'sonnet', 'ode', 'elegy']
                    }
                }
            },
            'chemistry': {
                'compound_pattern': re.compile(r'(?i)([A-Z][a-z]?\d*)+'),
                'reaction_pattern': re.compile(r'(?i)(.+?)\s*(?:→|⟶|⇒|⇌|⇆)\s*(.+)'),
                'condition_pattern': re.compile(r'(?i)(?:at|under|in)\s+(\d+(?:\.\d+)?)\s*(?:°C|K|atm|bar|MPa|kPa)')
            },
            'biology': {
                'species_pattern': re.compile(r'(?i)([A-Z][a-z]+\s+[a-z]+)'),
                'gene_pattern': re.compile(r'(?i)([A-Z0-9]+)(?:\s+gene)?'),
                'pathway_pattern': re.compile(r'(?i)([\w\s]+)(?:pathway|cycle|cascade)')
            },
            'physics': {
                'law_pattern': re.compile(r'(?i)([\w\s]+)(?:\'s)?\s+(?:law|principle|theory)'),
                'units_pattern': re.compile(r'(\d+(?:\.\d+)?)\s*(m|kg|s|A|K|mol|cd|N|Pa|J|W|V)(?:\^?(-?\d+))?'),
                'equation_pattern': re.compile(r'(?i)(?:\((\d+)\)|equation\s*(\d+))[:]\s*(.*?)(?=\n\n|\Z)')
            }
        }
        
        return domain_rules.get(domain.lower(), {})


In [6]:
import re

class TextProcessor:
    """Provides text manipulation and preprocessing utilities"""
    
    def __init__(self):
        self.stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 
                     'by', 'this', 'with', 'i', 'you', 'it', 'not', 'or', 'be', 'are', 
                     'from', 'at', 'as', 'your', 'have', 'has', 'had', 'was', 'were', 
                     'they', 'their', 'them', 'we', 'our', 'us', 'he', 'she', 'his', 
                     'her', 'him', 'an', 'can', 'will', 'just', 'do', 'does', 'did',
                     'but', 'if', 'so', 'what', 'when', 'where', 'how', 'all', 'any',
                     'both', 'each', 'few', 'more', 'most', 'some', 'such', 'no', 'nor',
                     'too', 'very', 'one', 'every', 'least', 'less', 'many', 'now',
                     'ever', 'never', 'say', 'says', 'said', 'also', 'like', 'time'}
    
    def segment_text_block(self, text):
        """Break a block of text into logical segments."""
        segments = []
        paragraphs = text.split('\n\n')
        
        for paragraph in paragraphs:
            if not paragraph.strip():
                continue
                
            segments.append({
                'type': 'paragraph',  # Default type
                'content': paragraph.strip()
            })
            
        return segments
    
    def find_term_positions(self, text, term):
        """Find all positions of a term in text."""
        positions = []
        pattern = r'\b' + re.escape(term) + r'\b'

        for match in re.finditer(pattern, text, re.IGNORECASE):
            positions.append(match.start())

        return positions
    
    def detect_poetic_structure(self, text):
        """Detect if text has poetic structure based on line length patterns"""
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        if not lines:
            return False
        
        # Count lines with similar length (potential verse structure)
        line_lengths = [len(line) for line in lines]
        
        # Get average line length
        avg_length = sum(line_lengths) / len(line_lengths)
        
        # Count lines within 50% of average length (consistent structure)
        consistent_lines = sum(1 for length in line_lengths if 0.5*avg_length <= length <= 1.5*avg_length)
        
        # If at least 60% of lines have consistent length, likely poetry
        return (consistent_lines / len(line_lengths)) > 0.6

    def find_term_locations(self, text, term):
        """
        Find locations of a term in the text.

        Args:
            text (str): The text to search in
            term (str): The term to locate

        Returns:
            List of positions (character indices) where the term appears
        """
        import re

        locations = []
        pattern = r'\b' + re.escape(term) + r'\b'

        for match in re.finditer(pattern, text, re.IGNORECASE):
            locations.append(match.start())

        return locations

    def count_term_occurrences(self, text, term):
        """
        Count occurrences of a term in text.

        Args:
            text (str): The text to search in
            term (str): The term to count

        Returns:
            Number of occurrences
        """
        import re
        # Use word boundaries to ensure we match whole words
        pattern = r'\b' + re.escape(term) + r'\b'
        return len(re.findall(pattern, text, re.IGNORECASE))

    def get_context_around_term(self, text, term, context_size=100):
        """
        Get text context around a term.

        Args:
            text (str): The text to search in
            term (str): The term to find context for
            context_size (int): Number of characters around the term

        Returns:
            String with context
        """
        import re

        # Find all occurrences of the term
        matches = list(re.finditer(r'\b' + re.escape(term) + r'\b', text, re.IGNORECASE))

        if not matches:
            return ""

        # Use the first occurrence
        match = matches[0]
        start_pos = max(0, match.start() - context_size)
        end_pos = min(len(text), match.end() + context_size)

        # Extract context
        context = text[start_pos:end_pos]

        # Add ellipsis if needed
        if start_pos > 0:
            context = "..." + context
        if end_pos < len(text):
            context = context + "..."

        return context

    def extract_noun_phrases(self, text):
        """
        Extract noun phrases from text.

        Args:
            text (str): Text to extract noun phrases from

        Returns:
            List of noun phrases
        """
        # Simple implementation using basic patterns
        import re

        # Pattern for simple noun phrases
        noun_phrase_pattern = re.compile(r'(?:[A-Z][a-z]+\s)?(?:[A-Z][a-z]+\s)?[A-Z]?[a-z]+')
        matches = noun_phrase_pattern.findall(text)

        # Filter short phrases and common words
        stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 'by', 'this', 'with'}
        phrases = [match for match in matches if len(match) > 4 and match.lower() not in stop_words]

        return phrases


class SemanticAnalyzer:
    """Handles semantic analysis of text content"""
    
    def __init__(self, domain_rules=None):
        self.domain_rules = domain_rules or {}
    
    def identify_segment_type(self, text):
        """Identify the type of a text segment."""
        text = text.strip().lower()
        
        # Check if it's a definition
        if text.startswith('definition') or ': is defined as' in text or 'refers to' in text:
            return 'definition'
            
        # Check if it's an example
        if text.startswith('example') or text.startswith('for example') or 'for instance' in text:
            return 'example'
            
        # Check if it's a theorem/proposition
        if text.startswith('theorem') or text.startswith('proposition') or text.startswith('lemma'):
            return 'theorem'
            
        # Check if it's a proof
        if text.startswith('proof') or text.endswith('q.e.d.') or text.endswith('□'):
            return 'proof'
            
        # Check domain-specific section types
        if 'section_types' in self.domain_rules:
            for section_type in self.domain_rules['section_types']:
                if section_type.lower() in text[:20]:
                    return section_type
        
        # Default to paragraph
        return 'paragraph'
    
    def generate_semantic_tags(self, text):
        """Generate semantic tags for a piece of text."""
        tags = []
        
        # Check for domain-specific terminology
        if 'terminologies' in self.domain_rules:
            for term in self.domain_rules['terminologies']:
                if term.lower() in text.lower():
                    tags.append(term)
        
        # Check if text contains numerical data
        if any(char.isdigit() for char in text):
            tags.append('numerical_data')
            
        # Check if text contains questions
        if '?' in text:
            tags.append('question')
            
        # Check if text contains lists
        if any(line.strip().startswith(('-', '•', '*')) for line in text.split('\n')):
            tags.append('list')
            
        return tags


class KeywordExtractor:
    """Extracts keywords and concepts from text"""
    
    def __init__(self, stop_words=None, domain_rules=None):
        self.stop_words = stop_words or set()
        self.domain_rules = domain_rules or {}
    
    def extract_keywords_and_concepts(self, text, domain=None, max_terms=20, min_word_length=3):
        """Extract key terms and concepts from text, optionally customized by domain."""
        # Split text into words and clean them
        if not text:
            return {'keywords': [], 'concepts': []}

        words = text.lower().split()
        words = [word.strip('.,;:()[]{}"\'-') for word in words if word.strip('.,;:()[]{}"\'-')]

        # Remove stop words and short words
        filtered_words = [word for word in words if word not in self.stop_words and len(word) > min_word_length]

        # Count word frequencies
        word_freq = {}
        for word in filtered_words:
            word_freq[word] = word_freq.get(word, 0) + 1

        # Get top keywords by frequency
        keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:max_terms]

        # Extract multi-word concepts (bigrams)
        bigrams = []
        for i in range(len(filtered_words) - 1):
            if filtered_words[i] and filtered_words[i+1]:
                bigram = filtered_words[i] + ' ' + filtered_words[i+1]
                bigrams.append(bigram)

        bigram_counts = {}
        for bigram in bigrams:
            bigram_counts[bigram] = bigram_counts.get(bigram, 0) + 1

        # Get top bigrams
        top_bigrams = sorted(bigram_counts.items(), key=lambda x: x[1], reverse=True)[:max_terms//2]

        # Extract domain-specific concepts if available
        domain_concepts = []
        if domain and self.domain_rules:
            if 'entity_patterns' in self.domain_rules:
                for pattern in self.domain_rules['entity_patterns']:
                    matches = re.findall(pattern, text)
                    domain_concepts.extend(matches)

        return {
            'keywords': [keyword for keyword, _ in keywords],
            'bigrams': [bigram for bigram, _ in top_bigrams],
            'domain_concepts': domain_concepts,
            'word_frequencies': word_freq,
            'raw_keywords': keywords,  # Include frequency information
            'raw_bigrams': top_bigrams  # Include frequency information
        }
    
    def extract_keywords_with_positions(self, text, domain=None, max_terms=20, min_word_length=3):
        """Extract keywords with their positions in the text."""
        # Get base extraction results
        base_results = self.extract_keywords_and_concepts(text, domain, max_terms, min_word_length)

        # Add position information
        terms_with_positions = {}
        text_processor = TextProcessor()

        # Find positions for keywords
        for keyword in base_results['keywords']:
            positions = text_processor.find_term_positions(text, keyword)
            if positions:
                terms_with_positions[keyword] = positions

        # Find positions for bigrams
        for bigram in base_results['bigrams']:
            positions = text_processor.find_term_positions(text, bigram)
            if positions:
                terms_with_positions[bigram] = positions

        # Find positions for domain concepts
        for concept in base_results['domain_concepts']:
            positions = text_processor.find_term_positions(text, concept)
            if positions:
                terms_with_positions[concept] = positions

        # Add position information to results
        base_results['term_positions'] = terms_with_positions

        return base_results
    
    def extract_key_terms_with_importance(self, text, domain_rules=None, max_terms=20, min_word_length=3):
        """Extract key terms with importance scores."""
        result = self.extract_keywords_and_concepts(text, domain_rules, max_terms*2, min_word_length)
        term_importance = {}
        
        # Add single words with their importance
        for term, count in result['raw_keywords']:
            # Calculate importance using frequency
            term_importance[term] = count / max(1, len(text.split())) * 0.8  # Single words get slightly lower weight
        
        # Add bigrams with higher weight
        for bigram, count in result['raw_bigrams']:
            if count > 1:  # Only include bigrams that occur more than once
                term_importance[bigram] = count / max(1, len(result.get('bigrams', []))) * 1.2  # Bigrams get higher weight
        
        # Add domain-specific concepts with highest weight
        for concept in result.get('domain_concepts', []):
            # Domain concepts get highest weight
            term_importance[concept] = 1.5
        
        # Sort by importance and return top terms
        sorted_terms = sorted(term_importance.items(), key=lambda x: x[1], reverse=True)
        
        return sorted_terms[:max_terms]

    def count_bigram_frequency(self, text, bigram):
        """Count frequency of a bigram in text"""
        words = text.lower().split()
        count = 0
        bigram_parts = bigram.lower().split()
        
        if len(bigram_parts) != 2:
            return 0
            
        for i in range(len(words) - 1):
            if words[i] == bigram_parts[0] and words[i+1] == bigram_parts[1]:
                count += 1
        
        return count
    
    def get_ngrams(self, text, n=3):
        """Generate n-grams from text"""
        return [text[i:i+n] for i in range(len(text)-n+1)]
    
    def calculate_term_similarity(self, term_a, term_b):
        """
        Calculate similarity between two terms using n-grams.
        
        Args:
            term_a (str): First term
            term_b (str): Second term
            
        Returns:
            Similarity score between 0 and 1
        """
        # Convert to lower case
        term_a = term_a.lower()
        term_b = term_b.lower()
        
        # Generate character 3-grams
        ngrams_a = set(self.get_ngrams(term_a))
        ngrams_b = set(self.get_ngrams(term_b))
        
        # Calculate Jaccard similarity
        if not ngrams_a or not ngrams_b:
            return 0
        
        intersection = len(ngrams_a.intersection(ngrams_b))
        union = len(ngrams_a.union(ngrams_b))
        
        return intersection / union if union > 0 else 0

class SentimentAnalyzer:
    """Analyzes sentiment and tone in text"""
    
    def perform_sentiment_analysis(self, text):
        """Analyze tone and sentiment of content sections."""
        # Simple sentiment analysis implementation
        # In a real implementation, this would use a proper sentiment analysis model
        
        positive_words = ['good', 'excellent', 'positive', 'beneficial', 'advantage', 
                          'effective', 'success', 'improve', 'best', 'important']
        negative_words = ['bad', 'poor', 'negative', 'problem', 'disadvantage', 
                          'ineffective', 'failure', 'worse', 'worst', 'difficult']
        
        # Count occurrences of positive and negative words
        text_lower = text.lower()
        positive_count = sum(text_lower.count(' ' + word + ' ') for word in positive_words)
        negative_count = sum(text_lower.count(' ' + word + ' ') for word in negative_words)
        
        # Determine overall sentiment
        if positive_count > negative_count * 1.5:
            sentiment = 'positive'
        elif negative_count > positive_count * 1.5:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
        
        # Check for academic/formal language
        academic_markers = ['therefore', 'thus', 'consequently', 'furthermore', 'moreover']
        academic_count = sum(text_lower.count(' ' + marker + ' ') for marker in academic_markers)
        is_academic = academic_count > len(text.split()) / 200  # Arbitrary threshold
        
        return {
            'overall_sentiment': sentiment,
            'positive_count': positive_count,
            'negative_count': negative_count,
            'is_academic_tone': is_academic
        }


class TextSummarizer:
    """Generates summaries of text content"""
    
    def __init__(self, keyword_extractor=None):
        self.keyword_extractor = keyword_extractor or KeywordExtractor()
    
    def generate_summaries(self, sections, max_length=200):
        """Create concise summaries of complex sections."""
        # Simple extractive summarization implementation
        
        if isinstance(sections, str):
            text = sections
        else:
            # If sections is a dictionary or list, concatenate all text
            text = ' '.join([section if isinstance(section, str) else section.get('text', '')
                             for section in sections])
        
        # Split into sentences
        sentences = text.replace('!', '.').replace('?', '.').split('.')
        sentences = [s.strip() for s in sentences if s.strip()]
        
        if not sentences:
            return ""
        
        # Simple scoring: prefer sentences with keywords and at the beginning/end
        keywords = self.keyword_extractor.extract_keywords_and_concepts(text)['keywords'][:10]
        
        scored_sentences = []
        for i, sentence in enumerate(sentences):
            # Score based on position (beginning and end sentences are important)
            position_score = 1.0
            if i < len(sentences) / 10:  # First 10%
                position_score = 2.0
            elif i > len(sentences) * 0.9:  # Last 10%
                position_score = 1.5
                
            # Score based on keyword presence
            keyword_score = sum(1.0 for keyword in keywords if keyword in sentence.lower())
            
            # Score based on sentence length (not too short, not too long)
            length = len(sentence.split())
            length_score = 1.0
            if 5 <= length <= 20:
                length_score = 1.5
                
            # Combine scores
            total_score = position_score + keyword_score + length_score
            scored_sentences.append((sentence, total_score))
        
        # Sort by score and select top sentences
        scored_sentences.sort(key=lambda x: x[1], reverse=True)
        
        # Select top sentences up to max_length
        summary = ""
        for sentence, _ in scored_sentences:
            if len(summary) + len(sentence) <= max_length:
                summary += sentence + ". "
            else:
                break
                
        return summary.strip()


In [7]:
class ContentAnalyzer:
    def __init__(self, language_model=None, domain=None, extractor=None):
        """
        Initialize with optional connection to an extractor
        
        Args:
            language_model: Optional pre-trained language model
            domain: Optional domain specification
            extractor: Optional PDFContentExtractor to use for segmentation
        """
        self.language_model = language_model
        self.domain = domain
        self.domain_specific_rules = self._load_domain_rules(domain)
        self.extractor = extractor
        
        # Create helper objects
        self.text_processor = TextProcessor()
        self.semantic_analyzer = SemanticAnalyzer(self.domain_specific_rules)
        self.keyword_extractor = KeywordExtractor(self.text_processor.stop_words, self.domain_specific_rules)
        self.sentiment_analyzer = SentimentAnalyzer()
        self.text_summarizer = TextSummarizer(self.keyword_extractor)
    
    def _load_domain_rules(self, domain):
        """
        Load domain-specific rules and patterns for content analysis.

        Args:
            domain: The domain to load rules for

        Returns:
            Dictionary of rules and patterns specific to the domain
        """
        return DomainRulesManager.get_domain_rules(domain)
    
    def analyze_content(self, extraction_result):
        """
        Main method to analyze extracted content from a PDF.
        
        Args:
            extraction_result: Dictionary containing the extracted content from PDFContentExtractor
            
        Returns:
            Dictionary containing the original content enriched with semantic annotations
        """
        analyzed_result = extraction_result.copy()
        if self.language_model is not None:
            return self._analyze_with_language_model(extraction_result)
        
        # Add semantic analysis to the content
        if 'content_blocks' in analyzed_result:
            analyzed_result['content_blocks'] = self.tag_content_blocks(analyzed_result['content_blocks'])
        
        # Extract keywords and concepts
        if 'text_content' in analyzed_result:
            analyzed_result['keywords_and_concepts'] = self.keyword_extractor.extract_keywords_and_concepts(
                analyzed_result['text_content'], self.domain
            )
        
        # Generate summaries for sections
        if 'sections' in analyzed_result:
            analyzed_result['section_summaries'] = {
                section_id: self.text_summarizer.generate_summaries(section_content)
                for section_id, section_content in analyzed_result['sections'].items()
            }
        
        # Segment content into logical units
        analyzed_result['content_segments'] = self.segment_content(extraction_result)
        
        # Add sentiment analysis if text is available
        if 'text_content' in analyzed_result:
            analyzed_result['sentiment_analysis'] = self.sentiment_analyzer.perform_sentiment_analysis(
                analyzed_result['text_content']
            )
        
        if self.domain == 'literature':
            try:
                # Extract full text content
                text_content = extraction_result.get('text_content', '')
                work_subtype = self._determine_literary_work_subtype(text_content)
    
                if work_subtype:
                    # Add literary analysis (without causing recursion)
                    analyzed_result['literary_analysis'] = self.analyze_literary_work(
                        extraction_result, work_subtype
                    )
            except Exception as e:
                # Fail gracefully if the literary analysis can't be performed
                analyzed_result['literary_analysis_error'] = str(e)
    
        return analyzed_result
    
    def segment_content(self, extraction_result):
        """
        Divides content into logical segments using extractor if available,
        otherwise uses built-in segmentation
        
        Args:
            extraction_result: Dictionary with extracted content
            
        Returns:
            List of content segments with semantic annotations
        """
        segments = []
        
        # Use extractor for segmentation if available
        if self.extractor and hasattr(self.extractor, 'segment_text'):
            # If we have structured content already
            if 'pages' in extraction_result:
                for page in extraction_result['pages']:
                    for block in page.get('blocks', []):
                        # Get base segmentation from extractor
                        if 'text' in block:
                            text_segments = self.extractor.segment_text(block['text'])
                            for text in text_segments:
                                # Add semantic annotation 
                                segment_type = self.semantic_analyzer.identify_segment_type(text)
                                segments.append({
                                    'type': segment_type,
                                    'content': text,
                                    'semantic_tags': self.semantic_analyzer.generate_semantic_tags(text)
                                })
            # If we only have raw text
            elif 'text_content' in extraction_result:
                text_segments = self.extractor.segment_text(extraction_result['text_content'])
                for text in text_segments:
                    segment_type = self.semantic_analyzer.identify_segment_type(text)
                    segments.append({
                        'type': segment_type,
                        'content': text,
                        'semantic_tags': self.semantic_analyzer.generate_semantic_tags(text)
                    })
        else:
            # Fall back to built-in segmentation
            if 'document_structure' in extraction_result:
                structure = extraction_result['document_structure']
                for section in structure:
                    # Create a segment for the heading
                    if 'heading' in section:
                        segments.append({
                            'type': 'heading',
                            'level': section.get('level', 1),
                            'content': section['heading'],
                            'semantic_tags': self.semantic_analyzer.generate_semantic_tags(section['heading'])
                        })
                    
                    # Process the content under this heading
                    if 'content' in section:
                        content_segments = self.text_processor.segment_text_block(section['content'])
                        segments.extend(content_segments)
            
            # If no structure is available, segment the raw text
            elif 'text_content' in extraction_result:
                segments = self.text_processor.segment_text_block(extraction_result['text_content'])
        
        return segments
    
    def tag_content_blocks(self, blocks):
        """
        Classify content blocks into specific types.
        
        Args:
            blocks: List of content blocks from the extraction result
            
        Returns:
            List of content blocks with type classification added
        """
        tagged_blocks = []
        
        for block in blocks:
            block_copy = block.copy()
            
            # Determine the block type based on content and structure
            if 'text' in block:
                block_copy['block_type'] = self.semantic_analyzer.identify_segment_type(block['text'])
                
                # Add additional semantic tags if possible
                block_copy['semantic_tags'] = self.semantic_analyzer.generate_semantic_tags(block['text'])
            
            tagged_blocks.append(block_copy)
            
        return tagged_blocks
    
    def analyze_literary_work(self, extraction_result, work_type='novel'):
        """
        Special method for analyzing literary works like Shakespeare plays

        Args:
            extraction_result: Dictionary containing the extracted content
            work_type: Type of literary work ('play', 'novel', 'poem')

        Returns:
            Dictionary with literary analysis results
        """
        # Create literary work analyzer
        literary_analyzer = LiteraryWorkAnalyzer(work_type)

        # Extract full text content
        text_content = extraction_result.get('text_content', '')

        # Initialize result dictionary
        literary_analysis = {
            'characters': literary_analyzer.analyze_characters(text_content),
            'structure': literary_analyzer.identify_scenes(text_content),
            'literary_devices': literary_analyzer.detect_literary_devices(text_content)
        }

        # For plays and poetry, add verse analysis
        if work_type in ['play', 'poem']:
            literary_analysis['verse_patterns'] = literary_analyzer.analyze_verse_patterns(text_content)

        # For Shakespeare works, add theme analysis
        if work_type == 'play:shakespeare':
            literary_analysis['themes'] = literary_analyzer.detect_shakespearean_themes(text_content)

        return literary_analysis

    # Delegate methods to appropriate classes for interface compatibility
    def extract_keywords_and_concepts(self, text, domain=None, max_terms=20, min_word_length=3):
        return self.keyword_extractor.extract_keywords_and_concepts(text, domain, max_terms, min_word_length)
    
    def extract_keywords_with_positions(self, text, domain=None, max_terms=20, min_word_length=3):
        return self.keyword_extractor.extract_keywords_with_positions(text, domain, max_terms, min_word_length)
    
    def extract_key_terms_with_importance(self, text, max_terms=20, min_word_length=3):
        return self.keyword_extractor.extract_key_terms_with_importance(text, self.domain_specific_rules, max_terms, min_word_length)
    
    def generate_summaries(self, sections, max_length=200):
        return self.text_summarizer.generate_summaries(sections, max_length)
    
    def perform_sentiment_analysis(self, text):
        return self.sentiment_analyzer.perform_sentiment_analysis(text)
    
    def _determine_literary_work_subtype(self, text_content):
        """
        Determine the subtype of literary work based on content analysis.

        Args:
            text_content: The raw text content

        Returns:
            String indicating work subtype or None
        """
        # Check for play-like structure (character names followed by dialogue)
        dialogue_patterns = re.findall(r'([A-Z]{2,}[A-Z\s]*)[\.:]\s', text_content)

        if len(dialogue_patterns) > 10:
            # This looks like a play
            work_subtype = 'play'

            # Check for Shakespeare-specific markers
            shakespeare_markers = ['thee', 'thou', 'thy', 'hath', 'doth', 'prithee', 'forsooth', 'anon']
            shakespeare_count = sum(text_content.lower().count(' ' + marker + ' ') for marker in shakespeare_markers)

            if shakespeare_count > 5:
                work_subtype += ':shakespeare'
        elif re.search(r'Chapter [IVXLCDM]+|Chapter \d+', text_content):
            work_subtype = 'novel'
        elif self.text_processor.detect_poetic_structure(text_content):
            work_subtype = 'poem'
        else:
            work_subtype = 'general'

        return work_subtype
    
    def _find_term_locations(self, text, term):
        return self.text_processor.find_term_locations(text, term)

    def _analyze_with_language_model(self, extraction_result):
        """
        Use the provided language model to perform content analysis.

        Args:
            extraction_result: Dictionary containing the extracted content

        Returns:
            Dictionary with enhanced analysis from the language model
        """
        analyzed_result = extraction_result.copy()
        text_content = extraction_result.get('text_content', '')

        if not text_content:
            # Fallback to original method if no text content is available
            return self.analyze_content(extraction_result)

        # Get comprehensive analysis from language model
        model_analysis = {
            'keywords_and_concepts': self.language_model.extract_keywords(text_content, self.domain),
            'sentiment_analysis': self.language_model.analyze_sentiment(text_content),
            'content_segments': self.language_model.segment_content(text_content),
            'semantic_tags': self.language_model.generate_semantic_tags(text_content, self.domain)
        }

        # Add section summaries if sections exist
        if 'sections' in extraction_result:
            model_analysis['section_summaries'] = {}
            for section_id, section_content in extraction_result['sections'].items():
                model_analysis['section_summaries'][section_id] = self.language_model.summarize_text(
                    section_content, max_length=200
                )

        # Add domain-specific analysis
        if self.domain == 'literature':
            try:
                work_subtype = self._determine_literary_work_subtype(text_content)
                if work_subtype:
                    model_analysis['literary_analysis'] = self.language_model.analyze_literary_work(
                        text_content, work_subtype
                    )
            except Exception as e:
                model_analysis['literary_analysis_error'] = str(e)

        # Merge the language model analysis with the extraction result
        analyzed_result.update(model_analysis)

        return analyzed_result

In [ ]:
import requests
import json

class OpenRouterMistralClient:
    CHAR_LIMIT = 30000  # approximate character limit to avoid context overflow

    def __init__(self, api_key: str,
                 model: str = "mistralai/mistral-7b-instruct-v0.2"):
        """
        LM client for Mistral‑7B‑Instruct via OpenRouter’s free API.

        Args:
            api_key: Your OpenRouter API key (starts with sk-…)
            model:   The model identifier on OpenRouter
        """
        self.url = "https://openrouter.ai/api/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        self.model = model

    def _chunk_text(self, text: str) -> list[str]:
        """
        Split text into chunks under CHAR_LIMIT characters.
        """
        chunks = []
        start = 0
        while start < len(text):
            end = min(start + self.CHAR_LIMIT, len(text))
            # try to split at last newline within limit for cleaner chunks
            if end < len(text):
                nl = text.rfind("\n", start, end)
                if nl > start:
                    end = nl
            chunks.append(text[start:end])
            start = end
        return chunks

    def _generate(self, prompt_template: str, text: str, max_tokens: int = 256) -> str:
        """
        Core chat‑completion call to OpenRouter, with input chunking.

        Args:
            prompt_template: a format string expecting placeholders subbed
            text:            the raw text to insert into the prompt
            max_tokens:      model's max new tokens

        Returns:
            Combined responses across all text chunks.
        """
        responses = []
        for chunk in self._chunk_text(text):
            prompt = prompt_template.replace("{text}", chunk)
            payload = {
                "model": self.model,
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens
            }
            resp = requests.post(self.url, headers=self.headers, json=payload)
            if not resp.ok:
                raise RuntimeError(
                    f"OpenRouter API error {resp.status_code}: {resp.text}"
                )
            data = resp.json()
            if "choices" not in data:
                raise KeyError(f"OpenRouter response missing 'choices': {json.dumps(data)}")
            content = data["choices"][0]["message"]["content"]
            responses.append(content)
        # Join chunk responses naturally
        return "\n".join(responses)

    def extract_keywords(self, text: str, domain: str = None):
        template = (
            "Extract the key terms and concepts from the following {domain} text:\n\n{text}"
        )
        return self._generate(template.replace("{domain}", domain or 'general'), text)

    def analyze_sentiment(self, text: str):
        template = (
            "Provide a sentiment analysis of the following text. "
            "Return a JSON with fields 'sentiment' and 'score':\n\n{text}"
        )
        return self._generate(template, text)

    def segment_content(self, text: str):
        template = "Split the following into logical content segments:\n\n{text}"
        segments = self._generate(template, text, max_tokens=512)
        return [seg.strip() for seg in segments.split("\n") if seg.strip()]

    def generate_semantic_tags(self, text: str, domain: str = None):
        template = (
            "Generate a list of semantic tags relevant to {domain} domain for this text:\n\n{text}"
        )
        tags = self._generate(template.replace("{domain}", domain or 'general'), text)
        return [tag.strip() for tag in tags.split(",")]

    def summarize_text(self, text: str, max_length: int = 200):
        template = (
            "Summarize the following text in no more than {max} words:\n\n{text}"
        )
        return self._generate(template.replace("{max}", str(max_length)), text, max_tokens=max_length)

    def analyze_literary_work(self, text: str, subtype: str):
        template = (
            "As a literary critic analyzing a {subtype}, provide characters, structure, devices, and themes:\n\n{text}"
        )
        return self._generate(template.replace("{subtype}", subtype), text, max_tokens=512)


# 1. Instantiate your LM client (replace HF_TOKEN with your actual token)

In [ ]:
api_token = "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"
lm_client = OpenRouterMistralClient(api_key=api_token)

# 2. Plug into ContentAnalyzer
pdf_extractor = EnhancedPDFExtractor()
analyzer = ContentAnalyzer(
    language_model=lm_client,
    domain="literature",
    extractor=pdf_extractor
)

extraction_result = pdf_extractor.extract_content_with_cleaning(pdf_path="input.pdf",output_dir="test_output")
# 3. Run analysis on your extraction result
result = analyzer.analyze_content(extraction_result)

# 4. Inspect the LM-driven fields
print(result["keywords_and_concepts"])
print(result["sentiment_analysis"])
print(result["section_summaries"])


In [17]:
print(result['structure'])

[{'type': 'page', 'elements': [{'type': 'heading', 'text': 'The Merchant of \nVenice PDF', 'position': {'x0': 132.77, 'y0': 566.1704480000001, 'x1': 473.78659999999996, 'y1': 682.114448}, 'font_size': 49.94399999999996}, {'type': 'heading', 'text': 'A full version of William Shakespeare’s The Merchant of Venice text', 'position': {'x0': 84.5281, 'y0': 516.87056, 'x1': 514.33408, 'y1': 537.13136}, 'font_size': 16.080000000000034}, {'type': 'figure', 'position': {'x0': 241.68000000000004, 'y0': 271.44, 'x1': 353.52000000000004, 'y1': 402.72}}, {'type': 'heading', 'text': 'NoSweatShakespeare.com', 'position': {'x0': 211.06, 'y0': 109.01384, 'x1': 387.80408, 'y1': 125.09384}, 'font_size': 16.079999999999988}, {'type': 'heading', 'text': 'Making Shakespeare easy and accessible', 'position': {'x0': 165.43, 'y0': 79.72176, 'x1': 433.6641857968, 'y1': 95.80176}, 'font_size': 16.08}, {'type': 'text', 'text': '1', 'position': {'x0': 294.84, 'y0': 49.645920000000004, 'x1': 302.855342496, 'y1': 60

In [11]:
# def analyze_segments(self, segments):
#         """
#         Analyze already segmented content

#         Args:
#             segments: List of content segments from PDF extractors

#         Returns:
#             List of segments with enhanced semantic analysis
#         """
#         analyzed_segments = []

#         for segment in segments:
#             segment_copy = segment.copy()

#             # Add semantic analysis
#             segment_type = self._identify_segment_type(segment["text"])
#             semantic_tags = self._generate_semantic_tags(segment["text"])

#             # Add analysis to segment
#             segment_copy["semantic_type"] = segment_type
#             segment_copy["semantic_tags"] = semantic_tags

#             # Extract keywords if it's a content segment
#             if segment["type"] in ["content", "paragraph"]:
#                 keywords = self.extract_keywords_and_concepts(segment["text"], self.domain)
#                 segment_copy["keywords"] = keywords["keywords"]

#             analyzed_segments.append(segment_copy)

#         return analyzed_segments

#     def segment_content(self, extraction_result):
#         """
#         Create content segments from extraction result - now just applying semantic analysis

#         Args:
#             extraction_result: Dictionary with extraction results

#         Returns:
#             List of semantically enhanced segments
#         """
#         # Check if we have segments already
#         if "content_segments" in extraction_result:
#             # Just add semantic analysis
#             return self.analyze_segments(extraction_result["content_segments"])

#         # Handle structured content
#         if "structured_content" in extraction_result:
#             flattened_segments = []

#             for section in extraction_result["structured_content"]:
#                 # Add heading
#                 if section.get("heading"):
#                     flattened_segments.append({
#                         "type": "heading",
#                         "level": section.get("heading_level", 1),
#                         "text": section["heading"]
#                     })

#                 # Add content segments
#                 for content in section.get("content", []):
#                     text_type = self._identify_segment_type(content)
#                     flattened_segments.append({
#                         "type": text_type,
#                         "text": content
#                     })

#             return self.analyze_segments(flattened_segments)

#         # Fallback to raw text segmentation (minimal implementation)
#         if "text_content" in extraction_result:
#             # Split text into paragraphs
#             paragraphs = extraction_result["text_content"].split('\n\n')
#             segments = []

#             for paragraph in paragraphs:
#                 if not paragraph.strip():
#                     continue

#                 segment_type = self._identify_segment_type(paragraph)
#                 segments.append({
#                     "type": segment_type,
#                     "text": paragraph.strip()
#                 })

#             return self.analyze_segments(segments)

#         return []

In [18]:
class BaseDomainProcessor:
    def __init__(self, domain_patterns=None):
        """
        Base class for domain-specific processors.
        
        Args:
            domain_patterns (dict): Patterns for the specific domain
        """
        self.domain_patterns = domain_patterns or {}
    
    def process(self, extraction_result):
        """
        Process the extraction result with domain-specific logic.
        Should be implemented by subclasses.
        
        Args:
            extraction_result (dict): The extraction result from PDF processor
            
        Returns:
            Dictionary with domain-specific data
        """
        raise NotImplementedError("Subclasses must implement this method")


class ScienceProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to scientific domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with science-specific entities
        """
        scientific_entities = {
            'terminology': self.extract_scientific_terminology(extraction_result),
            'methods': self.identify_experiments_and_methods(extraction_result),
        }
        
        return scientific_entities
    
    def extract_scientific_terminology(self, extraction_result):
        """
        Extract scientific terms and classify by field.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of scientific terminology by category
        """
        text_content = extraction_result.get('text_content', '')
        
        # Define common scientific terminology patterns
        scientific_terms = {
            'general': [],
            'methodology': [],
            'results': [],
            'concepts': []
        }
        
        # Lists of common scientific terms by category
        methodology_terms = ['experiment', 'method', 'procedure', 'protocol', 'analysis', 
                            'technique', 'assay', 'measurement', 'observation']
        
        result_terms = ['result', 'finding', 'data', 'evidence', 'observation', 
                       'outcome', 'conclusion', 'analysis', 'figure', 'table']
        
        concept_terms = ['theory', 'model', 'hypothesis', 'principle', 'law', 
                        'concept', 'framework', 'paradigm', 'mechanism']
        
        # Simple pattern matching for scientific terminology
        for term in methodology_terms:
            if term in text_content.lower():
                # Find the sentence containing the term
                sentences = text_content.replace('!', '.').replace('?', '.').split('.')
                for sentence in sentences:
                    if term in sentence.lower():
                        scientific_terms['methodology'].append(sentence.strip())
        
        for term in result_terms:
            if term in text_content.lower():
                sentences = text_content.replace('!', '.').replace('?', '.').split('.')
                for sentence in sentences:
                    if term in sentence.lower():
                        scientific_terms['results'].append(sentence.strip())
        
        for term in concept_terms:
            if term in text_content.lower():
                sentences = text_content.replace('!', '.').replace('?', '.').split('.')
                for sentence in sentences:
                    if term in sentence.lower():
                        scientific_terms['concepts'].append(sentence.strip())
        
        # Extract other scientific terms using patterns
        import re
        
        # Pattern for scientific terms (simplified)
        scientific_term_pattern = re.compile(r'(?i)([A-Z][a-z]+(?:ology|ography|ometry|onomy|physics|chemistry|ics))')
        matches = scientific_term_pattern.findall(text_content)
        
        scientific_terms['general'] = list(set(matches))
        
        return scientific_terms
    
    def identify_experiments_and_methods(self, extraction_result):
        """
        Identify experimental procedures and methods in the content.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of identified experiments and methods
        """
        text_content = extraction_result.get('text_content', '')
        
        experiments_and_methods = {
            'experiments': [],
            'methods': [],
            'procedures': []
        }
        
        import re
        
        # Extract methods
        if 'method_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['method_pattern'], text_content):
                method_text = match.group(2).strip()
                experiments_and_methods['methods'].append(method_text)
        
        # Extract experiments
        if 'experiment_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['experiment_pattern'], text_content):
                experiment_text = match.group(2).strip()
                experiments_and_methods['experiments'].append(experiment_text)
        
        # Look for procedure keywords
        procedure_keywords = ['procedure', 'protocol', 'step', 'process']
        sentences = text_content.replace('!', '.').replace('?', '.').split('.')
        
        for sentence in sentences:
            if any(keyword in sentence.lower() for keyword in procedure_keywords):
                experiments_and_methods['procedures'].append(sentence.strip())
        
        return experiments_and_methods


class MathProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to mathematical domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with math-specific entities
        """
        return {
            'mathematical_entities': self.extract_mathematical_entities(extraction_result),
            'problem_classifications': self.classify_math_problems(extraction_result)
        }
    
    def extract_mathematical_entities(self, extraction_result):
        """
        Extract theorems, formulas, proofs, and other mathematical entities.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of mathematical entities by type
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        math_entities = {
            'theorems': [],
            'proofs': [],
            'equations': [],
            'definitions': []
        }
        
        # Extract theorems
        if 'theorem_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['theorem_pattern'], text_content):
                entity_type = match.group(1).lower()  # theorem, lemma, etc.
                entity_name = match.group(2).strip()
                entity_content = match.group(3).strip()
                
                math_entities['theorems'].append({
                    'type': entity_type,
                    'name': entity_name,
                    'content': entity_content
                })
        
        # Extract proofs
        if 'proof_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['proof_pattern'], text_content):
                proof_content = match.group(1).strip()
                math_entities['proofs'].append(proof_content)
        
        # Extract equations
        if 'equation_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['equation_pattern'], text_content):
                equation_num = match.group(1) or match.group(2)
                equation_content = match.group(3).strip()
                
                math_entities['equations'].append({
                    'number': equation_num,
                    'content': equation_content
                })
        
        # Extract definitions
        definition_pattern = re.compile(r'(?i)Definition\s*(\d+|[\w\s]+)?[:]\s*(.*?)(?=\n\n|\Z)')
        for match in re.finditer(definition_pattern, text_content):
            def_name = match.group(1).strip() if match.group(1) else ""
            def_content = match.group(2).strip()
            
            math_entities['definitions'].append({
                'name': def_name,
                'content': def_content
            })
        
        return math_entities
    
    def classify_math_problems(self, extraction_result):
        """
        Classify mathematical problems by type and difficulty.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of classified mathematical problems
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        math_problems = []
        
        # Pattern to identify problems
        problem_pattern = re.compile(r'(?i)(?:Problem|Exercise|Question)\s*(\d+|[\w\s]+)?[:]\s*(.*?)(?=\n\n|\Z|Solution)')
        
        for match in re.finditer(problem_pattern, text_content):
            problem_num = match.group(1).strip() if match.group(1) else ""
            problem_text = match.group(2).strip()
            
            # Try to find the solution if available
            solution_pattern = re.compile(f'(?i)Solution(?:\s*to\s*(?:Problem|Exercise|Question)?\s*{re.escape(problem_num) if problem_num else ""})?\s*[:]\s*(.*?)(?=\n\n|\Z|Problem|Exercise|Question)')
            solution_match = solution_pattern.search(text_content)
            
            solution_text = solution_match.group(1).strip() if solution_match else ""
            
            # Classify problem type
            problem_type = self._classify_math_problem_type(problem_text)
            
            # Estimate difficulty
            difficulty = self._estimate_math_problem_difficulty(problem_text, solution_text)
            
            math_problems.append({
                'number': problem_num,
                'text': problem_text,
                'solution': solution_text,
                'type': problem_type,
                'difficulty': difficulty
            })
        
        return math_problems
    
    def _classify_math_problem_type(self, problem_text):
        """
        Classify the type of a mathematical problem.
        
        Args:
            problem_text (str): The text of the problem
            
        Returns:
            The type of the mathematical problem
        """
        problem_text = problem_text.lower()
        
        # Define keywords for different problem types
        type_keywords = {
            'algebra': ['solve', 'equation', 'polynomial', 'factor', 'simplify', 'expression'],
            'calculus': ['derivative', 'integral', 'differentiate', 'integrate', 'limit', 'maximize', 'minimize'],
            'geometry': ['triangle', 'circle', 'angle', 'polygon', 'rectangle', 'square', 'perimeter', 'area', 'volume'],
            'probability': ['probability', 'random', 'chance', 'likelihood', 'expected value', 'variance'],
            'statistics': ['mean', 'median', 'mode', 'standard deviation', 'normal distribution', 'variance'],
            'number theory': ['prime', 'divisor', 'factor', 'gcd', 'lcm', 'modulo', 'congruence'],
            'linear algebra': ['matrix', 'vector', 'eigenvalue', 'eigenvector', 'linear transformation'],
            'discrete math': ['graph', 'tree', 'network', 'combinatorial', 'permutation', 'combination']
        }
        
        # Count occurrences of each category's keywords
        type_scores = {category: 0 for category in type_keywords}
        
        for category, keywords in type_keywords.items():
            for keyword in keywords:
                if keyword in problem_text:
                    type_scores[category] += 1
        
        # Return the category with the highest score
        if any(type_scores.values()):
            return max(type_scores.items(), key=lambda x: x[1])[0]
        else:
            return 'general'
    
    def _estimate_math_problem_difficulty(self, problem_text, solution_text=""):
        """
        Estimate the difficulty level of a mathematical problem.
        
        Args:
            problem_text (str): The text of the problem
            solution_text (str): The text of the solution
            
        Returns:
            The estimated difficulty level (elementary, intermediate, advanced)
        """
        # Combine problem and solution for analysis
        combined_text = problem_text + " " + solution_text
        combined_text = combined_text.lower()
        
        # Define difficulty indicators
        elementary_indicators = ['basic', 'simple', 'elementary', 'straightforward', 'easy', 'find']
        advanced_indicators = ['complex', 'challenging', 'advanced', 'difficult', 'prove', 'proof', 'theorem', 'lemma']
        
        # Count advanced mathematical terms
        advanced_math_terms = ['eigenvalue', 'eigenvector', 'homomorphism', 'isomorphism', 'bijection',
                              'riemann', 'laplace', 'fourier', 'lebesgue', 'optimization', 'stochastic',
                              'differential equation', 'partial derivative', 'tensor', 'manifold', 'topology']
        
        # Count occurrences
        elementary_count = sum(indicator in combined_text for indicator in elementary_indicators)
        advanced_count = sum(indicator in combined_text for indicator in advanced_indicators)
        advanced_term_count = sum(term in combined_text for term in advanced_math_terms)
        
        # Text complexity heuristics
        word_count = len(combined_text.split())
        sentence_count = max(1, len(combined_text.split('.')))
        average_sentence_length = word_count / sentence_count
        
        # Decision logic for difficulty level
        if advanced_term_count >= 2 or advanced_count >= 3 or average_sentence_length > 25:
            return 'advanced'
        elif elementary_count > advanced_count or (word_count < 50 and advanced_term_count == 0):
            return 'elementary'
        else:
            return 'intermediate'


class LiteratureProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to literature domains by using the LiteraryWorkAnalyzer.

        Args:
            extraction_result (dict): The extraction result

        Returns:
            Dictionary with literature-specific entities
        """
        text_content = extraction_result.get('text_content', '')

        # Determine work type to use appropriate analyzer
        work_type = 'general'
        import re
        
        if 'shakespeare' in extraction_result.get('metadata', {}).get('title', '').lower() or 'shakespeare' in text_content.lower():
            work_type = 'play:shakespeare'
        elif re.search(r'([A-Z]{2,}[A-Z\s]*)[\.:]\s', text_content):
            work_type = 'play'
        elif re.search(r'Chapter [IVXLCDM]+|Chapter \d+', text_content):
            work_type = 'novel'

        # Use the dedicated LiteraryWorkAnalyzer
        literary_analyzer = LiteraryWorkAnalyzer(work_type)

        return {
            'literary_structure': literary_analyzer.identify_scenes(text_content),
            'literary_devices': literary_analyzer.detect_literary_devices(text_content)
        }


class ChemistryProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to chemistry domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with chemistry-specific entities
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        compounds = set()
        reactions = []
        conditions = []
        
        # Extract compounds
        if 'compound_pattern' in self.domain_patterns:
            compounds = set(re.findall(self.domain_patterns['compound_pattern'], text_content))
            # Filter common false positives
            compounds = {c for c in compounds if len(c) > 1 and any(char.isdigit() for char in c)}
        
        # Extract reactions
        if 'reaction_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['reaction_pattern'], text_content):
                reactants = match.group(1).strip()
                products = match.group(2).strip()
                reactions.append({
                    'reactants': reactants,
                    'products': products,
                    'full_reaction': f"{reactants} → {products}"
                })
        
        # Extract conditions
        if 'condition_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['condition_pattern'], text_content):
                conditions.append(match.group(0))
        
        return {
            'compounds': list(compounds),
            'reactions': reactions,
            'reaction_conditions': conditions
        }


class BiologyProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to biology domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with biology-specific entities
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        species = set()
        genes = set()
        pathways = set()
        
        # Extract species
        if 'species_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['species_pattern'], text_content):
                species_name = match.group(1)
                # Filter common false positives
                if len(species_name.split()) == 2 and all(len(word) > 1 for word in species_name.split()):
                    species.add(species_name)
        
        # Extract genes
        if 'gene_pattern' in self.domain_patterns:
            potential_genes = re.findall(self.domain_patterns['gene_pattern'], text_content)
            # Filter common false positives
            genes = {g for g in potential_genes if len(g) >= 2 and g.isupper()}
        
        # Extract pathways
        if 'pathway_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['pathway_pattern'], text_content):
                pathways.add(match.group(0))
        
        return {
            'species': list(species),
            'genes': list(genes),
            'biological_pathways': list(pathways)
        }


class PhysicsProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to physics domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with physics-specific entities
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        laws = set()
        physical_quantities = []
        equations = []
        
        # Extract physical laws
        if 'law_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['law_pattern'], text_content):
                laws.add(match.group(0))
        
        # Extract physical quantities with units
        if 'units_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['units_pattern'], text_content):
                value = match.group(1)
                unit = match.group(2)
                exponent = match.group(3) if match.group(3) else "1"
                
                physical_quantities.append({
                    'value': float(value),
                    'unit': unit,
                    'exponent': int(exponent),
                    'display': f"{value} {unit}{'' if exponent == '1' else f'^{exponent}'}"
                })
        
        # Extract equations
        if 'equation_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['equation_pattern'], text_content):
                equation_num = match.group(1) or match.group(2)
                equation_text = match.group(3)
                
                equations.append({
                    'equation_number': equation_num,
                    'equation_text': equation_text
                })
        
        return {
            'physical_laws': list(laws),
            'physical_quantities': physical_quantities,
            'physics_equations': equations
        }


In [19]:
class DomainSpecificProcessor:
    def __init__(self, domain_type):
        """
        Initialize the DomainSpecificProcessor with a specified domain type.
        
        Args:
            domain_type (str): The domain type to specialize in (science, math, literature, etc.)
        """
        self.domain_type = domain_type.lower() if domain_type else None
        
        # Domain-specific entity patterns
        self.domain_patterns = self._initialize_domain_patterns()
        
        # Initialize domain processors
        self._initialize_domain_processors()
    
    def _initialize_domain_patterns(self):
        """
        Initialize regex patterns and rules for different domains.

        Returns:
            Dictionary of patterns for each domain
        """
        # Simply use the domain rules manager instead of recreating logic
        domains = ['science', 'math', 'literature', 'chemistry', 'biology', 'physics']
        patterns = {}
     
        # Load rules for each domain
        for domain in domains:
            patterns[domain] = DomainRulesManager.get_domain_rules(domain)
            
        return patterns
    
    def _initialize_domain_processors(self):
        """
        Initialize domain-specific processors.
        """
        # Initialize domain processors with appropriate patterns
        self.domain_processors = {
            'science': ScienceProcessor(self.domain_patterns.get('science', {})),
            'math': MathProcessor(self.domain_patterns.get('math', {})),
            'literature': LiteratureProcessor(self.domain_patterns.get('literature', {})),
            'chemistry': ChemistryProcessor(self.domain_patterns.get('chemistry', {})),
            'biology': BiologyProcessor(self.domain_patterns.get('biology', {})),
            'physics': PhysicsProcessor(self.domain_patterns.get('physics', {}))
        }
    
    def process_domain_content(self, extraction_result):
        """
        Process content according to domain-specific rules.
        
        Args:
            extraction_result (dict): The extraction result from PDF processor
            
        Returns:
            Dictionary with domain-specific processing results added
        """
        if not self.domain_type or self.domain_type not in self.domain_processors:
            # If domain is not supported, return the original extraction result
            return extraction_result
        
        # Create a copy of the extraction result to modify
        processed_result = extraction_result.copy()
        
        # Apply domain-specific processing
        domain_processor = self.domain_processors[self.domain_type]
        domain_specific_data = domain_processor.process(processed_result)
        
        # Add domain-specific results to the processed result
        processed_result['domain_analysis'] = {
            'domain_type': self.domain_type,
            'domain_entities': domain_specific_data
        }
        
        return processed_result

In [20]:
class ContentEnrichmentProcessor:
    def __init__(self, knowledge_base=None, content_analyzer=None):
        # Initialize with optional knowledge base
        self.knowledge_base = knowledge_base
        # Use composition to leverage ContentAnalyzer
        self.content_analyzer = content_analyzer if content_analyzer else ContentAnalyzer()
        
    def enrich_content(self, extraction_result):
        # Add contextual information to extraction result
        enriched_result = extraction_result.copy() if hasattr(extraction_result, 'copy') else extraction_result
        
        # Extract concepts and terms for enrichment
        concepts = self.extract_concepts(enriched_result)
        
        # Add explanations for complex terms
        enriched_result = self.generate_explanations(enriched_result, concepts)
        
        # Add cross-references between related sections
        enriched_result = self.add_cross_references(enriched_result)
        
        # Suggest additional learning resources
        enriched_result = self.suggest_learning_resources(enriched_result, concepts)
        
        return enriched_result
        
    def extract_concepts(self, extraction_result):
        """Extract key concepts and terms from the extraction result with knowledge base enrichment"""
        concepts = []
        
        # Check if there's content to analyze
        if not extraction_result:
            return concepts
            
        # Get content text from extraction result
        if hasattr(extraction_result, 'content'):
            content = extraction_result.content
            text = ' '.join(content) if isinstance(content, list) else content
            
            # Use ContentAnalyzer to extract keywords - leverage the core class!
            analysis_result = self.content_analyzer.extract_keywords_and_concepts(text)
            concepts = analysis_result['keywords']
            
            # Add domain-specific enrichment here - class-specific logic
            if hasattr(self, 'knowledge_base') and self.knowledge_base:
                # Filter concepts based on domain knowledge
                domain_concepts = []
                for concept in concepts:
                    # Check if concept exists in our knowledge base
                    if concept in self.knowledge_base:
                        # Prioritize concepts that are in our knowledge base
                        domain_concepts.append(concept)
                
                # Add specialized domain concepts from knowledge base that might be relevant
                for key in self.knowledge_base:
                    # If term is in the text but wasn't caught by frequency analysis
                    if key.lower() in text.lower() and key not in concepts:
                        domain_concepts.append(key)
                
                # If we found domain-specific concepts, prioritize them
                if domain_concepts:
                    # Combine domain concepts with general concepts, prioritizing domain ones
                    remaining_slots = max(20 - len(domain_concepts), 0)
                    general_concepts = [c for c in concepts if c not in domain_concepts]
                    concepts = domain_concepts + general_concepts[:remaining_slots]
            
            # Process multi-word concepts (which may not be caught by frequency analysis)
            if hasattr(extraction_result, 'sections'):
                section_titles = []
                for section in extraction_result.sections:
                    if 'title' in section:
                        section_titles.append(section['title'])
                
                # Extract noun phrases from section titles
                import re
                for title in section_titles:
                    # Simple regex to find potential multi-word concepts
                    matches = re.findall(r'[A-Z][a-z]+(?:\s+[a-z]+){1,3}', title)
                    for match in matches:
                        if match.lower() in text.lower() and match not in concepts:
                            concepts.append(match)
        
        return concepts
        
    def generate_explanations(self, extraction_result, complex_terms):
        """Generate explanations for complex terms or concepts"""
        if not hasattr(extraction_result, 'explanations'):
            extraction_result['explanations'] = {}
            
        for term in complex_terms:
            # In a real implementation, would use knowledge base or external API
            # to generate explanations for complex terms
            if self.knowledge_base and term in self.knowledge_base:
                extraction_result.explanations[term] = self.knowledge_base[term]
            else:
                # Placeholder for explanation generation logic
                # Could use language models or other sources
                extraction_result.explanations[term] = f"Explanation for {term} would be generated here"
                
        return extraction_result
        
    def add_cross_references(self, extraction_result):
        """Add references to related sections within the document"""
        if not hasattr(extraction_result, 'cross_references'):
            extraction_result["cross_references"] = {}
            
        # Build section index
        sections = {}
        if hasattr(extraction_result, 'sections'):
            for i, section in enumerate(extraction_result.sections):
                # Extract key terms from section
                section_title = section.get('title', f"Section {i}")
                section_content = section.get('content', '')
                sections[section_title] = {
                    'index': i,
                    'content': section_content,
                    'key_terms': self.extract_concepts({'content': section_content})
                }
        
        # Find cross-references between sections based on shared key terms
        for title, section_data in sections.items():
            related_sections = []
            for other_title, other_data in sections.items():
                if title != other_title:
                    # Check for shared key terms
                    shared_terms = set(section_data['key_terms']) & set(other_data['key_terms'])
                    if shared_terms:
                        related_sections.append({
                            'title': other_title,
                            'index': other_data['index'],
                            'shared_terms': list(shared_terms)
                        })
            
            if related_sections:
                extraction_result.cross_references[title] = related_sections
                
        return extraction_result
        
    def suggest_learning_resources(self, extraction_result, concepts):
        """Suggest additional resources for understanding concepts"""
        if not hasattr(extraction_result, 'learning_resources'):
            extraction_result["learning_resources"] = {}
            
        # In a real implementation, would connect to a resource database or API
        # to find relevant learning materials based on the extracted concepts
        
        for concept in concepts[:5]:  # Limit to top 5 concepts for example
            # Placeholder for resource suggestion logic
            extraction_result.learning_resources[concept] = [
                {
                    'title': f"Introduction to {concept.capitalize()}",
                    'type': 'article',
                    'url': f"https://example.com/learn/{concept}"
                },
                {
                    'title': f"{concept.capitalize()} explained in depth",
                    'type': 'video',
                    'url': f"https://example.com/videos/{concept}"
                }
            ]
            
        return extraction_result

In [21]:
class ContentRelationshipAnalyzer:
    def __init__(self, content_analyzer=None):
        """
        Initialize the ContentRelationshipAnalyzer for understanding relationships between content elements.
        """
        # Store the content extraction result for analysis
        self.extraction_result = None
        
        # Internal data structures for analysis
        self.concept_graph = None
        self.dependency_tree = None
        self.concept_index = {}
        
        # Use composition to leverage ContentAnalyzer
        self.content_analyzer = content_analyzer if content_analyzer else ContentAnalyzer()
        
    def build_concept_graph(self, extraction_result):
        """
        Create a graph of related concepts in the document.
        
        Args:
            extraction_result (dict): The extraction result containing document content
            
        Returns:
            Dictionary representing a graph of concepts and their relationships
        """
        self.extraction_result = extraction_result
        
        # Initialize concept graph structure
        concept_graph = {
            'nodes': [],
            'edges': []
        }
        
        # Extract concepts from the document
        concepts = self._extract_concepts(extraction_result)
        
        # Add concepts as nodes to the graph
        for i, concept in enumerate(concepts):
            concept_node = {
                'id': f"concept_{i}",
                'label': concept['term'],
                'type': concept['type'],
                'frequency': concept['frequency'],
                'locations': concept['locations']
            }
            concept_graph['nodes'].append(concept_node)
            
            # Build index for quick lookup
            self.concept_index[concept['term']] = i
        
        # Build edges between related concepts
        for i, concept_a in enumerate(concepts):
            for j, concept_b in enumerate(concepts):
                if i == j:
                    continue
                
                # Calculate relationship strength between concepts
                relationship = self._calculate_concept_relationship(concept_a, concept_b)
                
                if relationship['strength'] > 0.3:  # Threshold for meaningful relationships
                    edge = {
                        'source': f"concept_{i}",
                        'target': f"concept_{j}",
                        'weight': relationship['strength'],
                        'type': relationship['type']
                    }
                    concept_graph['edges'].append(edge)
        
        self.concept_graph = concept_graph
        return concept_graph
    
    def _extract_concepts(self, extraction_result):
        """
        Extract concepts from document content with positions and types.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            List of concept dictionaries with term, type, frequency, and locations
        """
        concepts = []
        text_content = extraction_result.get('text_content', '')
        
        # First check if domain analysis is available - class-specific logic
        if 'domain_analysis' in extraction_result:
            domain_analysis = extraction_result['domain_analysis']
            domain_entities = domain_analysis.get('domain_entities', {})
            
            # Add domain-specific entities as concepts
            for entity_type, entities in domain_entities.items():
                if isinstance(entities, list):
                    for entity in entities:
                        if isinstance(entity, dict) and 'name' in entity:
                            entity_name = entity['name']
                            concepts.append({
                                'term': entity_name,
                                'type': entity_type,
                                'frequency': self._count_term_occurrences(text_content, entity_name),
                                'locations': self._find_term_locations(text_content, entity_name)
                            })
                        elif isinstance(entity, str):
                            concepts.append({
                                'term': entity,
                                'type': entity_type,
                                'frequency': self._count_term_occurrences(text_content, entity),
                                'locations': self._find_term_locations(text_content, entity)
                            })
        
        # Extract key terms from headings - class-specific logic
        structure = extraction_result.get('document_structure', [])
        heading_terms = set()
        for section in structure:
            if 'heading' in section:
                heading = section['heading']
                # Extract noun phrases from heading
                terms = self._extract_noun_phrases(heading)
                heading_terms.update(terms)
        
        # Add heading terms as concepts
        for term in heading_terms:
            if not any(concept['term'] == term for concept in concepts):
                concepts.append({
                    'term': term,
                    'type': 'heading_term',
                    'frequency': self._count_term_occurrences(text_content, term),
                    'locations': self._find_term_locations(text_content, term)
                })
        
        # Use ContentAnalyzer for core extraction - leverage the core class!
        domain = getattr(self.content_analyzer, 'domain', None)
        analysis_result = self.content_analyzer.extract_keywords_with_positions(text_content, domain)
        
        # Add keywords as concepts
        for term in analysis_result['keywords']:
            if not any(concept['term'] == term for concept in concepts):
                concepts.append({
                    'term': term,
                    'type': 'frequent_term',
                    'frequency': analysis_result['word_frequencies'].get(term, 0),
                    'locations': analysis_result['term_positions'].get(term, [])
                })
                
        # Add bigrams as concepts
        for term in analysis_result.get('bigrams', []):
            if not any(concept['term'] == term for concept in concepts):
                concepts.append({
                    'term': term,
                    'type': 'bigram_term',
                    'frequency': self._count_bigram_frequency(text_content, term),
                    'locations': analysis_result['term_positions'].get(term, [])
                })
        
        return concepts 

    def _count_bigram_frequency(self, text, bigram):
        """Count frequency of a bigram in text"""
        words = text.lower().split()
        count = 0
        bigram_parts = bigram.lower().split()
        
        if len(bigram_parts) != 2:
            return 0
            
        for i in range(len(words) - 1):
            if words[i] == bigram_parts[0] and words[i+1] == bigram_parts[1]:
                count += 1
        
        return count
    
    def _extract_noun_phrases(self, text):
        """
        Extract noun phrases from text.
        
        Args:
            text (str): Text to extract noun phrases from
            
        Returns:
            List of noun phrases
        """
        # Simple implementation using basic patterns
        # In a production environment, you would use a more sophisticated NLP library
        import re
        
        # Pattern for simple noun phrases
        noun_phrase_pattern = re.compile(r'(?:[A-Z][a-z]+\s)?(?:[A-Z][a-z]+\s)?[A-Z]?[a-z]+')
        matches = noun_phrase_pattern.findall(text)
        
        # Filter short phrases and common words
        stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 'by', 'this', 'with'}
        phrases = [match for match in matches if len(match) > 4 and match.lower() not in stop_words]
        
        return phrases
    
    def _extract_frequent_terms(self, text_content, max_terms=30):
        """
        Extract frequent terms from text content.
        
        Args:
            text_content (str): Text to analyze
            max_terms (int): Maximum number of terms to extract
            
        Returns:
            List of (term, frequency) tuples sorted by frequency
        """
        # Tokenize text into words
        words = text_content.lower().split()
        
        # Remove common stop words
        stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 
                      'by', 'this', 'with', 'i', 'you', 'it', 'not', 'or', 'be', 'are', 
                      'from', 'at', 'as', 'your', 'have', 'has', 'had', 'was', 'were', 
                      'they', 'their', 'them', 'we', 'our', 'us', 'he', 'she', 'his', 
                      'her', 'him', 'an', 'can', 'will', 'just', 'do', 'does', 'did',
                      'but', 'if', 'so', 'what', 'when', 'where', 'how', 'all', 'any',
                      'both', 'each', 'few', 'more', 'most', 'some', 'such', 'no', 'nor',
                      'too', 'very', 'one', 'every', 'least', 'less', 'many', 'now',
                      'ever', 'never', 'say', 'says', 'said', 'also', 'like', 'time'}
        
        filtered_words = [word.strip('.,;:()[]{}"\'-') for word in words if word.strip('.,;:()[]{}"\'-') not in stop_words]
        
        # Count word frequencies
        word_counts = {}
        for word in filtered_words:
            if len(word) > 3:  # Only consider words with more than 3 characters
                word_counts[word] = word_counts.get(word, 0) + 1
        
        # Sort by frequency and return top terms
        sorted_terms = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
        
        return sorted_terms[:max_terms]
    
    def _count_term_occurrences(self, text, term):
        return self.content_analyzer.text_processor.count_term_occurrences(text, term)
    
    def _find_term_locations(self, text, term):
        return self.content_analyzer.text_processor.find_term_locations(text, term)
    
    def _calculate_concept_relationship(self, concept_a, concept_b):
        """
        Calculate relationship between two concepts.
        
        Args:
            concept_a (dict): First concept
            concept_b (dict): Second concept
            
        Returns:
            Dictionary with relationship type and strength
        """
        # Calculate co-occurrence proximity
        proximity_score = self._calculate_proximity(concept_a['locations'], concept_b['locations'])
        
        # Check for term similarity
        similarity_score = self._calculate_term_similarity(concept_a['term'], concept_b['term'])
        
        # Check if one is a subterm of the other
        subterm_score = 0
        if concept_a['term'].lower() in concept_b['term'].lower():
            subterm_score = 0.8
        elif concept_b['term'].lower() in concept_a['term'].lower():
            subterm_score = 0.8
        
        # Calculate overall relationship strength
        strength = max(proximity_score, similarity_score, subterm_score)
        
        # Determine relationship type
        relationship_type = 'related'
        if subterm_score > 0:
            relationship_type = 'is_a'
        elif proximity_score > 0.6:
            relationship_type = 'co_occurs'
        elif similarity_score > 0.7:
            relationship_type = 'similar_to'
        
        return {
            'strength': strength,
            'type': relationship_type
        }
    
    def _calculate_proximity(self, locations_a, locations_b, max_distance=500):
        """
        Calculate proximity score based on locations.
        
        Args:
            locations_a (list): List of locations for first term
            locations_b (list): List of locations for second term
            max_distance (int): Maximum distance to consider for proximity
            
        Returns:
            Proximity score between 0 and 1
        """
        if not locations_a or not locations_b:
            return 0
        
        # Find minimum distance between any two locations
        min_distance = float('inf')
        for loc_a in locations_a:
            for loc_b in locations_b:
                distance = abs(loc_a - loc_b)
                min_distance = min(min_distance, distance)
        
        # Convert distance to proximity score
        if min_distance == float('inf') or min_distance > max_distance:
            return 0
        
        return 1 - (min_distance / max_distance)
    
    def _calculate_term_similarity(self, term_a, term_b):
        return self.content_analyzer.keyword_extractor.calculate_term_similarity(term_a, term_b)
    
    def identify_prerequisites(self, sections):
        """
        Determine which sections depend on understanding other sections.
        
        Args:
            sections (list): List of section dictionaries with content
            
        Returns:
            Dictionary mapping section IDs to their prerequisites
        """
        if not sections:
            return {}
        
        prerequisites = {}
        
        # Build terminology dictionary for each section
        section_terms = {}
        for i, section in enumerate(sections):
            section_id = section.get('id', f"section_{i}")
            content = section.get('content', '')
            
            # Extract key terms from this section
            if isinstance(content, str):
                terms = self._extract_key_terms(content)
            else:
                # If content is not a string (e.g., a list or dict), convert it
                content_str = str(content)
                terms = self._extract_key_terms(content_str)
            
            section_terms[section_id] = terms
        
        # For each section, identify which earlier sections introduce terms it uses
        for i, section in enumerate(sections):
            section_id = section.get('id', f"section_{i}")
            content = section.get('content', '')
            
            if not content:
                continue
            
            # Convert content to string if it's not already
            if not isinstance(content, str):
                content = str(content)
            
            # Find terms from previous sections that appear in this section
            section_prerequisites = {}
            
            for j, prior_section in enumerate(sections[:i]):
                prior_id = prior_section.get('id', f"section_{j}")
                prior_terms = section_terms.get(prior_id, [])
                
                # Count how many terms from the prior section appear in this section
                referenced_terms = []
                for term, _ in prior_terms:
                    if term in content and len(term) > 4:  # Minimum length to avoid common words
                        referenced_terms.append(term)
                
                if referenced_terms:
                    section_prerequisites[prior_id] = {
                        'strength': len(referenced_terms) / max(1, len(prior_terms)),
                        'referenced_terms': referenced_terms
                    }
            
            # Store prerequisites with strength above threshold
            filtered_prerequisites = {
                section_id: data
                for section_id, data in section_prerequisites.items()
                if data['strength'] > 0.2  # Threshold for prerequisite relationship
            }
            
            if filtered_prerequisites:
                prerequisites[section_id] = filtered_prerequisites
        
        return prerequisites
    
    def _extract_key_terms(self, text, max_terms=20):
        """
        Extract key terms from text using the ContentAnalyzer.
        
        Args:
            text (str): Text to analyze
            max_terms (int): Maximum number of terms to extract
            
        Returns:
            List of (term, importance) tuples sorted by importance
        """
        # Use the content analyzer for core extraction logic
        return self.content_analyzer.extract_key_terms_with_importance(text, max_terms)
    
    def find_related_content(self, concept, extraction_result):
        """
        Find all content related to a specific concept.
        
        Args:
            concept (str): The concept to find related content for
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of related content sections
        """
        if not extraction_result:
            return {}
        
        related_content = {
            'sections': [],
            'mentions': [],
            'related_concepts': []
        }
        
        # First, ensure we have built the concept graph
        if not self.concept_graph:
            self.build_concept_graph(extraction_result)
        
        # Find the concept in our index
        concept_id = None
        for i, node in enumerate(self.concept_graph.get('nodes', [])):
            if node['label'].lower() == concept.lower():
                concept_id = node['id']
                break
        
        # If concept not found, try substring matching
        if not concept_id:
            for i, node in enumerate(self.concept_graph.get('nodes', [])):
                if concept.lower() in node['label'].lower() or node['label'].lower() in concept.lower():
                    concept_id = node['id']
                    break
        
        # If still not found, return empty result
        if not concept_id:
            return related_content
        
        # Find all edges connected to this concept
        related_concept_ids = []
        for edge in self.concept_graph.get('edges', []):
            if edge['source'] == concept_id:
                related_concept_ids.append(edge['target'])
            elif edge['target'] == concept_id:
                related_concept_ids.append(edge['source'])
        
        # Get the related concepts
        for node in self.concept_graph.get('nodes', []):
            if node['id'] in related_concept_ids:
                related_concepts = related_content['related_concepts']
                related_concepts.append({
                    'term': node['label'],
                    'type': node['type'],
                    'frequency': node['frequency']
                })
        
        # Get content sections that mention the concept
        text_content = extraction_result.get('text_content', '')
        document_structure = extraction_result.get('document_structure', [])
        
        # Find the concept node
        concept_node = None
        for node in self.concept_graph.get('nodes', []):
            if node['id'] == concept_id:
                concept_node = node
                break
        
        if concept_node:
            concept_term = concept_node['label']
            
            # Find all sections that mention the concept
            for section in document_structure:
                section_content = section.get('content', '')
                if not isinstance(section_content, str):
                    section_content = str(section_content)
                
                if concept_term.lower() in section_content.lower():
                    related_content['sections'].append({
                        'heading': section.get('heading', 'Untitled Section'),
                        'level': section.get('level', 1),
                        'content_preview': self._get_context_around_term(section_content, concept_term)
                    })
            
            # Find specific mentions with context
            related_content['mentions'] = self._find_mentions_with_context(text_content, concept_term)
        
        return related_content
    
    def _get_context_around_term(self, text, term, context_size=100):
        """
        Get text context around a term.
        
        Args:
            text (str): The text to search in
            term (str): The term to find context for
            context_size (int): Number of characters around the term
            
        Returns:
            String with context
        """
        import re
        
        # Find all occurrences of the term
        matches = list(re.finditer(r'\b' + re.escape(term) + r'\b', text, re.IGNORECASE))
        
        if not matches:
            return ""
        
        # Use the first occurrence
        match = matches[0]
        start_pos = max(0, match.start() - context_size)
        end_pos = min(len(text), match.end() + context_size)
        
        # Extract context
        context = text[start_pos:end_pos]
        
        # Add ellipsis if needed
        if start_pos > 0:
            context = "..." + context
        if end_pos < len(text):
            context = context + "..."
        
        return context
    
    def _find_mentions_with_context(self, text, term, max_mentions=5, context_size=100):
        """
        Find mentions of a term with surrounding context.
        
        Args:
            text (str): The text to search in
            term (str): The term to find mentions of
            max_mentions (int): Maximum number of mentions to return
            context_size (int): Number of characters around each mention
            
        Returns:
            List of mention dictionaries with context
        """
        import re
        
        # Find all occurrences of the term
        pattern = r'\b' + re.escape(term) + r'\b'
        matches = list(re.finditer(pattern, text, re.IGNORECASE))
        
        mentions = []
        for match in matches[:max_mentions]:
            start_pos = max(0, match.start() - context_size)
            end_pos = min(len(text), match.end() + context_size)
            
            # Extract context
            context = text[start_pos:end_pos]
            
            # Add ellipsis if needed
            if start_pos > 0:
                context = "..." + context
            if end_pos < len(text):
                context = context + "..."
            
            mentions.append({
                'position': match.start(),
                'context': context
            })
        
        return mentions
    
    def create_dependency_tree(self, extraction_result):
        """
        Build a tree showing how concepts build upon each other.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary representing a dependency tree of concepts
        """
        # Ensure we have built the concept graph
        if not self.concept_graph:
            self.build_concept_graph(extraction_result)
        
        # Create dependency tree structure
        dependency_tree = {
            'nodes': [],
            'links': []
        }
        
        # Use document structure to identify concept ordering
        document_structure = extraction_result.get('document_structure', [])
        
        # Extract concepts in order of appearance
        ordered_concepts = []
        for section in document_structure:
            section_content = section.get('content', '')
            if not isinstance(section_content, str):
                section_content = str(section_content)
            
            # Find concepts mentioned in this section
            section_concepts = []
            for node in self.concept_graph.get('nodes', []):
                concept_term = node['label']
                if concept_term.lower() in section_content.lower():
                    section_concepts.append({
                        'id': node['id'],
                        'term': concept_term,
                        'first_position': section_content.lower().find(concept_term.lower())
                    })
            
            # Sort concepts by order of appearance
            section_concepts.sort(key=lambda x: x['first_position'])
            
            # Add to ordered concepts
            ordered_concepts.extend(section_concepts)
        
        # Remove duplicates while preserving order
        seen_concepts = set()
        ordered_concepts_unique = []
        for concept in ordered_concepts:
            if concept['id'] not in seen_concepts:
                seen_concepts.add(concept['id'])
                ordered_concepts_unique.append(concept)
        
        # Build the dependency tree
        # The tree is built based on concept ordering and relationships
        for i, concept in enumerate(ordered_concepts_unique):
            # Add node to tree
            dependency_tree['nodes'].append({
                'id': concept['id'],
                'label': concept['term'],
                'level': self._determine_concept_level(concept, ordered_concepts_unique[:i])
            })
            
            # Link to prerequisite concepts
            prerequisites = self._identify_concept_prerequisites(concept, ordered_concepts_unique[:i])
            for prereq in prerequisites:
                dependency_tree['links'].append({
                    'source': prereq['id'],
                    'target': concept['id'],
                    'strength': prereq['strength']
                })
        
        self.dependency_tree = dependency_tree
        return dependency_tree
    
    def _determine_concept_level(self, concept, prior_concepts):
        """
        Determine the hierarchy level of a concept.
        
        Args:
            concept (dict): The concept to determine level for
            prior_concepts (list): List of concepts that appear earlier
            
        Returns:
            Integer level (1 is highest level, higher numbers are more specific/dependent)
        """
        # If no prior concepts, this is a root concept (level 1)
        if not prior_concepts:
            return 1
        
        # Check if this concept is a subterm of any prior concept
        for prior in prior_concepts:
            # If this concept is a more specific version of a prior concept, increase level
            if concept['term'].lower() != prior['term'].lower() and concept['term'].lower() in prior['term'].lower():
                return 2
            
            # If this concept contains a prior concept, it might be a higher level concept
            if prior['term'].lower() in concept['term'].lower():
                return 1
        
        # Default level based on position
        return min(3, 1 + len(prior_concepts) // 5)
    
    def _identify_concept_prerequisites(self, concept, prior_concepts, max_prereqs=3):
        """
        Identify prerequisite concepts for a given concept.
        
        Args:
            concept (dict): The concept to find prerequisites for
            prior_concepts (list): List of concepts that appear earlier
            max_prereqs (int): Maximum number of prerequisites to identify
            
        Returns:
            List of prerequisite concepts with strength scores
        """
        if not prior_concepts:
            return []
        
        prerequisites = []
        
        # Calculate relationship strengths with prior concepts
        for prior in prior_concepts:
            # Skip if same concept
            if concept['id'] == prior['id']:
                continue
            
            # Calculate relationship strength
            term_similarity = self._calculate_term_similarity(concept['term'], prior['term'])
            
            # Check if prior concept is contained in this concept
            subterm_score = 0
            if prior['term'].lower() in concept['term'].lower():
                subterm_score = 0.7
            
            # Check if concepts are connected in the concept graph
            graph_connection = 0
            for edge in self.concept_graph.get('edges', []):
                if (edge['source'] == concept['id'] and edge['target'] == prior['id']) or \
                   (edge['target'] == concept['id'] and edge['source'] == prior['id']):
                    graph_connection = edge['weight']
                    break
            
            # Calculate overall prerequisite strength
            strength = max(term_similarity * 0.5, subterm_score, graph_connection)
            
            if strength > 0.3:  # Threshold for prerequisite relationship
                prerequisites.append({
                    'id': prior['id'],
                    'term': prior['term'],
                    'strength': strength
                })
        
        # Sort by strength and return top prerequisites
        prerequisites.sort(key=lambda x: x['strength'], reverse=True)
        return prerequisites[:max_prereqs]

In [22]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from PIL import Image
import pytesseract
from typing import Dict, List, Tuple, Any, Optional


class VisualizationElementExtractor:
    """
    A class focused on identifying and processing visual elements beyond just images in PDFs.
    This includes diagrams, charts, graphs, and understanding their relationships with text.
    """
    
    def __init__(self, ocr_engine=None, diagram_classifier=None):
        """
        Initialize visualization analysis tools.
        
        Args:
            ocr_engine: Optional OCR engine to use (defaults to pytesseract if None)
            diagram_classifier: Optional pre-trained classifier for diagram types
        """
        self.ocr_engine = ocr_engine or pytesseract
        self.diagram_classifier = diagram_classifier
        
    def extract_diagrams(self, extraction_result: Dict) -> Dict:
        """
        Identify and classify diagrams (flow charts, process diagrams, etc.)
        
        Args:
            extraction_result: Dictionary containing PDF extraction results
            
        Returns:
            Dictionary with identified diagrams and their classification
        """
        diagrams = []
        
        for idx, image_info in enumerate(extraction_result.get('images', [])):
            image = image_info.get('image_data')
            if image is None:
                continue
                
            # Determine if the image is a diagram
            if self._is_diagram(image):
                diagram_type = self._classify_diagram(image)
                diagram_text = self._extract_diagram_text(image)
                diagram_components = self._identify_diagram_components(image)
                
                diagrams.append({
                    'id': f"diagram_{idx}",
                    'page_num': image_info.get('page_num'),
                    'position': image_info.get('position'),
                    'type': diagram_type,
                    'text': diagram_text,
                    'components': diagram_components,
                    'connections': self._extract_component_connections(image, diagram_components)
                })
        
        # Add diagrams to the extraction result
        extraction_result['diagrams'] = diagrams
        return extraction_result
    
    def analyze_figures(self, extraction_result: Dict) -> Dict:
        """
        Analyze figures, charts, and graphs to extract data and meaning.
        
        Args:
            extraction_result: Dictionary containing PDF extraction results
            
        Returns:
            Dictionary with analyzed figures and their data
        """
        figures = []
        
        for idx, image_info in enumerate(extraction_result.get('images', [])):
            image = image_info.get('image_data')
            if image is None:
                continue
                
            # Check if image is a chart or graph
            visualization_type = self.classify_visualization_types(image)
            
            if visualization_type in ['chart', 'graph', 'plot']:
                figure_data = {
                    'id': f"figure_{idx}",
                    'page_num': image_info.get('page_num'),
                    'position': image_info.get('position'),
                    'type': visualization_type,
                    'subtype': self._get_chart_subtype(image, visualization_type),
                    'axes_labels': self._extract_axes_labels(image),
                    'legend_items': self._extract_legend(image),
                    'data_points': self._extract_data_points(image, visualization_type),
                    'chart_title': self._extract_chart_title(image)
                }
                figures.append(figure_data)
        
        # Add figures to the extraction result
        extraction_result['figures'] = figures
        return extraction_result
    
    def extract_visual_relationships(self, extraction_result: Dict) -> Dict:
        """
        Understand relationships between visuals and text in the document.
        
        Args:
            extraction_result: Dictionary containing PDF extraction results
            
        Returns:
            Dictionary with visual-text relationships identified
        """
        visual_relationships = []
        
        # Get all visual elements (images, diagrams, figures)
        visuals = (
            extraction_result.get('images', []) + 
            extraction_result.get('diagrams', []) + 
            extraction_result.get('figures', [])
        )
        
        text_blocks = extraction_result.get('text_blocks', [])
        
        for visual in visuals:
            page_num = visual.get('page_num')
            visual_position = visual.get('position')
            
            # Skip if missing critical information
            if not page_num or not visual_position:
                continue
            
            # Find related text based on proximity and references
            related_text = self._find_related_text(visual, text_blocks, page_num)
            
            # Determine relationship type (caption, explanation, reference)
            relationship_type = self._determine_relationship_type(visual, related_text)
            
            visual_relationships.append({
                'visual_id': visual.get('id'),
                'visual_type': visual.get('type', 'image'),
                'page_num': page_num,
                'related_text_blocks': related_text,
                'relationship_type': relationship_type,
                'visual_context': self._extract_visual_context(visual, related_text)
            })
        
        # Add visual relationships to the extraction result
        extraction_result['visual_relationships'] = visual_relationships
        return extraction_result
    
    def classify_visualization_types(self, image: np.ndarray) -> str:
        """
        Determine the type of visualization (chart, diagram, graph, etc.)
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String identifying the visualization type
        """
        # Convert to grayscale if it's not already
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # Extract features that help identify visualization type
        edge_density = self._calculate_edge_density(gray)
        text_density = self._calculate_text_density(image)
        has_grid_lines = self._detect_grid_lines(gray)
        color_variety = self._calculate_color_variety(image)
        
        # Use features to classify
        if edge_density > 0.15 and edge_density < 0.3 and text_density < 0.1:
            if has_grid_lines:
                return "chart"
            else:
                return "diagram"
        elif has_grid_lines and text_density > 0.1:
            return "graph"
        elif color_variety > 20 and text_density < 0.05:
            return "illustration"
        elif text_density > 0.2:
            return "text_with_visuals"
        else:
            return "image"
    
    def _is_diagram(self, image: np.ndarray) -> bool:
        """
        Determine if an image is likely a diagram rather than a photo or other image type.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating if the image is likely a diagram
        """
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Calculate features indicative of diagrams
        edge_density = self._calculate_edge_density(gray)
        color_count = self._count_distinct_colors(image)
        text_regions = self._count_text_regions(image)
        
        # Diagrams typically have:
        # 1. High edge density
        # 2. Limited color palette
        # 3. Multiple text regions
        return edge_density > 0.1 and color_count < 20 and text_regions > 3
    
    def _classify_diagram(self, image: np.ndarray) -> str:
        """
        Classify the type of diagram.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String identifying the diagram type
        """
        # If a pre-trained classifier was provided, use it
        if self.diagram_classifier:
            return self.diagram_classifier.predict(image)
        
        # Otherwise use heuristics to classify
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Detect lines in the image
        edges = cv2.Canny(gray, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)
        
        if lines is None:
            return "unknown_diagram"
            
        # Count horizontal, vertical, and diagonal lines
        h_lines, v_lines, d_lines = 0, 0, 0
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
            
            if angle < 10 or angle > 170:
                h_lines += 1
            elif 80 < angle < 100:
                v_lines += 1
            else:
                d_lines += 1
        
        # Extract text to look for keywords
        text = self._extract_diagram_text(image).lower()
        
        # Classify based on features
        if "flow" in text or "process" in text or (h_lines > 5 and v_lines > 5 and d_lines > 5):
            return "flowchart"
        elif "class" in text or "uml" in text or (h_lines > 10 and v_lines > 10 and d_lines < 5):
            return "uml_diagram"
        elif "entity" in text or "relationship" in text or (h_lines > 10 and v_lines > 10):
            return "er_diagram"
        elif "network" in text or "topology" in text or (d_lines > h_lines + v_lines):
            return "network_diagram"
        elif "sequence" in text or (h_lines > v_lines and d_lines < 5):
            return "sequence_diagram"
        else:
            return "general_diagram"
    
    def _extract_diagram_text(self, image: np.ndarray) -> str:
        """
        Extract text from a diagram using OCR.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String containing extracted text
        """
        # Use the OCR engine to extract text
        if isinstance(self.ocr_engine, type(pytesseract)):
            text = pytesseract.image_to_string(Image.fromarray(image))
        else:
            # Assuming the OCR engine has a compatible interface
            text = self.ocr_engine.extract_text(image)
            
        return text
    
    def _identify_diagram_components(self, image: np.ndarray) -> List[Dict]:
        """
        Identify components within a diagram (boxes, circles, arrows, etc.)
        
        Args:
            image: Image data as numpy array
            
        Returns:
            List of dictionaries containing component information
        """
        components = []
        
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Threshold the image
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for i, contour in enumerate(contours):
            # Skip small contours
            if cv2.contourArea(contour) < 100:
                continue
                
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Determine shape type
            shape_type = self._identify_shape(contour)
            
            # Extract text within the component
            component_image = image[y:y+h, x:x+w]
            component_text = self._extract_diagram_text(component_image)
            
            components.append({
                'id': f"component_{i}",
                'type': shape_type,
                'position': {'x': int(x), 'y': int(y), 'width': int(w), 'height': int(h)},
                'text': component_text
            })
            
        return components
    
    def _extract_component_connections(self, image: np.ndarray, components: List[Dict]) -> List[Dict]:
        """
        Extract connections between diagram components (arrows, lines).
        
        Args:
            image: Image data as numpy array
            components: List of identified components
            
        Returns:
            List of dictionaries describing connections between components
        """
        connections = []
        
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Create a mask excluding components to isolate lines/arrows
        mask = np.ones_like(gray) * 255
        
        for component in components:
            pos = component['position']
            x, y, w, h = pos['x'], pos['y'], pos['width'], pos['height']
            mask[y:y+h, x:x+w] = 0
            
        # Apply mask to the grayscale image
        masked_gray = cv2.bitwise_and(gray, mask)
        
        # Detect edges and lines
        edges = cv2.Canny(masked_gray, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=30, minLineLength=30, maxLineGap=10)
        
        if lines is None:
            return connections
        
        # Group line segments that likely form a single connection
        grouped_lines = self._group_line_segments(lines)
        
        for i, line_group in enumerate(grouped_lines):
            # Determine start and end points
            start_point, end_point = self._get_line_endpoints(line_group)
            
            # Find components connected by this line
            start_component = self._find_closest_component(start_point, components)
            end_component = self._find_closest_component(end_point, components)
            
            if start_component and end_component and start_component != end_component:
                # Determine if it's an arrow and its direction
                is_arrow, direction = self._check_arrow_properties(line_group, gray)
                
                connections.append({
                    'id': f"connection_{i}",
                    'from_component': start_component,
                    'to_component': end_component,
                    'type': 'arrow' if is_arrow else 'line',
                    'direction': direction if is_arrow else None,
                    'path': [{'x': int(x1), 'y': int(y1), 'x2': int(x2), 'y2': int(y2)} 
                            for x1, y1, x2, y2 in line_group]
                })
        
        return connections
    
    def _get_chart_subtype(self, image: np.ndarray, visualization_type: str) -> str:
        """
        Determine the specific subtype of chart or graph.
        
        Args:
            image: Image data as numpy array
            visualization_type: General visualization type
            
        Returns:
            String specifying the chart subtype
        """
        if visualization_type == 'chart':
            # Detect features specific to chart types
            has_bars = self._detect_bars(image)
            has_pie_segments = self._detect_pie_segments(image)
            has_lines = self._detect_lines(image)
            has_scatter_points = self._detect_scatter_points(image)
            
            if has_bars:
                # Check if bars are horizontal or vertical
                horizontal_bars = self._detect_horizontal_orientation(image)
                if horizontal_bars:
                    return "horizontal_bar_chart"
                else:
                    return "vertical_bar_chart"
            elif has_pie_segments:
                return "pie_chart"
            elif has_lines and has_scatter_points:
                return "scatter_line_chart"
            elif has_lines:
                return "line_chart"
            elif has_scatter_points:
                return "scatter_plot"
            else:
                return "unknown_chart"
        elif visualization_type == 'graph':
            # Detect graph specific features
            has_nodes = self._detect_nodes(image)
            has_edges = self._detect_edges(image)
            
            if has_nodes and has_edges:
                return "network_graph"
            elif self._detect_tree_structure(image):
                return "tree_graph"
            else:
                return "generic_graph"
        else:
            return "unknown_visualization"
    
    def _extract_axes_labels(self, image: np.ndarray) -> Dict[str, str]:
        """
        Extract labels for the x and y axes from charts and graphs.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Dictionary with x and y axis labels
        """
        # Get image dimensions
        height, width = image.shape[:2]
        
        # Define regions where axis labels are typically found
        x_axis_region = image[int(height * 0.85):height, int(width * 0.1):int(width * 0.9)]
        y_axis_region = image[int(height * 0.1):int(height * 0.9), 0:int(width * 0.15)]
        
        # Extract text from these regions
        x_label = self._extract_diagram_text(x_axis_region).strip()
        y_label = self._extract_diagram_text(y_axis_region).strip()
        
        return {'x_axis': x_label, 'y_axis': y_label}
    
    def _extract_legend(self, image: np.ndarray) -> List[Dict]:
        """
        Extract legend items from charts and graphs.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            List of legend items with color and label
        """
        # Common places for legends are top right, bottom right, or bottom of image
        height, width = image.shape[:2]
        
        # Check top right corner
        top_right = image[0:int(height * 0.3), int(width * 0.7):width]
        # Check bottom right corner
        bottom_right = image[int(height * 0.7):height, int(width * 0.7):width]
        # Check bottom center
        bottom_center = image[int(height * 0.85):height, int(width * 0.2):int(width * 0.8)]
        
        # Try to extract legend from these regions
        legend_regions = [top_right, bottom_right, bottom_center]
        
        for region in legend_regions:
            legend_items = self._process_legend_region(region)
            if legend_items:
                return legend_items
        
        return []
    
    def _process_legend_region(self, region: np.ndarray) -> List[Dict]:
        """
        Process a potential legend region to extract legend items.
        
        Args:
            region: Image region to process
            
        Returns:
            List of legend items
        """
        legend_items = []
        
        # Convert to grayscale
        if len(region.shape) > 2 and region.shape[2] > 1:
            gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
        else:
            gray = region.copy()
            
        # Threshold to find boxes/markers
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            # Skip small contours
            if cv2.contourArea(contour) < 20:
                continue
                
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Check if there's text next to this symbol/color
            text_region = region[max(0, y-5):min(region.shape[0], y+h+5), 
                               min(region.shape[1]-1, x+w+5):min(region.shape[1], x+w+150)]
            
            legend_text = self._extract_diagram_text(text_region).strip()
            
            if legend_text:
                # Get the color of the legend marker
                marker_region = region[y:y+h, x:x+w]
                color = self._get_dominant_color(marker_region)
                
                legend_items.append({
                    'label': legend_text,
                    'color': color,
                    'marker_type': self._identify_marker_type(marker_region)
                })
        
        return legend_items
    
    def _extract_data_points(self, image: np.ndarray, visualization_type: str) -> List:
        """
        Extract data points from charts and graphs where possible.
        This is a simplified version that returns placeholder data.
        
        Args:
            image: Image data as numpy array
            visualization_type: Type of visualization
            
        Returns:
            List of extracted data points (format depends on chart type)
        """
        # For a production system, this would implement computer vision techniques
        # to actually extract data points from the visualization
        
        # Placeholder implementation
        if visualization_type == 'chart':
            # Return placeholder data
            return [
                {'x': 'placeholder', 'y': 0},
                {'x': 'placeholder', 'y': 0}
            ]
        else:
            return []
    
    def _extract_chart_title(self, image: np.ndarray) -> str:
        """
        Extract the title of a chart or graph.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String containing the chart title
        """
        # Chart titles are typically at the top
        height, width = image.shape[:2]
        
        # Extract from top region
        title_region = image[0:int(height * 0.15), int(width * 0.1):int(width * 0.9)]
        
        title = self._extract_diagram_text(title_region).strip()
        
        return title
    
    def _find_related_text(self, visual: Dict, text_blocks: List[Dict], page_num: int) -> List[Dict]:
        """
        Find text blocks related to a visual element.
        
        Args:
            visual: Visual element dictionary
            text_blocks: List of text blocks from extraction
            page_num: Page number of the visual
            
        Returns:
            List of related text blocks
        """
        related_blocks = []
        visual_position = visual.get('position', {})
        
        # Skip if no position information
        if not visual_position:
            return related_blocks
            
        visual_id = visual.get('id', '')
        
        for block in text_blocks:
            # Only consider blocks on the same page
            if block.get('page_num') != page_num:
                continue
                
            block_text = block.get('text', '').lower()
            
            # Check for direct references
            if visual_id in block_text:
                related_blocks.append(block)
                continue
                
            # Check for reference patterns like "figure 1", "diagram 2", etc.
            visual_type = visual.get('type', 'image')
            if f"{visual_type}" in block_text or "fig" in block_text or "chart" in block_text:
                related_blocks.append(block)
                continue
                
            # Check for proximity
            block_position = block.get('position', {})
            if block_position and self._check_proximity(visual_position, block_position):
                related_blocks.append(block)
        
        return related_blocks
    
    def _determine_relationship_type(self, visual: Dict, related_text: List[Dict]) -> str:
        """
        Determine the type of relationship between visual and text.
        
        Args:
            visual: Visual element dictionary
            related_text: List of related text blocks
            
        Returns:
            String describing relationship type
        """
        if not related_text:
            return "no_relationship"
            
        # Check for caption patterns
        for block in related_text:
            text = block.get('text', '').lower()
            
            if any(x in text for x in ["figure", "fig.", "fig", "chart", "graph", "diagram", "illustration"]):
                # Check if it's just a reference or a full caption
                if len(text) > 50:
                    return "caption"
                else:
                    return "reference"
        
        # Check position relative to the visual
        visual_pos = visual.get('position', {})
        
        if not visual_pos:
            return "unknown_relationship"
            
        # Check if text is directly below the visual (potential caption)
        for block in related_text:
            block_pos = block.get('position', {})
            
            if block_pos:
                # If text is directly below the visual and close
                if (block_pos.get('y') > visual_pos.get('y') + visual_pos.get('height') and
                    self._check_horizontal_overlap(visual_pos, block_pos)):
                    return "caption"
        
        # Check if visual is embedded in text (illustration)
        text_surrounds = self._check_if_surrounded_by_text(visual, related_text)
        if text_surrounds:
            return "illustration"
            
        return "related_content"
    
    def _extract_visual_context(self, visual: Dict, related_text: List[Dict]) -> str:
        """
        Extract the context in which the visual is used.
        
        Args:
            visual: Visual element dictionary
            related_text: List of related text blocks
            
        Returns:
            String describing the visual context
        """
        # Join all related text
        all_text = " ".join([block.get('text', '') for block in related_text])
        
        # Look for context clues
        context_keywords = {
            "example": "example_context",
            "illustrat": "illustration_context",
            "demonstrat": "demonstration_context",
            "explain": "explanation_context",
            "show": "demonstration_context",
            "depict": "depiction_context",
            "represent": "representation_context"
        }
        
        for keyword, context in context_keywords.items():
            if keyword in all_text.lower():
                return context
                
        return "general_context"
    
    def _calculate_edge_density(self, gray_image: np.ndarray) -> float:
        """
        Calculate the density of edges in an image.
        
        Args:
            gray_image: Grayscale image as numpy array
            
        Returns:
            Float representing edge density
        """
        edges = cv2.Canny(gray_image, 50, 150)
        return np.count_nonzero(edges) / (edges.shape[0] * edges.shape[1])
    
    def _count_distinct_colors(self, image: np.ndarray) -> int:
        """
        Count approximately how many distinct colors are in an image.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Integer count of distinct colors
        """
        # Resize image to make computation faster
        small = cv2.resize(image, (100, 100))
        
        # Reshape to a list of pixels
        pixels = small.reshape(-1, 3) if len(small.shape) > 2 else small.reshape(-1, 1)
        
        # Convert to float and scale to [0,1]
        pixels = pixels.astype('float32') / 255.0
        
        # Cluster colors using DBSCAN
        db = DBSCAN(eps=0.1, min_samples=5).fit(pixels)
        
        # Get number of clusters (-1 is noise)
        labels = set(db.labels_)
        n_colors = len(labels - {-1}) if -1 in labels else len(labels)
        
        return n_colors
    
    def _count_text_regions(self, image: np.ndarray) -> int:
        """
        Count the number of text regions in an image.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Integer count of text regions
        """
        # Use OCR to detect text regions
        if isinstance(self.ocr_engine, type(pytesseract)):
            data = pytesseract.image_to_data(Image.fromarray(image), output_type=pytesseract.Output.DICT)
            # Filter out empty text
            text_regions = [i for i, text in enumerate(data['text']) if text.strip()]
            return len(text_regions)
        else:
            # Simplified fallback if OCR engine doesn't support region detection
            # Count distinct areas with text based on connected components
            
            # Convert to grayscale if needed
            if len(image.shape) > 2 and image.shape[2] > 1:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image
                
            # Apply threshold
            _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
            
            # Find connected components
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary)
            
            # Filter small components (noise)
            text_regions = 0
            for i in range(1, num_labels):  # Start from 1 to skip background
                area = stats[i, cv2.CC_STAT_AREA]
                if 50 < area < 5000:  # Typical text size range
                    text_regions += 1
                    
            return text_regions
    
    def _identify_shape(self, contour: np.ndarray) -> str:
        """
        Identify the shape type from a contour.
        
        Args:
            contour: Contour points from OpenCV
            
        Returns:
            String describing the shape type
        """
        # Approximate the contour
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Get contour features
        vertices = len(approx)
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        # Circle detection using circularity
        circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
        
        if 0.85 < circularity < 1.15:
            return "circle"
        elif vertices == 4:
            # Check if it's a square or rectangle
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h if h > 0 else 0
            
            if 0.95 < aspect_ratio < 1.05:
                return "square"
            else:
                return "rectangle"
        elif vertices == 3:
            return "triangle"
        elif vertices == 5:
            return "pentagon"
        elif vertices == 6:
            return "hexagon"
        elif vertices > 6:
            return "polygon"
        else:
            return "unknown_shape"
    
    def _group_line_segments(self, lines: np.ndarray) -> List[List]:
        """
        Group line segments that are likely part of the same connection.
        
        Args:
            lines: Line segments from HoughLinesP
            
        Returns:
            List of grouped line segments
        """
        if lines is None:
            return []
            
        # Extract line segments into a list of (x1, y1, x2, y2)
        line_segments = [line[0] for line in lines]
        
        # Group lines by proximity and angle
        groups = []
        processed = set()
        
        for i, line1 in enumerate(line_segments):
            if i in processed:
                continue
                
            group = [line1]
            processed.add(i)
            
            x1, y1, x2, y2 = line1
            angle1 = np.arctan2(y2 - y1, x2 - x1)
            
            for j, line2 in enumerate(line_segments):
                if j in processed or i == j:
                    continue
                    
                x3, y3, x4, y4 = line2
                angle2 = np.arctan2(y4 - y3, x4 - x3)
                
                # Check if angles are similar (within 15 degrees)
                if abs(angle1 - angle2) < 0.26 or abs(abs(angle1 - angle2) - np.pi) < 0.26:
                    # Check if endpoints are close
                    if (self._distance((x1, y1), (x3, y3)) < 20 or 
                        self._distance((x1, y1), (x4, y4)) < 20 or
                        self._distance((x2, y2), (x3, y3)) < 20 or
                        self._distance((x2, y2), (x4, y4)) < 20):
                        
                        group.append(line2)
                        processed.add(j)
            
            groups.append(group)
            
        return groups
    
    def _distance(self, p1: Tuple[int, int], p2: Tuple[int, int]) -> float:
        """
        Calculate Euclidean distance between two points.
        
        Args:
            p1: First point (x, y)
            p2: Second point (x, y)
            
        Returns:
            Float distance
        """
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    def _get_line_endpoints(self, line_group: List) -> Tuple[Tuple[int, int], Tuple[int, int]]:
        """
        Find the endpoints of a group of connected line segments.
        
        Args:
            line_group: List of line segments
            
        Returns:
            Tuple of (start_point, end_point)
        """
        # Collect all endpoints
        endpoints = []
        for x1, y1, x2, y2 in line_group:
            endpoints.append((x1, y1))
            endpoints.append((x2, y2))
            
        # Count occurrences of each endpoint
        from collections import Counter
        endpoint_counts = Counter(endpoints)
        
        # Endpoints that appear only once are the actual endpoints of the line
        true_endpoints = [point for point, count in endpoint_counts.items() if count == 1]
        
        # If we have two endpoints, that's our start and end
        if len(true_endpoints) == 2:
            return true_endpoints[0], true_endpoints[1]
        elif len(true_endpoints) > 2:
            # If more than 2 endpoints, find the two that are furthest apart
            max_distance = 0
            furthest_pair = (true_endpoints[0], true_endpoints[0])
            
            for i, p1 in enumerate(true_endpoints):
                for p2 in true_endpoints[i+1:]:
                    dist = self._distance(p1, p2)
                    if dist > max_distance:
                        max_distance = dist
                        furthest_pair = (p1, p2)
                        
            return furthest_pair
        else:
            # If no true endpoints (closed shape), just return any two points
            return endpoints[0], endpoints[-1]
    
    def _find_closest_component(self, point: Tuple[int, int], components: List[Dict]) -> Optional[str]:
        """
        Find the component closest to a given point.
        
        Args:
            point: Point coordinates (x, y)
            components: List of component dictionaries
            
        Returns:
            ID of the closest component or None
        """
        closest_dist = float('inf')
        closest_id = None
        
        for component in components:
            pos = component['position']
            # Calculate center of component
            center_x = pos['x'] + pos['width'] / 2
            center_y = pos['y'] + pos['height'] / 2
            
            # Check if point is inside the component
            if (pos['x'] <= point[0] <= pos['x'] + pos['width'] and 
                pos['y'] <= point[1] <= pos['y'] + pos['height']):
                return component['id']
                
            # Check distance to component center
            dist = self._distance(point, (center_x, center_y))
            
            if dist < closest_dist:
                closest_dist = dist
                closest_id = component['id']
                
        # Only return if reasonably close (within 30 pixels)
        return closest_id if closest_dist < 30 else None
    
    def _check_arrow_properties(self, line_group: List, gray_image: np.ndarray) -> Tuple[bool, str]:
        """
        Check if a line group represents an arrow and determine its direction.
        
        Args:
            line_group: List of line segments
            gray_image: Grayscale image
            
        Returns:
            Tuple of (is_arrow, direction)
        """
        # Get the endpoints of the line
        start_point, end_point = self._get_line_endpoints(line_group)
        
        # Define a region around the end point to check for arrowhead
        x, y = end_point
        region_size = 20
        x1, y1 = max(0, x - region_size), max(0, y - region_size)
        x2, y2 = min(gray_image.shape[1] - 1, x + region_size), min(gray_image.shape[0] - 1, y + region_size)
        
        # Extract the region around the endpoint
        end_region = gray_image[y1:y2, x1:x2]
        
        # Apply edge detection
        edges = cv2.Canny(end_region, 50, 150)
        
        # Find lines in the end region
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=10, minLineLength=5, maxLineGap=5)
        
        if lines is None or len(lines) < 2:
            return False, "none"
            
        # Count lines at angles consistent with arrowheads
        arrow_lines = 0
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            
            # Check if the angle is consistent with arrowheads
            # (approximately 30-60 degrees from the main line direction)
            if 30 < abs(angle) < 150:
                arrow_lines += 1
                
        # If we have at least 2 lines at appropriate angles, it's an arrow
        is_arrow = arrow_lines >= 2
        
        # Determine direction based on start and end points
        if is_arrow:
            # Calculate angle of the line
            dx = end_point[0] - start_point[0]
            dy = end_point[1] - start_point[1]
            angle = np.arctan2(dy, dx) * 180 / np.pi
            
            # Determine direction based on angle
            if -45 <= angle <= 45:
                direction = "right"
            elif 45 < angle <= 135:
                direction = "down"
            elif angle > 135 or angle < -135:
                direction = "left"
            else:
                direction = "up"
        else:
            direction = "none"
            
        return is_arrow, direction
    
    def _calculate_text_density(self, image: np.ndarray) -> float:
        """
        Calculate the density of text in an image.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Float representing text density
        """
        # Extract text regions using OCR
        if isinstance(self.ocr_engine, type(pytesseract)):
            data = pytesseract.image_to_data(Image.fromarray(image), output_type=pytesseract.Output.DICT)
            
            # Calculate total area of text regions
            text_area = 0
            for i, text in enumerate(data['text']):
                if text.strip():
                    width = data['width'][i]
                    height = data['height'][i]
                    text_area += width * height
                    
            # Calculate image area
            image_area = image.shape[0] * image.shape[1]
            
            return text_area / image_area if image_area > 0 else 0
        else:
            # Simplified alternative approach
            # Convert to grayscale if needed
            if len(image.shape) > 2 and image.shape[2] > 1:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image.copy()
                
            # Apply threshold to find potential text regions
            _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
            
            # Find connected components
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary)
            
            # Calculate area of components likely to be text (filtering by size)
            text_area = 0
            for i in range(1, num_labels):  # Start from 1 to skip background
                area = stats[i, cv2.CC_STAT_AREA]
                width = stats[i, cv2.CC_STAT_WIDTH]
                height = stats[i, cv2.CC_STAT_HEIGHT]
                
                if 20 < area < 5000 and 1 < height < 100 and width > height:
                    text_area += area
                    
            # Calculate image area
            image_area = gray.shape[0] * gray.shape[1]
            
            return text_area / image_area if image_area > 0 else 0
    
    def _detect_grid_lines(self, gray_image: np.ndarray) -> bool:
        """
        Detect if an image contains grid lines typical of charts and graphs.
        
        Args:
            gray_image: Grayscale image as numpy array
            
        Returns:
            Boolean indicating presence of grid lines
        """
        # Apply edge detection
        edges = cv2.Canny(gray_image, 50, 150)
        
        # Find lines
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)
        
        if lines is None:
            return False
            
        # Count horizontal and vertical lines
        h_lines, v_lines = 0, 0
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
            
            if angle < 5 or angle > 175:
                h_lines += 1
            elif 85 < angle < 95:
                v_lines += 1
                
        # A grid should have multiple horizontal and vertical lines
        return h_lines >= 3 and v_lines >= 3
    
    def _calculate_color_variety(self, image: np.ndarray) -> int:
        """
        Calculate the variety of colors in an image.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Integer representing color variety
        """
        # If grayscale, return 0
        if len(image.shape) < 3:
            return 0
            
        # Resize image to make computation faster
        small = cv2.resize(image, (50, 50))
        
        # Convert to HSV for better color analysis
        hsv = cv2.cvtColor(small, cv2.COLOR_BGR2HSV)
        
        # Count distinct hues
        hues = hsv[:, :, 0].flatten()
        
        # Use bins to count approximately distinct hues
        hist, _ = np.histogram(hues, bins=18, range=(0, 180))
        
        # Count non-empty bins
        return np.count_nonzero(hist > 5)
    
    def _detect_bars(self, image: np.ndarray) -> bool:
        """
        Detect if an image contains bars typical of bar charts.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating presence of bars
        """
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply threshold to identify potential bars
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Count rectangular contours of appropriate size
        bar_count = 0
        
        for contour in contours:
            # Skip small contours
            if cv2.contourArea(contour) < 100:
                continue
                
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Check if it's a rectangle (approximate)
            rect_area = w * h
            contour_area = cv2.contourArea(contour)
            
            if contour_area / rect_area > 0.8:
                # Check dimensions
                aspect_ratio = max(w / h if h > 0 else 0, h / w if w > 0 else 0)
                
                if aspect_ratio > 2:
                    bar_count += 1
                    
        return bar_count >= 3
    
    def _detect_pie_segments(self, image: np.ndarray) -> bool:
        """
        Detect if an image contains pie segments typical of pie charts.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating presence of pie segments
        """
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply threshold to identify potential segments
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Find contours that could be pie segments
        pie_segment_count = 0
        
        for contour in contours:
            # Skip small contours
            if cv2.contourArea(contour) < 200:
                continue
                
            # Check if it's roughly a circular sector
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            contour_area = cv2.contourArea(contour)
            
            if contour_area / hull_area > 0.7:
                # Get enclosing circle
                (x, y), radius = cv2.minEnclosingCircle(contour)
                circle_area = np.pi * radius * radius
                
                # Check if it's a sector of a circle
                if 0.2 < contour_area / circle_area < 0.8:
                    pie_segment_count += 1
                    
        return pie_segment_count >= 3
    
    def _detect_lines(self, image: np.ndarray) -> bool:
        """
        Detect if an image contains lines typical of line charts.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating presence of lines
        """
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply edge detection
        edges = cv2.Canny(gray, 50, 150)
        
        # Find lines
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=30, minLineLength=30, maxLineGap=10)
        
        if lines is None:
            return False
            
        # Count lines that are not horizontal or vertical
        diagonal_lines = 0
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
            
            # Not horizontal or vertical
            if 10 < angle < 80 or 100 < angle < 170:
                diagonal_lines += 1
                
        return diagonal_lines >= 2
    
    def _detect_scatter_points(self, image: np.ndarray) -> bool:
        """
        Detect if an image contains scatter points typical of scatter plots.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating presence of scatter points
        """
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply threshold to identify potential points
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Count small, roughly circular contours
        scatter_point_count = 0
        
        for contour in contours:
            # Only consider small contours
            area = cv2.contourArea(contour)
            if 10 < area < 200:
                # Check if it's roughly circular
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
                
                if 0.7 < circularity < 1.3:
                    scatter_point_count += 1
                    
        return scatter_point_count >= 5
    
    def _detect_horizontal_orientation(self, image: np.ndarray) -> bool:
        """
        Detect if bars in a bar chart are oriented horizontally.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating horizontal orientation
        """
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply threshold to identify potential bars
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Count horizontal vs vertical bars
        horizontal_bars = 0
        vertical_bars = 0
        
        for contour in contours:
            # Skip small contours
            if cv2.contourArea(contour) < 100:
                continue
                
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Check if it's a rectangle (approximate)
            rect_area = w * h
            contour_area = cv2.contourArea(contour)
            
            if contour_area / rect_area > 0.8:
                # Check dimensions
                if w > 2 * h:
                    horizontal_bars += 1
                elif h > 2 * w:
                    vertical_bars += 1
                    
        return horizontal_bars > vertical_bars
    
    def _detect_nodes(self, image: np.ndarray) -> bool:
        """
        Detect if an image contains nodes typical of network graphs.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating presence of nodes
        """
        # Similar to scatter points but with connections
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply threshold to identify potential nodes
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Count nodes
        node_count = 0
        
        for contour in contours:
            # Only consider small contours
            area = cv2.contourArea(contour)
            if 20 < area < 500:
                # Check if it's roughly circular
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
                
                if 0.7 < circularity < 1.3:
                    node_count += 1
                    
        return node_count >= 3
    
    def _detect_edges(self, image: np.ndarray) -> bool:
        """
        Detect if an image contains edges/connections typical of network graphs.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating presence of edges
        """
        # Similar to line detection but focused on graph connections
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply edge detection
        edges = cv2.Canny(gray, 50, 150)
        
        # Find lines
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=30, minLineLength=30, maxLineGap=10)
        
        return lines is not None and len(lines) >= 3
    
    def _detect_tree_structure(self, image: np.ndarray) -> bool:
        """
        Detect if an image contains a tree structure with hierarchy.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating presence of tree structure
        """
        # This is a complex detection that requires analyzing the structure
        # For simplicity, we'll use a heuristic approach
        
        # Check for nodes
        has_nodes = self._detect_nodes(image)
        
        # Check for edges
        has_edges = self._detect_edges(image)
        
        # If no nodes or edges, not a tree
        if not has_nodes or not has_edges:
            return False
            
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply edge detection
        edges = cv2.Canny(gray, 50, 150)
        
        # Find lines
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=30, minLineLength=30, maxLineGap=10)
        
        if lines is None:
            return False
            
        # Count horizontal and vertical lines which are typical in tree structures
        h_lines, v_lines = 0, 0
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
            
            if angle < 10 or angle > 170:
                h_lines += 1
            elif 80 < angle < 100:
                v_lines += 1
                
        # Trees typically have more horizontal than vertical lines
        # and have a specific ratio between them
        return h_lines >= 3 and v_lines >= 1 and h_lines > v_lines
    
    def _check_proximity(self, pos1: Dict, pos2: Dict) -> bool:
        """
        Check if two positions are in proximity.
        
        Args:
            pos1: First position dictionary
            pos2: Second position dictionary
            
        Returns:
            Boolean indicating proximity
        """
        # Extract coordinates
        x1, y1 = pos1.get('x', 0), pos1.get('y', 0)
        w1, h1 = pos1.get('width', 0), pos1.get('height', 0)
        
        x2, y2 = pos2.get('x', 0), pos2.get('y', 0)
        w2, h2 = pos2.get('width', 0), pos2.get('height', 0)
        
        # Calculate centers
        center_x1, center_y1 = x1 + w1/2, y1 + h1/2
        center_x2, center_y2 = x2 + w2/2, y2 + h2/2
        
        # Calculate distance
        distance = np.sqrt((center_x1 - center_x2)**2 + (center_y1 - center_y2)**2)
        
        # Check if distance is less than the sum of half the widths and heights
        max_distance = (w1 + h1 + w2 + h2) / 2
        
        return distance < max_distance
    
    def _check_horizontal_overlap(self, pos1: Dict, pos2: Dict) -> bool:
        """
        Check if two positions overlap horizontally.
        
        Args:
            pos1: First position dictionary
            pos2: Second position dictionary
            
        Returns:
            Boolean indicating horizontal overlap
        """
        # Extract coordinates
        x1, w1 = pos1.get('x', 0), pos1.get('width', 0)
        x2, w2 = pos2.get('x', 0), pos2.get('width', 0)
        
        # Check overlap
        return (x1 <= x2 <= x1 + w1) or (x2 <= x1 <= x2 + w2)
    
    def _check_if_surrounded_by_text(self, visual: Dict, text_blocks: List[Dict]) -> bool:
        """
        Check if a visual is surrounded by text on multiple sides.
        
        Args:
            visual: Visual element dictionary
            text_blocks: List of text blocks
            
        Returns:
            Boolean indicating if visual is surrounded by text
        """
        visual_pos = visual.get('position', {})
        
        if not visual_pos:
            return False
            
        # Extract coordinates
        x, y = visual_pos.get('x', 0), visual_pos.get('y', 0)
        w, h = visual_pos.get('width', 0), visual_pos.get('height', 0)
        
        # Check for text blocks in each direction
        has_text_left = False
        has_text_right = False
        has_text_above = False
        has_text_below = False
        
        for block in text_blocks:
            block_pos = block.get('position', {})
            
            if not block_pos:
                continue
                
            bx, by = block_pos.get('x', 0), block_pos.get('y', 0)
            bw, bh = block_pos.get('width', 0), block_pos.get('height', 0)
            
            # Check left
            if bx + bw < x and by < y + h and by + bh > y:
                has_text_left = True
                
            # Check right
            if bx > x + w and by < y + h and by + bh > y:
                has_text_right = True
                
            # Check above
            if by + bh < y and bx < x + w and bx + bw > x:
                has_text_above = True
                
            # Check below
            if by > y + h and bx < x + w and bx + bw > x:
                has_text_below = True
                
        # Visual is surrounded if text exists on at least 3 sides
        sides_with_text = sum([has_text_left, has_text_right, has_text_above, has_text_below])
        return sides_with_text >= 3
    
    def _get_dominant_color(self, image: np.ndarray) -> str:
        """
        Get the dominant color in an image region.

        Args:
            image: Image data as numpy array

        Returns:
            String representing the dominant color in hex format
        """
        # If grayscale, convert to BGR
        if len(image.shape) < 3:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        # Reshape the image to be a list of pixels
        pixels = image.reshape(-1, 3)

        # Convert to list of pixels
        pixels = pixels.astype(np.float32)

        # Define criteria for k-means
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

        # Apply k-means to find dominant colors
        k = 3  # Extract top 3 colors
        _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

        # Get counts of each cluster
        unique, counts = np.unique(labels, return_counts=True)

        # Get the most dominant cluster
        dominant_index = unique[np.argmax(counts)]
        dominant_color = centers[dominant_index]

        # Convert to hex
        dominant_color_int = dominant_color.astype(np.uint8)
        hex_color = "#{:02x}{:02x}{:02x}".format(
            dominant_color_int[2], dominant_color_int[1], dominant_color_int[0])

        return hex_color
    
    def _identify_marker_type(self, marker_region: np.ndarray) -> str:
        """
        Identify the type of marker used in a legend.
        
        Args:
            marker_region: Image region containing the marker
            
        Returns:
            String describing the marker type
        """
        # Convert to grayscale if needed
        if len(marker_region.shape) > 2 and marker_region.shape[2] > 1:
            gray = cv2.cvtColor(marker_region, cv2.COLOR_BGR2GRAY)
        else:
            gray = marker_region
            
        # Get shape features
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if not contours:
            return "unknown"
            
        # Get largest contour
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Get contour features
        area = cv2.contourArea(largest_contour)
        perimeter = cv2.arcLength(largest_contour, True)
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Check circularity
        circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
        
        # Check aspect ratio
        aspect_ratio = w / h if h > 0 else 0
        
        # Identify marker type
        if 0.8 < circularity < 1.2:
            return "circle"
        elif 0.8 < aspect_ratio < 1.2 and area / (w * h) > 0.8:
            return "square"
        elif area / (w * h) < 0.6:
            return "triangle"
        elif w > 3 * h or h > 3 * w:
            return "line"
        else:
            return "custom"
            
    def visualize_analysis_results(self, extraction_result: Dict, output_path: str = None) -> np.ndarray:
        """
        Visualize the analysis results for debugging and verification.
        
        Args:
            extraction_result: Dictionary containing extraction and analysis results
            output_path: Optional path to save visualization
            
        Returns:
            Numpy array containing visualization image
        """
        # Create a copy of the first page image as background
        images = extraction_result.get('images', [])
        if not images or 'image_data' not in images[0]:
            # Create a blank canvas if no image is available
            visualization = np.ones((800, 600, 3), dtype=np.uint8) * 255
        else:
            # Use the first image as background
            visualization = images[0]['image_data'].copy()
            
        # Resize if too large
        max_dim = 1200
        h, w = visualization.shape[:2]
        if h > max_dim or w > max_dim:
            if h > w:
                visualization = cv2.resize(visualization, (int(w * max_dim / h), max_dim))
            else:
                visualization = cv2.resize(visualization, (max_dim, int(h * max_dim / w)))
                
        # Draw bounding boxes around identified diagrams
        for diagram in extraction_result.get('diagrams', []):
            pos = diagram.get('position', {})
            if pos:
                x, y = pos.get('x', 0), pos.get('y', 0)
                w, h = pos.get('width', 0), pos.get('height', 0)
                
                # Draw rectangle
                cv2.rectangle(visualization, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Add label
                label = f"Diagram: {diagram.get('type', 'unknown')}"
                cv2.putText(visualization, label, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                        
        # Draw bounding boxes around identified figures
        for figure in extraction_result.get('figures', []):
            pos = figure.get('position', {})
            if pos:
                x, y = pos.get('x', 0), pos.get('y', 0)
                w, h = pos.get('width', 0), pos.get('height', 0)
                
                # Draw rectangle
                cv2.rectangle(visualization, (x, y), (x+w, y+h), (255, 0, 0), 2)
                
                # Add label
                label = f"Figure: {figure.get('subtype', figure.get('type', 'unknown'))}"
                cv2.putText(visualization, label, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                        
        # Draw lines for visual relationships
        for rel in extraction_result.get('visual_relationships', []):
            # Find the visual element
            visual_id = rel.get('visual_id')
            visual_element = None
            
            for element_list in [
                extraction_result.get('images', []),
                extraction_result.get('diagrams', []),
                extraction_result.get('figures', [])
            ]:
                for element in element_list:
                    if element.get('id') == visual_id:
                        visual_element = element
                        break
                if visual_element:
                    break
                    
            if not visual_element:
                continue
                
            # Get visual position
            vis_pos = visual_element.get('position', {})
            if not vis_pos:
                continue
                
            vis_x = vis_pos.get('x', 0) + vis_pos.get('width', 0) // 2
            vis_y = vis_pos.get('y', 0) + vis_pos.get('height', 0) // 2
            
            # Draw connections to related text blocks
            for block in rel.get('related_text_blocks', []):
                block_pos = block.get('position', {})
                if not block_pos:
                    continue
                    
                block_x = block_pos.get('x', 0) + block_pos.get('width', 0) // 2
                block_y = block_pos.get('y', 0) + block_pos.get('height', 0) // 2
                
                # Draw line between visual and text
                cv2.line(visualization, (vis_x, vis_y), (block_x, block_y), (0, 0, 255), 1)
                
                # Add relationship type
                mid_x = (vis_x + block_x) // 2
                mid_y = (vis_y + block_y) // 2
                rel_type = rel.get('relationship_type', 'unknown')
                cv2.putText(visualization, rel_type, (mid_x, mid_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            
        # Save if output path is provided
        if output_path:
            cv2.imwrite(output_path, visualization)
            
        return visualization

In [17]:
# class DomainSpecificProcessor:
#     def __init__(self, domain_type):
#         """
#         Initialize the DomainSpecificProcessor with a specified domain type.
        
#         Args:
#             domain_type (str): The domain type to specialize in (science, math, literature, etc.)
#         """
#         self.domain_type = domain_type.lower() if domain_type else None
#         self.domain_processors = {
#             'science': self._process_science_content,
#             'math': self._process_math_content,
#             'literature': self._process_literature_content,
#             'chemistry': self._process_chemistry_content,
#             'biology': self._process_biology_content,
#             'physics': self._process_physics_content
#         }
        
#         # Domain-specific entity patterns
#         self.domain_patterns = self._initialize_domain_patterns()
    
#     def _initialize_domain_patterns(self):
#         """
#         Initialize regex patterns and rules for different domains.

#         Returns:
#             Dictionary of patterns for each domain
#         """
#         # Simply use the domain rules manager instead of recreating logic
#         domains = ['science', 'math', 'literature', 'chemistry', 'biology', 'physics']
#         patterns = {}
     
#         # Load rules for each domain
#         for domain in domains:
#             patterns[domain] = DomainRulesManager.get_domain_rules(domain)
            
#         return patterns
    
#     def process_domain_content(self, extraction_result):
#         """
#         Process content according to domain-specific rules.
        
#         Args:
#             extraction_result (dict): The extraction result from PDF processor
            
#         Returns:
#             Dictionary with domain-specific processing results added
#         """
#         if not self.domain_type or self.domain_type not in self.domain_processors:
#             # If domain is not supported, return the original extraction result
#             return extraction_result
        
#         # Create a copy of the extraction result to modify
#         processed_result = extraction_result.copy()
        
#         # Apply domain-specific processing
#         domain_processor = self.domain_processors[self.domain_type]
#         domain_specific_data = domain_processor(processed_result)
        
#         # Add domain-specific results to the processed result
#         processed_result['domain_analysis'] = {
#             'domain_type': self.domain_type,
#             'domain_entities': domain_specific_data
#         }
        
#         return processed_result
    
#     def _process_science_content(self, extraction_result):
#         """
#         Process content specific to scientific domains.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary with science-specific entities
#         """
#         scientific_entities = {
#             'terminology': self.extract_scientific_terminology(extraction_result),
#             'methods': self.identify_experiments_and_methods(extraction_result),
#         }
        
#         # Add more specific science subdomain processing if available
#         if self.domain_type in ['biology', 'chemistry', 'physics']:
#             subdomain_processor = self.domain_processors.get(self.domain_type)
#             if subdomain_processor:
#                 scientific_entities.update(subdomain_processor(extraction_result))
        
#         return scientific_entities
    
#     def _process_math_content(self, extraction_result):
#         """
#         Process content specific to mathematical domains.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary with math-specific entities
#         """
#         return {
#             'mathematical_entities': self.extract_mathematical_entities(extraction_result),
#             'problem_classifications': self.classify_math_problems(extraction_result)
#         }
    
#     def _process_literature_content(self, extraction_result):
#         """
#         Process content specific to literature domains by using the LiteraryWorkAnalyzer.

#         Args:
#             extraction_result (dict): The extraction result

#         Returns:
#             Dictionary with literature-specific entities
#         """
#         text_content = extraction_result.get('text_content', '')

#         # Determine work type to use appropriate analyzer
#         work_type = 'general'
#         if 'shakespeare' in extraction_result.get('metadata', {}).get('title', '').lower() or 'shakespeare' in text_content.lower():
#             work_type = 'play:shakespeare'
#         elif re.search(r'([A-Z]{2,}[A-Z\s]*)[\.:]\s', text_content):
#             work_type = 'play'
#         elif re.search(r'Chapter [IVXLCDM]+|Chapter \d+', text_content):
#             work_type = 'novel'

#         # Use the dedicated LiteraryWorkAnalyzer
#         literary_analyzer = LiteraryWorkAnalyzer(work_type)

#         return {
#             'literary_structure': literary_analyzer.identify_scenes(text_content),
#             'literary_devices': literary_analyzer.detect_literary_devices(text_content)
#         }
    
#     def _process_chemistry_content(self, extraction_result):
#         """
#         Process content specific to chemistry domains.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary with chemistry-specific entities
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         compounds = set()
#         reactions = []
#         conditions = []
        
#         patterns = self.domain_patterns.get('chemistry', {})
        
#         # Extract compounds
#         if 'compound_pattern' in patterns:
#             compounds = set(re.findall(patterns['compound_pattern'], text_content))
#             # Filter common false positives
#             compounds = {c for c in compounds if len(c) > 1 and any(char.isdigit() for char in c)}
        
#         # Extract reactions
#         if 'reaction_pattern' in patterns:
#             for match in re.finditer(patterns['reaction_pattern'], text_content):
#                 reactants = match.group(1).strip()
#                 products = match.group(2).strip()
#                 reactions.append({
#                     'reactants': reactants,
#                     'products': products,
#                     'full_reaction': f"{reactants} → {products}"
#                 })
        
#         # Extract conditions
#         if 'condition_pattern' in patterns:
#             for match in re.finditer(patterns['condition_pattern'], text_content):
#                 conditions.append(match.group(0))
        
#         return {
#             'compounds': list(compounds),
#             'reactions': reactions,
#             'reaction_conditions': conditions
#         }
    
#     def _process_biology_content(self, extraction_result):
#         """
#         Process content specific to biology domains.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary with biology-specific entities
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         species = set()
#         genes = set()
#         pathways = set()
        
#         patterns = self.domain_patterns.get('biology', {})
        
#         # Extract species
#         if 'species_pattern' in patterns:
#             for match in re.finditer(patterns['species_pattern'], text_content):
#                 species_name = match.group(1)
#                 # Filter common false positives
#                 if len(species_name.split()) == 2 and all(len(word) > 1 for word in species_name.split()):
#                     species.add(species_name)
        
#         # Extract genes
#         if 'gene_pattern' in patterns:
#             potential_genes = re.findall(patterns['gene_pattern'], text_content)
#             # Filter common false positives
#             genes = {g for g in potential_genes if len(g) >= 2 and g.isupper()}
        
#         # Extract pathways
#         if 'pathway_pattern' in patterns:
#             for match in re.finditer(patterns['pathway_pattern'], text_content):
#                 pathways.add(match.group(0))
        
#         return {
#             'species': list(species),
#             'genes': list(genes),
#             'biological_pathways': list(pathways)
#         }
    
#     def _process_physics_content(self, extraction_result):
#         """
#         Process content specific to physics domains.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary with physics-specific entities
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         laws = set()
#         physical_quantities = []
#         equations = []
        
#         patterns = self.domain_patterns.get('physics', {})
        
#         # Extract physical laws
#         if 'law_pattern' in patterns:
#             for match in re.finditer(patterns['law_pattern'], text_content):
#                 laws.add(match.group(0))
        
#         # Extract physical quantities with units
#         if 'units_pattern' in patterns:
#             for match in re.finditer(patterns['units_pattern'], text_content):
#                 value = match.group(1)
#                 unit = match.group(2)
#                 exponent = match.group(3) if match.group(3) else "1"
                
#                 physical_quantities.append({
#                     'value': float(value),
#                     'unit': unit,
#                     'exponent': int(exponent),
#                     'display': f"{value} {unit}{'' if exponent == '1' else f'^{exponent}'}"
#                 })
        
#         # Extract equations
#         if 'equation_pattern' in patterns:
#             for match in re.finditer(patterns['equation_pattern'], text_content):
#                 equation_num = match.group(1) or match.group(2)
#                 equation_text = match.group(3)
                
#                 equations.append({
#                     'equation_number': equation_num,
#                     'equation_text': equation_text
#                 })
        
#         return {
#             'physical_laws': list(laws),
#             'physical_quantities': physical_quantities,
#             'physics_equations': equations
#         }
    
#     def extract_scientific_terminology(self, extraction_result):
#         """
#         Extract scientific terms and classify by field.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary of scientific terminology by category
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         # Define common scientific terminology patterns
#         scientific_terms = {
#             'general': [],
#             'methodology': [],
#             'results': [],
#             'concepts': []
#         }
        
#         # Lists of common scientific terms by category
#         methodology_terms = ['experiment', 'method', 'procedure', 'protocol', 'analysis', 
#                             'technique', 'assay', 'measurement', 'observation']
        
#         result_terms = ['result', 'finding', 'data', 'evidence', 'observation', 
#                        'outcome', 'conclusion', 'analysis', 'figure', 'table']
        
#         concept_terms = ['theory', 'model', 'hypothesis', 'principle', 'law', 
#                         'concept', 'framework', 'paradigm', 'mechanism']
        
#         # Simple pattern matching for scientific terminology
#         for term in methodology_terms:
#             if term in text_content.lower():
#                 # Find the sentence containing the term
#                 sentences = text_content.replace('!', '.').replace('?', '.').split('.')
#                 for sentence in sentences:
#                     if term in sentence.lower():
#                         scientific_terms['methodology'].append(sentence.strip())
        
#         for term in result_terms:
#             if term in text_content.lower():
#                 sentences = text_content.replace('!', '.').replace('?', '.').split('.')
#                 for sentence in sentences:
#                     if term in sentence.lower():
#                         scientific_terms['results'].append(sentence.strip())
        
#         for term in concept_terms:
#             if term in text_content.lower():
#                 sentences = text_content.replace('!', '.').replace('?', '.').split('.')
#                 for sentence in sentences:
#                     if term in sentence.lower():
#                         scientific_terms['concepts'].append(sentence.strip())
        
#         # Extract other scientific terms using patterns
#         import re
        
#         # Pattern for scientific terms (simplified)
#         scientific_term_pattern = re.compile(r'(?i)([A-Z][a-z]+(?:ology|ography|ometry|onomy|physics|chemistry|ics))')
#         matches = scientific_term_pattern.findall(text_content)
        
#         scientific_terms['general'] = list(set(matches))
        
#         return scientific_terms
    
#     def identify_experiments_and_methods(self, extraction_result):
#         """
#         Identify experimental procedures and methods in the content.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary of identified experiments and methods
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         experiments_and_methods = {
#             'experiments': [],
#             'methods': [],
#             'procedures': []
#         }
        
#         patterns = self.domain_patterns.get('science', {})
        
#         # Extract methods
#         if 'method_pattern' in patterns:
#             for match in re.finditer(patterns['method_pattern'], text_content):
#                 method_text = match.group(2).strip()
#                 experiments_and_methods['methods'].append(method_text)
        
#         # Extract experiments
#         if 'experiment_pattern' in patterns:
#             for match in re.finditer(patterns['experiment_pattern'], text_content):
#                 experiment_text = match.group(2).strip()
#                 experiments_and_methods['experiments'].append(experiment_text)
        
#         # Look for procedure keywords
#         procedure_keywords = ['procedure', 'protocol', 'step', 'process']
#         sentences = text_content.replace('!', '.').replace('?', '.').split('.')
        
#         for sentence in sentences:
#             if any(keyword in sentence.lower() for keyword in procedure_keywords):
#                 experiments_and_methods['procedures'].append(sentence.strip())
        
#         return experiments_and_methods
    
#     def extract_mathematical_entities(self, extraction_result):
#         """
#         Extract theorems, formulas, proofs, and other mathematical entities.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary of mathematical entities by type
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         math_entities = {
#             'theorems': [],
#             'proofs': [],
#             'equations': [],
#             'definitions': []
#         }
        
#         patterns = self.domain_patterns.get('math', {})
        
#         # Extract theorems
#         if 'theorem_pattern' in patterns:
#             for match in re.finditer(patterns['theorem_pattern'], text_content):
#                 entity_type = match.group(1).lower()  # theorem, lemma, etc.
#                 entity_name = match.group(2).strip()
#                 entity_content = match.group(3).strip()
                
#                 math_entities['theorems'].append({
#                     'type': entity_type,
#                     'name': entity_name,
#                     'content': entity_content
#                 })
        
#         # Extract proofs
#         if 'proof_pattern' in patterns:
#             for match in re.finditer(patterns['proof_pattern'], text_content):
#                 proof_content = match.group(1).strip()
#                 math_entities['proofs'].append(proof_content)
        
#         # Extract equations
#         if 'equation_pattern' in patterns:
#             for match in re.finditer(patterns['equation_pattern'], text_content):
#                 equation_num = match.group(1) or match.group(2)
#                 equation_content = match.group(3).strip()
                
#                 math_entities['equations'].append({
#                     'number': equation_num,
#                     'content': equation_content
#                 })
        
#         # Extract definitions
#         definition_pattern = re.compile(r'(?i)Definition\s*(\d+|[\w\s]+)?[:]\s*(.*?)(?=\n\n|\Z)')
#         for match in re.finditer(definition_pattern, text_content):
#             def_name = match.group(1).strip() if match.group(1) else ""
#             def_content = match.group(2).strip()
            
#             math_entities['definitions'].append({
#                 'name': def_name,
#                 'content': def_content
#             })
        
#         return math_entities
    
#     def classify_math_problems(self, extraction_result):
#         """
#         Classify mathematical problems by type and difficulty.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary of classified mathematical problems
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         math_problems = []
        
#         # Pattern to identify problems
#         problem_pattern = re.compile(r'(?i)(?:Problem|Exercise|Question)\s*(\d+|[\w\s]+)?[:]\s*(.*?)(?=\n\n|\Z|Solution)')
        
#         for match in re.finditer(problem_pattern, text_content):
#             problem_num = match.group(1).strip() if match.group(1) else ""
#             problem_text = match.group(2).strip()
            
#             # Try to find the solution if available
#             solution_pattern = re.compile(f'(?i)Solution(?:\s*to\s*(?:Problem|Exercise|Question)?\s*{re.escape(problem_num) if problem_num else ""})?\s*[:]\s*(.*?)(?=\n\n|\Z|Problem|Exercise|Question)')
#             solution_match = solution_pattern.search(text_content)
            
#             solution_text = solution_match.group(1).strip() if solution_match else ""
            
#             # Classify problem type
#             problem_type = self._classify_math_problem_type(problem_text)
            
#             # Estimate difficulty
#             difficulty = self._estimate_math_problem_difficulty(problem_text, solution_text)
            
#             math_problems.append({
#                 'number': problem_num,
#                 'text': problem_text,
#                 'solution': solution_text,
#                 'type': problem_type,
#                 'difficulty': difficulty
#             })
        
#         return math_problems
    
#     def _classify_math_problem_type(self, problem_text):
#         """
#         Classify the type of a mathematical problem.
        
#         Args:
#             problem_text (str): The text of the problem
            
#         Returns:
#             The type of the mathematical problem
#         """
#         problem_text = problem_text.lower()
        
#         # Define keywords for different problem types
#         type_keywords = {
#             'algebra': ['solve', 'equation', 'polynomial', 'factor', 'simplify', 'expression'],
#             'calculus': ['derivative', 'integral', 'differentiate', 'integrate', 'limit', 'maximize', 'minimize'],
#             'geometry': ['triangle', 'circle', 'angle', 'polygon', 'rectangle', 'square', 'perimeter', 'area', 'volume'],
#             'probability': ['probability', 'random', 'chance', 'likelihood', 'expected value', 'variance'],
#             'statistics': ['mean', 'median', 'mode', 'standard deviation', 'normal distribution', 'variance'],
#             'number theory': ['prime', 'divisor', 'factor', 'gcd', 'lcm', 'modulo', 'congruence'],
#             'linear algebra': ['matrix', 'vector', 'eigenvalue', 'eigenvector', 'linear transformation'],
#             'discrete math': ['graph', 'tree', 'network', 'combinatorial', 'permutation', 'combination']
#         }
        
#         # Count occurrences of each category's keywords
#         type_scores = {category: 0 for category in type_keywords}
        
#         for category, keywords in type_keywords.items():
#             for keyword in keywords:
#                 if keyword in problem_text:
#                     type_scores[category] += 1
        
#         # Return the category with the highest score
#         if any(type_scores.values()):
#             return max(type_scores.items(), key=lambda x: x[1])[0]
#         else:
#             return 'general'
    
#     def _estimate_math_problem_difficulty(self, problem_text, solution_text=""):
#         """
#         Estimate the difficulty level of a mathematical problem.
        
#         Args:
#             problem_text (str): The text of the problem
#             solution_text (str): The text of the solution
            
#         Returns:
#             The estimated difficulty level (elementary, intermediate, advanced)
#         """
#         # Combine problem and solution for analysis
#         combined_text = problem_text + " " + solution_text
#         combined_text = combined_text.lower()
        
#         # Define difficulty indicators
#         elementary_indicators = ['basic', 'simple', 'elementary', 'straightforward', 'easy', 'find']
#         advanced_indicators = ['complex', 'challenging', 'advanced', 'difficult', 'prove', 'proof', 'theorem', 'lemma']
        
#         # Count advanced mathematical terms
#         advanced_math_terms = ['eigenvalue', 'eigenvector', 'homomorphism', 'isomorphism', 'bijection',
#                               'riemann', 'laplace', 'fourier', 'lebesgue', 'optimization', 'stochastic',
#                               'differential equation', 'partial derivative', 'tensor', 'manifold', 'topology']
        
#         # Count occurrences
#         elementary_count = sum(indicator in combined_text for indicator in elementary_indicators)
#         advanced_count = sum(indicator in combined_text for indicator in advanced_indicators)
#         advanced_term_count = sum(term in combined_text for term in advanced_math_terms)
        
#         # Text complexity heuristics
#         word_count = len(combined_text.split())
#         sentence_count = max(1, len(combined_text.split('.')))
#         average_sentence_length = word_count / sentence_count
        
#         # Decision logic for difficulty level
#         if advanced_term_count >= 2 or advanced_count >= 3 or average_sentence_length > 25:
#             return 'advanced'
#         elif elementary_count > advanced_count or (word_count < 50 and advanced_term_count == 0):
#             return 'elementary'
#         else:
#             return 'intermediate'
    
#     def analyze_literary_structure(self, extraction_result):
#         """
#         Identify plot elements, character dialogue, themes in literary texts.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary of literary structure elements
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         literary_structure = {
#             'characters': [],
#             'dialogue': [],
#             'plot_elements': [],
#             'settings': []
#         }
        
#         patterns = self.domain_patterns.get('literature', {})
        
#         # Extract character dialogue
#         if 'character_pattern' in patterns:
#             for match in re.finditer(patterns['character_pattern'], text_content):
#                 character_name = match.group(1).strip()
#                 dialogue = match.group(2).strip()
                
#                 if character_name not in literary_structure['characters']:
#                     literary_structure['characters'].append(character_name)
                
#                 literary_structure['dialogue'].append({
#                     'character': character_name,
#                     'text': dialogue
#                 })
        
#         # Extract plot elements
#         if 'plot_pattern' in patterns:
#             for match in re.finditer(patterns['plot_pattern'], text_content):
#                 plot_element = match.group(1).strip().lower()
#                 plot_content = match.group(2).strip()
                
#                 literary_structure['plot_elements'].append({
#                     'type': plot_element,
#                     'content': plot_content
#                 })
        
#         # Extract settings
#         setting_pattern = re.compile(r'(?i)(?:in|at)\s+(the\s+)?([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)')
#         settings = set()
        
#         for match in re.finditer(setting_pattern, text_content):
#             setting = match.group(2).strip()
#             # Filter out common false positives
#             if len(setting.split()) <= 3 and setting not in ['I', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']:
#                 settings.add(setting)
        
#         literary_structure['settings'] = list(settings)
        
#         # If Shakespeare was specified, check for act/scene structure
#         if 'shakespeare' in extraction_result.get('metadata', {}).get('title', '').lower() or 'shakespeare' in text_content.lower():
#             act_scene_pattern = re.compile(r'(?i)Act\s+([IVX]+|[0-9]+)(?:,\s+Scene\s+([IVX]+|[0-9]+))?')
#             act_scenes = []
            
#             for match in re.finditer(act_scene_pattern, text_content):
#                 act = match.group(1)
#                 scene = match.group(2) if match.group(2) else '1'
                
#                 act_scenes.append({
#                     'act': act,
#                     'scene': scene
#                 })
            
#             if act_scenes:
#                 literary_structure['act_scene_structure'] = act_scenes
        
#         return literary_structure
    
#     def extract_literary_devices(self, extraction_result):
#         """
#         Identify metaphors, similes, symbolism, and other literary devices.
        
#         Args:
#             extraction_result (dict): The extraction result
            
#         Returns:
#             Dictionary of literary devices
#         """
#         text_content = extraction_result.get('text_content', '')
        
#         literary_devices = {
#             'similes': [],
#             'metaphors': [],
#             'symbolism': [],
#             'alliteration': [],
#             'personification': []
#         }
        
#         # Extract similes (using "like" or "as")
#         simile_pattern = re.compile(r'(?i)([\w\s]+)\s+(like|as)\s+([\w\s]+)')
#         for match in re.finditer(simile_pattern, text_content):
#             before_text = match.group(1).strip()
#             connector = match.group(2)
#             after_text = match.group(3).strip()
            
#             # Avoid false positives by checking length
#             if len(before_text.split()) <= 5 and len(after_text.split()) <= 5:
#                 literary_devices['similes'].append(f"{before_text} {connector} {after_text}")
        
#         # Extract metaphors (more challenging, using some common patterns)
#         metaphor_pattern = re.compile(r'(?i)([\w\s]+)\s+(?:is|was|are|were)\s+([\w\s]+)')
#         for match in re.finditer(metaphor_pattern, text_content):
#             subject = match.group(1).strip()
#             predicate = match.group(2).strip()
            
#             # Avoid common literal statements
#             common_predicates = ['the', 'a', 'an', 'that', 'this', 'these', 'those', 'here', 'there']
#             if not any(predicate.startswith(word) for word in common_predicates) and len(subject.split()) <= 3 and len(predicate.split()) <= 3:
#                 metaphor_candidate = f"{subject} is {predicate}"
#                 literary_devices['metaphors'].append(metaphor_candidate)
        
#         # Extract symbolism
#         if 'theme_pattern' in self.domain_patterns.get('literature', {}):
#             theme_pattern = self.domain_patterns['literature']['theme_pattern']
#             for match in re.finditer(theme_pattern, text_content):
#                 symbolism_type = match.group(1).strip().lower()
#                 symbolism_content = match.group(2).strip()
                
#                 if symbolism_type == 'symbol':
#                     literary_devices['symbolism'].append(symbolism_content)
        
#         # Extract alliteration
#         sentences = text_content.replace('!', '.').replace('?', '.').split('.')
#         for sentence in sentences:
#             words = [word.strip().lower() for word in sentence.split() if word.strip()]
#             if len(words) >= 3:
#                 for i in range(len(words) - 2):
#                     if words[i] and words[i+1] and words[i+2]:
#                         if words[i][0] == words[i+1][0] == words[i+2][0]:
#                             alliteration = ' '.join([words[i], words[i+1], words[i+2]])
#                             literary_devices['alliteration'].append(alliteration)
        
#         # Extract personification
#         personification_subjects = ['sun', 'moon', 'stars', 'wind', 'rain', 'sea', 'ocean', 'mountain', 'tree',
#                                    'forest', 'river', 'sky', 'earth', 'death', 'time', 'nature', 'city', 'country']
#         personification_verbs = ['spoke', 'whispered', 'called', 'sighed', 'laughed', 'cried', 'sang', 'danced',
#                                 'watched', 'waited', 'loved', 'hated', 'feared', 'hoped', 'dreamed']
        
#         for subject in personification_subjects:
#             for verb in personification_verbs:
#                 pattern = re.compile(r'\b' + re.escape(subject) + r'\s+(?:\w+\s+){0,2}' + re.escape(verb) + r'\b', re.IGNORECASE)
#                 for match in re.finditer(pattern, text_content):
#                     literary_devices['personification'].append(match.group(0))
        
#         return literary_devices
    

In [32]:
import json
from collections.abc import Mapping

def print_key_structure(d: dict) -> None:
    """
    Given a dictionary `d`, prints its keys and any nested subkeys
    as a JSON-formatted structure where leaf values are replaced by {}.
    """
    def extract_keys(obj):
        if isinstance(obj, Mapping):
            return {k: extract_keys(v) for k, v in obj.items()}
        else:
            # non‐dict value → leaf key
            return type(obj).__name__
    
    key_tree = extract_keys(d)
    print(json.dumps(key_tree, indent=4))

In [ ]:
class ContentAnalysisPipeline:
    def __init__(self, domain:str=None,lm_client : OpenRouterMistralClient =None):
        self.pdf_extractor = EnhancedPDFExtractor()
        self.content_analyzer = ContentAnalyzer(domain=domain,extractor=self.pdf_extractor,language_model=lm_client)
        self.domain_processor = DomainSpecificProcessor(domain)
        self.relationship_analyzer = ContentRelationshipAnalyzer(content_analyzer=self.content_analyzer)
        self.visual_extractor = VisualizationElementExtractor()
        self.enrichment_processor = ContentEnrichmentProcessor()
        self.domain = domain
        
    def process_document(self, pdf_path, output_dir=None, extract_images=True, extract_tables=True, respect_heading_hierarchy=True):
        # Extract raw content
        extraction_result = self.pdf_extractor.extract_content_with_cleaning(
            pdf_path, 
            output_dir,
            extract_images=extract_images,
            extract_tables=extract_tables,
            respect_heading_hierarchy=respect_heading_hierarchy
        )
        
        # Analyze content
        analyzed_content = self.content_analyzer.analyze_content(extraction_result)
        
        # Process domain-specific elements
        domain_processed = self.domain_processor.process_domain_content(analyzed_content)
        
        # Analyze relationships
        relationship_data = self.relationship_analyzer.build_concept_graph(domain_processed)
        
        # Extract visualization elements
        visual_elements = self.visual_extractor.extract_diagrams(domain_processed)
        
        # Enrich content
        enriched_content = self.enrichment_processor.enrich_content(domain_processed)
        
        # Combine all results
        final_result = self.combine_results(extraction_result, analyzed_content, 
                                           domain_processed, relationship_data, 
                                           visual_elements, enriched_content)
                                           
        return final_result
    
    def combine_results(self, extraction_result, analyzed_content, domain_processed,
                   relationship_data, visual_elements, enriched_content):
        """Combine results from all processing steps into a unified output"""

        # Create a comprehensive result object that integrates all analysis components
        combined_result = {
            'document_metadata': extraction_result.get('metadata', {}),
            'raw_extraction': {
                'text': extraction_result.get('text_content', ''),
                'tables': extraction_result.get('tables', []),
                'images': extraction_result.get('images', [])
            },
            'semantic_analysis': {
                'content_blocks': analyzed_content.get('content_blocks', []),
                'content_segments': analyzed_content.get('content_segments', []),
                'keywords_and_concepts': analyzed_content.get('keywords_and_concepts', []),
                'section_summaries': analyzed_content.get('section_summaries', {}),
                'sentiment_analysis': analyzed_content.get('sentiment_analysis', {})
            },
            'domain_specific': self._extract_domain_specific_data(domain_processed),
            'relationships': {
                'concept_graph': relationship_data.get('graph', {}),
                'prerequisites': relationship_data.get('prerequisites', {}),
                'dependency_tree': relationship_data.get('dependency_tree', {})
            },
            'visual_elements': {
                'diagrams': visual_elements.get('diagrams', []),
                'figures': visual_elements.get('figures', []),
                'visual_text_relationships': visual_elements.get('relationships', {})
            },
            'enrichments': {
                'explanations': enriched_content.get('explanations', {}),
                'cross_references': enriched_content.get('cross_references', {}),
                'learning_resources': enriched_content.get('learning_resources', {})
            }
        }

        # Add literary analysis if available
        if 'literary_analysis' in analyzed_content:
            combined_result['semantic_analysis']['literary_analysis'] = analyzed_content.get('literary_analysis', {})

        return combined_result
    
    def _extract_domain_specific_data(self, domain_processed):
        """Extract domain-specific data based on the configured domain"""
        result = {}
        
        # Check if domain_processed contains the new domain_analysis structure
        if 'domain_analysis' in domain_processed:
            domain_entities = domain_processed['domain_analysis'].get('domain_entities', {})
            
            if self.domain == 'science' or self.domain == 'biology':
                result.update({
                    'scientific_terminology': domain_entities.get('terminology', []),
                    'experiments_and_methods': domain_entities.get('methods', {})
                })
            elif self.domain == 'math':
                result.update({
                    'mathematical_entities': domain_entities.get('mathematical_entities', []),
                    'problems_by_type': domain_entities.get('problem_classifications', {})
                })
            elif self.domain == 'literature' or self.domain == 'shakespeare':
                result.update({
                    'literary_structure': domain_entities.get('literary_structure', {}),
                    'literary_devices': domain_entities.get('literary_devices', [])
                })
            elif self.domain == 'chemistry':
                result.update({
                    'chemical_compounds': domain_entities.get('compounds', []),
                    'reactions': domain_entities.get('reactions', []),
                    'laboratory_procedures': domain_entities.get('reaction_conditions', [])
                })
            elif self.domain == 'physics':
                result.update({
                    'physical_laws': domain_entities.get('physical_laws', []),
                    'physical_quantities': domain_entities.get('physical_quantities', []),
                    'physics_equations': domain_entities.get('physics_equations', [])
                })
            else:
                # Generic domain processing
                result = domain_entities
        else:
            # For backward compatibility with old format
            result = domain_processed
        
        return result
    
    def save_results(self, result, output_path, formats=None):
        """Save the combined results in various formats"""
        if formats is None:
            formats = ['json', 'markdown', 'text']
            
        import os
        import json
        print_key_structure(result)
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        base_path = os.path.splitext(output_path)[0]
        
        if 'json' in formats:
            # Save as JSON
            with open(f"{base_path}.json", 'w', encoding='utf-8') as f:
                json.dump(result, f, indent=2, ensure_ascii=False)
                
        if 'markdown' in formats:
            # Generate markdown representation
            markdown_content = self._generate_markdown(result)
            with open(f"{base_path}.md", 'w', encoding='utf-8') as f:
                f.write(markdown_content)
                
        if 'text' in formats:
            # Generate plain text representation
            text_content = self._generate_text(result)
            with open(f"{base_path}.txt", 'w', encoding='utf-8') as f:
                f.write(text_content)
                
        return [f"{base_path}.{fmt.replace('text', 'txt')}" for fmt in formats]
    
    def _generate_text(self, result):
        """Generate a plain text representation focusing on content with improved schema handling"""
        text_lines = []
    
        # Add document title
        if isinstance(result.get('document_metadata', {}), dict) and result['document_metadata'].get('title'):
            text_lines.append(f"DOCUMENT: {result['document_metadata']['title']}")
        else:
            text_lines.append("DOCUMENT")
        text_lines.append("=" * 50)
        text_lines.append("")
    
        # Add comprehensive metadata
        if 'document_metadata' in result and isinstance(result['document_metadata'], dict):
            metadata = result['document_metadata']
            metadata_items = {
                'Author': metadata.get('author'),
                'Subject': metadata.get('subject'),
                'Creator': metadata.get('creator'),
                'Producer': metadata.get('producer'),
                'Creation Date': metadata.get('creation_date'),
                'Modification Date': metadata.get('modification_date'),
                'Page Count': metadata.get('page_count')
            }
    
            for label, value in metadata_items.items():
                if value:  # Only add if the value exists
                    text_lines.append(f"{label}: {value}")
    
            if any(metadata_items.values()):
                text_lines.append("")
    
        # Process domain-specific literary structure
        if 'domain_specific' in result and isinstance(result['domain_specific'], dict) and 'literary_structure' in result['domain_specific']:
            literary_structure = result['domain_specific']['literary_structure']
            text_lines.append("CONTENT")
            text_lines.append("-" * 50)
    
            # Handle different structure formats
            if isinstance(literary_structure, list):
                for page in literary_structure:
                    if isinstance(page, dict) and 'elements' in page:
                        # Extract content from elements on the page
                        for element in page['elements']:
                            if isinstance(element, dict):
                                if element.get('type') == 'heading':
                                    heading_text = element.get('text', '').strip()
                                    text_lines.append(heading_text.upper())
                                    text_lines.append("-" * len(heading_text))
                                elif element.get('type') == 'text':
                                    text_lines.append(element.get('text', '').strip())
                                    text_lines.append("")
            elif isinstance(literary_structure, dict):
                # Handle dictionary structure - common in narrative analysis
                for section_name, section_data in literary_structure.items():
                    text_lines.append(section_name.upper())
                    text_lines.append("-" * len(section_name))
    
                    if isinstance(section_data, dict):
                        for subsection, content in section_data.items():
                            text_lines.append(f"{subsection}:")
    
                            if isinstance(content, dict):
                                if content.get('setting'):
                                    text_lines.append(f"Setting: {content['setting']}")
                                if content.get('characters'):
                                    if isinstance(content['characters'], list):
                                        text_lines.append(f"Characters: {', '.join(content['characters'])}")
                                    else:
                                        text_lines.append(f"Characters: {content['characters']}")
                                if content.get('word_count'):
                                    text_lines.append(f"Word Count: {content['word_count']}")
                            else:
                                text_lines.append(str(content))
    
                            text_lines.append("")
                    else:
                        text_lines.append(str(section_data))
                        text_lines.append("")
    
        # Extract raw text if no structured content was found
        raw_text_condition = (
            'domain_specific' not in result or
            not isinstance(result.get('domain_specific'), dict) or
            'literary_structure' not in result['domain_specific'] or
            not result['domain_specific']['literary_structure']
        )
        
        if raw_text_condition and 'raw_extraction' in result and isinstance(result['raw_extraction'], dict):
            # Add raw text if available
            raw_text = result['raw_extraction'].get('text')
            if isinstance(raw_text, str) and raw_text.strip():
                text_lines.append("CONTENT")
                text_lines.append("-" * 50)
                text_lines.append(raw_text)
                text_lines.append("")
    
            # Add image information
            images = result['raw_extraction'].get('images')
            if images:
                text_lines.append("IMAGES")
                text_lines.append("-" * 50)
    
                if isinstance(images, list):
                    for i, img in enumerate(images):
                        if isinstance(img, dict):
                            text_lines.append(f"Image {i+1}:")
                            if img.get('file_name'):
                                text_lines.append(f"Filename: {img['file_name']}")
                            if img.get('page'):
                                text_lines.append(f"Page: {img['page']}")
                            if img.get('ocr_text'):
                                text_lines.append(f"OCR Text: {img['ocr_text']}")
                            text_lines.append("")
                elif isinstance(images, dict):
                    for image_id, img_data in images.items():
                        text_lines.append(f"Image: {image_id}")
                        if isinstance(img_data, dict):
                            for key, value in img_data.items():
                                if key != 'data' and key != 'binary':
                                    text_lines.append(f"{key}: {value}")
                        else:
                            text_lines.append(str(img_data))
                        text_lines.append("")
    
            # Add table information
            tables = result['raw_extraction'].get('tables')
            if tables:
                text_lines.append("TABLES")
                text_lines.append("-" * 50)
    
                if isinstance(tables, list):
                    for i, _ in enumerate(tables):
                        text_lines.append(f"Table {i+1}: Content available in original document")
                        text_lines.append("")
                elif isinstance(tables, dict):
                    for table_id, _ in tables.items():
                        text_lines.append(f"Table {table_id}: Content available in original document")
                        text_lines.append("")
    
        # Add keywords and concepts with improved structure handling
        if 'semantic_analysis' in result and isinstance(result['semantic_analysis'], dict):
            kw_concepts = result['semantic_analysis'].get('keywords_and_concepts')
            if kw_concepts:
                text_lines.append("KEYWORDS AND CONCEPTS")
                text_lines.append("-" * 50)
    
                # Handle string or dict for keywords
                keywords = kw_concepts.get('keywords') if isinstance(kw_concepts, dict) else None
                if keywords:
                    if isinstance(keywords, list):
                        text_lines.append(f"Keywords: {', '.join(keywords)}")
                    elif isinstance(keywords, str):
                        text_lines.append(f"Keywords: {keywords}")
    
                # Handle string or dict for bigrams
                bigrams = kw_concepts.get('bigrams') if isinstance(kw_concepts, dict) else None
                if bigrams:
                    if isinstance(bigrams, list):
                        text_lines.append(f"Bigrams: {', '.join(bigrams)}")
                    elif isinstance(bigrams, str):
                        text_lines.append(f"Bigrams: {bigrams}")
    
                # Handle string or dict for domain_concepts
                domain_concepts = kw_concepts.get('domain_concepts') if isinstance(kw_concepts, dict) else None
                if domain_concepts:
                    if isinstance(domain_concepts, list):
                        text_lines.append(f"Domain Concepts: {', '.join(domain_concepts)}")
                    elif isinstance(domain_concepts, str):
                        text_lines.append(f"Domain Concepts: {domain_concepts}")
    
                # Handle word frequencies
                word_frequencies = kw_concepts.get('word_frequencies') if isinstance(kw_concepts, dict) else None
                if word_frequencies:
                    text_lines.append("Word Frequencies:")
                    if isinstance(word_frequencies, dict):
                        for word, freq in word_frequencies.items():
                            text_lines.append(f"- {word}: {freq}")
                    elif isinstance(word_frequencies, str):
                        text_lines.append(word_frequencies)
                    elif isinstance(word_frequencies, list):
                        for item in word_frequencies:
                            if isinstance(item, tuple) and len(item) == 2:
                                text_lines.append(f"- {item[0]}: {item[1]}")
                            else:
                                text_lines.append(f"- {item}")
    
                # Handle raw_keywords
                raw_keywords = kw_concepts.get('raw_keywords') if isinstance(kw_concepts, dict) else None
                if raw_keywords:
                    text_lines.append("Top Keywords:")
                    if isinstance(raw_keywords, list):
                        for item in raw_keywords:
                            if isinstance(item, tuple) and len(item) == 2:
                                text_lines.append(f"- {item[0]}: {item[1]}")
                            else:
                                text_lines.append(f"- {item}")
                    elif isinstance(raw_keywords, str):
                        text_lines.append(raw_keywords)
    
                text_lines.append("")
    
            # Add section summaries
            section_summaries = result['semantic_analysis'].get('section_summaries')
            if section_summaries:
                text_lines.append("SECTION SUMMARIES")
                text_lines.append("-" * 50)
    
                if isinstance(section_summaries, dict):
                    for section, summary in section_summaries.items():
                        text_lines.append(f"{section}:")
                        text_lines.append(str(summary))
                        text_lines.append("")
                elif isinstance(section_summaries, str):
                    text_lines.append(section_summaries)
                    text_lines.append("")
    
            # Add sentiment analysis if available
            sentiment = result['semantic_analysis'].get('sentiment_analysis')
            if sentiment:
                text_lines.append("SENTIMENT ANALYSIS")
                text_lines.append("-" * 50)
    
                if isinstance(sentiment, dict):
                    if sentiment.get('overall_sentiment'):
                        text_lines.append(f"Overall Sentiment: {sentiment['overall_sentiment']}")
    
                    if 'positive_count' in sentiment and 'negative_count' in sentiment:
                        text_lines.append(f"Sentiment Count: {sentiment['positive_count']} positive, {sentiment['negative_count']} negative")
    
                    if 'is_academic_tone' in sentiment:
                        academic = "Yes" if sentiment['is_academic_tone'] else "No"
                        text_lines.append(f"Academic Tone: {academic}")
                elif isinstance(sentiment, str):
                    text_lines.append(sentiment)
    
                text_lines.append("")
    
            # Add character information if available
            literary_analysis = result['semantic_analysis'].get('literary_analysis')
            if isinstance(literary_analysis, dict) and literary_analysis.get('characters'):
                characters = literary_analysis['characters']
                if characters:
                    text_lines.append("CHARACTERS")
                    text_lines.append("-" * 50)
    
                    if isinstance(characters, dict):
                        for character, data in characters.items():
                            text_lines.append(character)
    
                            if isinstance(data, dict):
                                if data.get('line_count'):
                                    text_lines.append(f"Lines: {data['line_count']}")
                                if data.get('word_count'):
                                    text_lines.append(f"Words: {data['word_count']}")
                                if data.get('avg_speech_length'):
                                    text_lines.append(f"Average Speech Length: {data['avg_speech_length']} words")
                                if data.get('sample_speech'):
                                    text_lines.append(f"Sample Speech: {data['sample_speech']}")
                            else:
                                text_lines.append(str(data))
    
                            text_lines.append("")
                    elif isinstance(characters, str):
                        text_lines.append(characters)
                        text_lines.append("")
    
        # Add literary devices if available
        if 'domain_specific' in result and isinstance(result['domain_specific'], dict) and 'literary_devices' in result['domain_specific']:
            devices = result['domain_specific']['literary_devices']
            if devices:
                text_lines.append("LITERARY DEVICES")
                text_lines.append("-" * 50)
    
                if isinstance(devices, dict):
                    for device, instances in devices.items():
                        if instances:
                            text_lines.append(f"{device.upper()}:")
    
                            if isinstance(instances, list):
                                for instance in instances:
                                    text_lines.append(f"- {instance}")
                            elif isinstance(instances, str):
                                text_lines.append(instances)
    
                            text_lines.append("")
                elif isinstance(devices, str):
                    text_lines.append(devices)
                    text_lines.append("")
    
        # Add relationships if available
        if 'relationships' in result and isinstance(result['relationships'], dict) and any(result['relationships'].values()):
            relationships = result['relationships']
            text_lines.append("RELATIONSHIPS")
            text_lines.append("-" * 50)
    
            prerequisites = relationships.get('prerequisites')
            if prerequisites:
                text_lines.append("Prerequisites:")
    
                if isinstance(prerequisites, dict):
                    for concept, prereqs in prerequisites.items():
                        if isinstance(prereqs, list):
                            text_lines.append(f"{concept}: {', '.join(prereqs)}")
                        else:
                            text_lines.append(f"{concept}: {prereqs}")
                elif isinstance(prerequisites, list):
                    for item in prerequisites:
                        text_lines.append(f"- {item}")
                elif isinstance(prerequisites, str):
                    text_lines.append(prerequisites)
    
                text_lines.append("")
    
            if relationships.get('dependency_tree'):
                text_lines.append("Dependency Tree: Available in original analysis")
                text_lines.append("")
    
        return "\n".join(text_lines)

    def _generate_markdown(self, result):
        """Generate a markdown representation of the document content with improved handling of the schema structure"""
        # First, get the text representation to help with data extraction
        text_content = self._generate_text(result)

        md_lines = []

        # Add document title if available, otherwise use default
        if isinstance(result.get('document_metadata', {}), dict) and result['document_metadata'].get('title'):
            md_lines.append(f"# {result['document_metadata']['title']}")
        else:
            md_lines.append("# Document Content")

        md_lines.append("")

        # Add comprehensive metadata section
        if 'document_metadata' in result and isinstance(result['document_metadata'], dict):
            metadata = result['document_metadata']
            metadata_items = {
                'Author': metadata.get('author'),
                'Subject': metadata.get('subject'),
                'Creator': metadata.get('creator'),
                'Producer': metadata.get('producer'),
                'Creation Date': metadata.get('creation_date'),
                'Modification Date': metadata.get('modification_date'),
                'Page Count': metadata.get('page_count')
            }

            for label, value in metadata_items.items():
                if value:  # Only add if the value exists
                    md_lines.append(f"**{label}:** {value}")

            if any(metadata_items.values()):
                md_lines.append("")

        # Process the literary structure which contains the actual content
        if 'domain_specific' in result and isinstance(result['domain_specific'], dict) and 'literary_structure' in result['domain_specific']:
            literary_structure = result['domain_specific']['literary_structure']

            # Handle different structure formats
            if isinstance(literary_structure, list):
                for page in literary_structure:
                    if isinstance(page, dict) and 'elements' in page:
                        # Process each element in the page as in original code
                        for element in page['elements']:
                            if not isinstance(element, dict):
                                continue

                            element_type = element.get('type')

                            if element_type == 'heading':
                                # Add headings with appropriate markdown formatting
                                heading_text = element.get('text', '')
                                font_size = element.get('font_size', 12)

                                # Determine heading level based on font size
                                if font_size > 30:
                                    heading_level = '##'
                                elif font_size > 20:
                                    heading_level = '###'
                                elif font_size > 16:
                                    heading_level = '####'
                                else:
                                    heading_level = '#####'

                                md_lines.append(f"{heading_level} {heading_text}")
                                md_lines.append("")

                            elif element_type == 'text':
                                # Add regular text content
                                text = element.get('text', '')
                                md_lines.append(text)
                                md_lines.append("")

                            elif element_type == 'figure':
                                # Add figure references with proper markdown image syntax
                                caption = element.get('caption', 'Figure')
                                md_lines.append(f"![{caption}](figure_placeholder)")
                                md_lines.append(f"*{caption}*")
                                md_lines.append("")
            elif isinstance(literary_structure, dict):
                # Handle dictionary structure - common in narrative analysis
                for section_name, section_data in literary_structure.items():
                    md_lines.append(f"## {section_name}")
                    md_lines.append("")

                    if isinstance(section_data, dict):
                        for subsection, content in section_data.items():
                            md_lines.append(f"### {subsection}")

                            if isinstance(content, dict):
                                if content.get('setting'):
                                    md_lines.append(f"**Setting:** {content['setting']}")
                                if content.get('characters'):
                                    if isinstance(content['characters'], list):
                                        md_lines.append(f"**Characters:** {', '.join(content['characters'])}")
                                    else:
                                        md_lines.append(f"**Characters:** {content['characters']}")
                                if content.get('word_count'):
                                    md_lines.append(f"**Word Count:** {content['word_count']}")
                            else:
                                md_lines.append(str(content))

                            md_lines.append("")
                    else:
                        md_lines.append(str(section_data))
                        md_lines.append("")

        # Extract raw text if no structured content was found and raw text is available
        raw_text_condition = (
            'domain_specific' not in result or
            not isinstance(result.get('domain_specific'), dict) or
            'literary_structure' not in result['domain_specific'] or
            not result['domain_specific']['literary_structure']
        )

        if raw_text_condition and 'raw_extraction' in result and isinstance(result['raw_extraction'], dict):
            # Add raw text if available
            raw_text = result['raw_extraction'].get('text')
            if isinstance(raw_text, str) and raw_text.strip():
                md_lines.append("## Document Text")
                md_lines.append(raw_text)
                md_lines.append("")

        # Process images with better handling of the images structure
        if 'raw_extraction' in result and isinstance(result['raw_extraction'], dict) and 'images' in result['raw_extraction']:
            images = result['raw_extraction']['images']
            if images:
                md_lines.append("## Images")

                # Handle both list and dictionary formats for images
                if isinstance(images, list):
                    for i, img in enumerate(images):
                        if not isinstance(img, dict):
                            continue

                        image_name = img.get('file_name', f'image_{i+1}')
                        md_lines.append(f"![{image_name}](images/{image_name})")

                        # Add metadata as available
                        metadata_items = []
                        if img.get('page'):
                            metadata_items.append(f"Page {img['page']}")
                        if img.get('format'):
                            metadata_items.append(f"Format: {img['format']}")

                        if metadata_items:
                            md_lines.append(f"**Image {i+1}:** {', '.join(metadata_items)}")

                        # Add OCR text if available
                        if img.get('ocr_text'):
                            md_lines.append(f"*OCR Text:* {img['ocr_text']}")

                        md_lines.append("")
                elif isinstance(images, dict):
                    for image_id, img_data in images.items():
                        md_lines.append(f"![{image_id}](images/{image_id})")
                        md_lines.append(f"**{image_id}**")

                        if isinstance(img_data, dict):
                            for key, value in img_data.items():
                                if key != 'data' and key != 'binary':  # Skip binary data
                                    md_lines.append(f"*{key}:* {value}")
                        else:
                            md_lines.append(str(img_data))

                        md_lines.append("")

        # Add tables section if available
        if 'raw_extraction' in result and isinstance(result['raw_extraction'], dict) and 'tables' in result['raw_extraction']:
            tables = result['raw_extraction']['tables']
            if tables:
                md_lines.append("## Tables")

                if isinstance(tables, list):
                    for i, table in enumerate(tables):
                        md_lines.append(f"### Table {i+1}")

                        # For markdown we'd normally render the table here
                        # This is a simplified representation
                        md_lines.append("*Table content available in the original document*")
                        md_lines.append("")
                elif isinstance(tables, dict):
                    for table_id, table_data in tables.items():
                        md_lines.append(f"### {table_id}")
                        md_lines.append("*Table content available in the original document*")
                        md_lines.append("")

        # Add keywords and concepts with improved structure handling
        if 'semantic_analysis' in result and isinstance(result['semantic_analysis'], dict):
            kw_concepts = result['semantic_analysis'].get('keywords_and_concepts')
            if kw_concepts:
                md_lines.append("## Keywords and Concepts")

                # Handle string or dict for keywords
                keywords = kw_concepts.get('keywords') if isinstance(kw_concepts, dict) else None
                if keywords:
                    if isinstance(keywords, list):
                        md_lines.append(f"**Keywords:** {', '.join(keywords)}")
                    elif isinstance(keywords, str):
                        md_lines.append(f"**Keywords:** {keywords}")

                # Handle string or dict for bigrams
                bigrams = kw_concepts.get('bigrams') if isinstance(kw_concepts, dict) else None
                if bigrams:
                    if isinstance(bigrams, list):
                        md_lines.append(f"**Bigrams:** {', '.join(bigrams)}")
                    elif isinstance(bigrams, str):
                        md_lines.append(f"**Bigrams:** {bigrams}")

                # Handle string or dict for domain_concepts
                domain_concepts = kw_concepts.get('domain_concepts') if isinstance(kw_concepts, dict) else None
                if domain_concepts:
                    if isinstance(domain_concepts, list):
                        md_lines.append(f"**Domain Concepts:** {', '.join(domain_concepts)}")
                    elif isinstance(domain_concepts, str):
                        md_lines.append(f"**Domain Concepts:** {domain_concepts}")

                # Handle word frequencies
                word_frequencies = kw_concepts.get('word_frequencies') if isinstance(kw_concepts, dict) else None
                if word_frequencies:
                    md_lines.append("**Word Frequencies:**")
                    if isinstance(word_frequencies, dict):
                        for word, freq in word_frequencies.items():
                            md_lines.append(f"- {word}: {freq}")
                    elif isinstance(word_frequencies, str):
                        md_lines.append(word_frequencies)
                    elif isinstance(word_frequencies, list):
                        for item in word_frequencies:
                            if isinstance(item, tuple) and len(item) == 2:
                                md_lines.append(f"- {item[0]}: {item[1]}")
                            else:
                                md_lines.append(f"- {item}")

                # Handle raw_keywords
                raw_keywords = kw_concepts.get('raw_keywords') if isinstance(kw_concepts, dict) else None
                if raw_keywords:
                    md_lines.append("**Top Keywords:**")
                    if isinstance(raw_keywords, list):
                        for item in raw_keywords:
                            if isinstance(item, tuple) and len(item) == 2:
                                md_lines.append(f"- {item[0]}: {item[1]}")
                            else:
                                md_lines.append(f"- {item}")
                    elif isinstance(raw_keywords, str):
                        md_lines.append(raw_keywords)

                md_lines.append("")

        # Add section summaries with improved handling
        if 'semantic_analysis' in result and isinstance(result['semantic_analysis'], dict) and 'section_summaries' in result['semantic_analysis']:
            summaries = result['semantic_analysis']['section_summaries']
            if summaries:
                md_lines.append("## Section Summaries")

                if isinstance(summaries, dict):
                    for section, summary in summaries.items():
                        md_lines.append(f"### {section}")
                        md_lines.append(str(summary))
                        md_lines.append("")
                elif isinstance(summaries, str):
                    md_lines.append(summaries)
                    md_lines.append("")

        # Add sentiment analysis if available
        if 'semantic_analysis' in result and isinstance(result['semantic_analysis'], dict) and 'sentiment_analysis' in result['semantic_analysis']:
            sentiment = result['semantic_analysis']['sentiment_analysis']
            if sentiment:
                md_lines.append("## Sentiment Analysis")

                if isinstance(sentiment, dict):
                    if sentiment.get('overall_sentiment'):
                        md_lines.append(f"**Overall Sentiment:** {sentiment['overall_sentiment']}")

                    if 'positive_count' in sentiment and 'negative_count' in sentiment:
                        md_lines.append(f"**Sentiment Count:** {sentiment['positive_count']} positive, {sentiment['negative_count']} negative")

                    if 'is_academic_tone' in sentiment:
                        academic = "Yes" if sentiment['is_academic_tone'] else "No"
                        md_lines.append(f"**Academic Tone:** {academic}")
                elif isinstance(sentiment, str):
                    md_lines.append(sentiment)

                md_lines.append("")

        # Add literary analysis if available
        if ('semantic_analysis' in result and isinstance(result['semantic_analysis'], dict) and 
            'literary_analysis' in result['semantic_analysis'] and 
            isinstance(result['semantic_analysis']['literary_analysis'], dict)):

            lit_analysis = result['semantic_analysis']['literary_analysis']

            if 'characters' in lit_analysis:
                characters = lit_analysis['characters']
                if characters:
                    md_lines.append("## Characters")

                    if isinstance(characters, dict):
                        for character, data in characters.items():
                            md_lines.append(f"### {character}")

                            if isinstance(data, dict):
                                if data.get('line_count'):
                                    md_lines.append(f"**Lines:** {data['line_count']}")
                                if data.get('word_count'):
                                    md_lines.append(f"**Words:** {data['word_count']}")
                                if data.get('avg_speech_length'):
                                    md_lines.append(f"**Average Speech Length:** {data['avg_speech_length']} words")
                                if data.get('sample_speech'):
                                    md_lines.append(f"**Sample Speech:**\n> {data['sample_speech']}")
                            else:
                                md_lines.append(str(data))

                            md_lines.append("")
                    elif isinstance(characters, str):
                        md_lines.append(characters)
                        md_lines.append("")

            if 'literary_devices' in lit_analysis:
                devices = lit_analysis['literary_devices']
                if devices:
                    md_lines.append("## Literary Devices")

                    if isinstance(devices, dict):
                        for device, instances in devices.items():
                            if instances:
                                md_lines.append(f"### {device.title()}")

                                if isinstance(instances, list):
                                    for instance in instances:
                                        md_lines.append(f"- {instance}")
                                elif isinstance(instances, dict):
                                    for key, value in instances.items():
                                        md_lines.append(f"- **{key}:** {value}")
                                elif isinstance(instances, str):
                                    md_lines.append(instances)

                                md_lines.append("")
                    elif isinstance(devices, str):
                        md_lines.append(devices)
                        md_lines.append("")

        # Add literary devices from domain_specific if available
        elif ('domain_specific' in result and isinstance(result['domain_specific'], dict) and 
              'literary_devices' in result['domain_specific']):

            devices = result['domain_specific']['literary_devices']
            if devices:
                md_lines.append("## Literary Devices")

                if isinstance(devices, dict):
                    for device, instances in devices.items():
                        if instances:
                            md_lines.append(f"### {device.title()}")

                            if isinstance(instances, list):
                                for instance in instances:
                                    md_lines.append(f"- {instance}")
                            elif isinstance(instances, str):
                                md_lines.append(instances)

                            md_lines.append("")
                elif isinstance(devices, str):
                    md_lines.append(devices)
                    md_lines.append("")

        # Add relationships if available
        if 'relationships' in result and isinstance(result['relationships'], dict):
            relationships = result['relationships']
            if relationships:
                md_lines.append("## Relationships and Connections")

                if relationships.get('concept_graph'):
                    md_lines.append("### Concept Graph")
                    md_lines.append("*Concept graph available in the original analysis*")
                    md_lines.append("")

                prerequisites = relationships.get('prerequisites')
                if prerequisites:
                    md_lines.append("### Prerequisites")

                    if isinstance(prerequisites, dict):
                        for concept, prereqs in prerequisites.items():
                            if isinstance(prereqs, list):
                                md_lines.append(f"**{concept}:** {', '.join(prereqs)}")
                            else:
                                md_lines.append(f"**{concept}:** {prereqs}")
                    elif isinstance(prerequisites, list):
                        for item in prerequisites:
                            md_lines.append(f"- {item}")
                    elif isinstance(prerequisites, str):
                        md_lines.append(prerequisites)

                    md_lines.append("")

        return "\n".join(md_lines)

In [ ]:
raise SystemExit(0)

SystemExit: 0

c:\Users\adity\envs\4thyearproject\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
if __name__ == "__main__":
    extractor = EnhancedPDFExtractor(
        tesseract_path=r"C:\Program Files\Tesseract-OCR\tesseract.exe",  # Adjust for your system
        math_recognition=True,
        chem_recognition=True
    )
    
    pdf_path = "input.pdf"
    output_dir = "extracted_images"
    
    # Extract all content with enhanced cleaning
    result = extractor.extract_content_with_cleaning(
        pdf_path,
        output_dir=output_dir,
        extract_images=True,
        extract_tables=True
    )
    
    # Save the extraction result with both JSON and text output
    extractor.save_extraction_result(
        result,output_path_base="document_output",
        formats=['json', 'text', 'markdown']
    )
    
    print(f"Extracted {len(result['pages'])} pages")
    print(f"Found {len(result.get('images', []))} images")
    print(f"Found {len(result.get('tables', []))} tables")
    print(f"Found {len(result.get('math_expressions', []))} mathematical expressions")
    print(f"Found {len(result.get('chemical_content', []))} chemical formulas/reactions")   

KeyboardInterrupt: 

In [ ]:
# Example usage
if __name__ == "__main__":
    # Create analyzer with science domain knowledge
    analyzer = ComprehensivePDFAnalyzer(domain="science", math_recognition=True)
    
    # Process a scientific PDF document with full analysis
    result = analyzer.extract_analyze_and_save(
        pdf_path="example_scientific_paper.pdf",
        output_path_base="output/scientific_paper_analyzed",
        extract_images=True,
        extract_tables=True,
        perform_analysis=True,
        formats=["json", "markdown"]
    )
    
    print(f"Processing complete. Files saved to: {result['output_files']}")

In [ ]:
# Example usage
# if __name__ == "__main__":
#     extractor = PDFContentExtractor(
#         tesseract_path=r"C:\Program Files\Tesseract-OCR\tesseract.exe",  # Adjust for your system
#         math_recognition=True,
#         chem_recognition=True
#     )
    
#     pdf_path = "input.pdf"
#     output_dir = "extracted_images"
    
#     # Extract all content
#     result = extractor.extract_content(
#         pdf_path,
#         output_dir=output_dir,
#         extract_images=True,
#         extract_tables=True
#     )
    
#     # Save the extraction result
#     extractor.save_extraction_result(result, "extraction_result.json")
    
#     print(f"Extracted {len(result['pages'])} pages")
#     print(f"Found {len(result['images'])} images")
#     print(f"Found {len(result['tables'])} tables")
#     print(f"Found {len(result['math_expressions'])} mathematical expressions")
#     print(f"Found {len(result['chemical_content'])} chemical formulas/reactions")

In [ ]:
# Verify installations and dependencies
def verify_dependencies():
    import sys
    
    dependencies = {
        'PyMuPDF': 'fitz',
        'OpenCV': 'cv2',
        'Pillow': 'PIL',
        'Tesseract': 'pytesseract',
        'PDFMiner': 'pdfminer',
        'Camelot': 'camelot',
        'SymPy': 'sympy',
        'NumPy': 'numpy'
    }
    
    print('Checking dependencies...')
    print('-' * 50)
    
    all_good = True
    for name, module in dependencies.items():
        try:
            if module == 'PIL':
                import PIL
                version = PIL.__version__
            else:
                exec(f'import {module}')
                version = eval(f'{module}.__version__')
            print(f'✓ {name:<12} - v{version}')
        except ImportError:
            print(f'✗ {name:<12} - Not installed')
            all_good = False
        except Exception as e:
            print(f'? {name:<12} - Installed but version unknown')
    
    print('-' * 50)
    
    # Check Tesseract installation
    import pytesseract
    try:
        pytesseract.get_tesseract_version()
        print('✓ Tesseract OCR is properly configured')
    except Exception as e:
        print('✗ Tesseract OCR is not properly configured')
        print(f'  Error: {str(e)}')
        all_good = False
    
    return all_good

is_setup_complete = verify_dependencies()

In [ ]:
# class ContentAnalyzer:
#     def __init__(self, language_model=None, domain=None):
#         """
#         Initialize the ContentAnalyzer with optional language model and domain specification.
        
#         Args:
#             language_model: Optional pre-trained language model for advanced text analysis
#             domain: Optional domain specification to customize analysis for specific fields
#                    (e.g., science, literature, mathematics)
#         """
#         self.language_model = language_model
#         self.domain = domain
#         self.domain_specific_rules = self._load_domain_rules(domain)
    
#     def _load_domain_rules(self, domain):
#         """
#         Load domain-specific rules and patterns for content analysis.
        
#         Args:
#             domain: The domain to load rules for
            
#         Returns:
#             Dictionary of rules and patterns specific to the domain
#         """
#         if domain is None:
#             return {}
            
#         domain_rules = {
#             'science': {
#                 'terminologies': ['experiment', 'hypothesis', 'theory', 'data'],
#                 'section_types': ['methodology', 'results', 'discussion', 'conclusion'],
#                 'entity_patterns': [r'[A-Z][a-z]+ [a-z]+ium', r'\d+\.\d+ [a-z]+']
#             },
#             'math': {
#                 'terminologies': ['theorem', 'proof', 'lemma', 'corollary'],
#                 'section_types': ['definition', 'theorem', 'proof', 'example'],
#                 'entity_patterns': [r'[A-Z][a-z]+ (theorem|formula|equation)', r'equation \(\d+\)']
#             },
#             'literature': {
#                 'terminologies': ['symbolism', 'metaphor', 'character', 'plot'],
#                 'section_types': ['exposition', 'climax', 'resolution', 'analysis'],
#                 'entity_patterns': [r'[A-Z][a-z]+ (symbolizes|represents)', r'theme of [a-z]+']
#             }
#         }
        
#         return domain_rules.get(domain.lower(), {})
    
#     def analyze_content(self, extraction_result):
#         """
#         Main method to analyze extracted content from a PDF.
        
#         Args:
#             extraction_result: Dictionary containing the extracted content from PDFContentExtractor
            
#         Returns:
#             Dictionary containing the original content enriched with semantic annotations
#         """
#         analyzed_result = extraction_result.copy()
        
#         # Add semantic analysis to the content
#         if 'content_blocks' in analyzed_result:
#             analyzed_result['content_blocks'] = self.tag_content_blocks(analyzed_result['content_blocks'])
        
#         # Extract keywords and concepts
#         if 'text_content' in analyzed_result:
#             analyzed_result['keywords_and_concepts'] = self.extract_keywords_and_concepts(
#                 analyzed_result['text_content'], self.domain
#             )
        
#         # Generate summaries for sections
#         if 'sections' in analyzed_result:
#             analyzed_result['section_summaries'] = {
#                 section_id: self.generate_summaries(section_content)
#                 for section_id, section_content in analyzed_result['sections'].items()
#             }
        
#         # Segment content into logical units
#         analyzed_result['content_segments'] = self.segment_content(extraction_result)
        
#         # Add sentiment analysis if text is available
#         if 'text_content' in analyzed_result:
#             analyzed_result['sentiment_analysis'] = self.perform_sentiment_analysis(
#                 analyzed_result['text_content']
#             )
        
#         return analyzed_result
    
#     def segment_content(self, extraction_result):
#         """
#         Divide content into logical segments beyond just headings.
        
#         Args:
#             extraction_result: Dictionary containing the extracted content
            
#         Returns:
#             List of content segments with their type and content
#         """
#         segments = []
        
#         # If document is already structured by headings
#         if 'document_structure' in extraction_result:
#             structure = extraction_result['document_structure']
#             for section in structure:
#                 # Create a segment for the heading
#                 if 'heading' in section:
#                     segments.append({
#                         'type': 'heading',
#                         'level': section.get('level', 1),
#                         'content': section['heading']
#                     })
                
#                 # Process the content under this heading
#                 if 'content' in section:
#                     content_segments = self._segment_text_block(section['content'])
#                     segments.extend(content_segments)
        
#         # If no structure is available, segment the raw text
#         elif 'text_content' in extraction_result:
#             segments = self._segment_text_block(extraction_result['text_content'])
        
#         return segments
    
#     def _segment_text_block(self, text):
#         """
#         Break a block of text into logical segments.
        
#         Args:
#             text: Text content to segment
            
#         Returns:
#             List of segments with their identified type and content
#         """
#         # Simple implementation - split by paragraphs and try to determine type
#         segments = []
#         paragraphs = text.split('\n\n')
        
#         for paragraph in paragraphs:
#             if not paragraph.strip():
#                 continue
                
#             segment_type = self._identify_segment_type(paragraph)
#             segments.append({
#                 'type': segment_type,
#                 'content': paragraph.strip()
#             })
            
#         return segments
    
#     def _identify_segment_type(self, text):
#         """
#         Identify the type of a text segment.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Segment type as string
#         """
#         text = text.strip().lower()
        
#         # Check if it's a definition
#         if text.startswith('definition') or ': is defined as' in text or 'refers to' in text:
#             return 'definition'
            
#         # Check if it's an example
#         if text.startswith('example') or text.startswith('for example') or 'for instance' in text:
#             return 'example'
            
#         # Check if it's a theorem/proposition
#         if text.startswith('theorem') or text.startswith('proposition') or text.startswith('lemma'):
#             return 'theorem'
            
#         # Check if it's a proof
#         if text.startswith('proof') or text.endswith('q.e.d.') or text.endswith('□'):
#             return 'proof'
            
#         # Check domain-specific section types
#         if self.domain and 'section_types' in self.domain_specific_rules:
#             for section_type in self.domain_specific_rules['section_types']:
#                 if section_type.lower() in text[:20]:
#                     return section_type
        
#         # Default to paragraph
#         return 'paragraph'
    
#     def tag_content_blocks(self, blocks):
#         """
#         Classify content blocks into specific types.
        
#         Args:
#             blocks: List of content blocks from the extraction result
            
#         Returns:
#             List of content blocks with type classification added
#         """
#         tagged_blocks = []
        
#         for block in blocks:
#             block_copy = block.copy()
            
#             # Determine the block type based on content and structure
#             if 'text' in block:
#                 block_copy['block_type'] = self._identify_segment_type(block['text'])
                
#                 # Add additional semantic tags if possible
#                 block_copy['semantic_tags'] = self._generate_semantic_tags(block['text'])
            
#             tagged_blocks.append(block_copy)
            
#         return tagged_blocks
    
#     def _generate_semantic_tags(self, text):
#         """
#         Generate semantic tags for a piece of text.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             List of semantic tags
#         """
#         tags = []
        
#         # Check for domain-specific terminology
#         if self.domain and 'terminologies' in self.domain_specific_rules:
#             for term in self.domain_specific_rules['terminologies']:
#                 if term.lower() in text.lower():
#                     tags.append(term)
        
#         # Check if text contains numerical data
#         if any(char.isdigit() for char in text):
#             tags.append('numerical_data')
            
#         # Check if text contains questions
#         if '?' in text:
#             tags.append('question')
            
#         # Check if text contains lists
#         if any(line.strip().startswith(('-', '•', '*')) for line in text.split('\n')):
#             tags.append('list')
            
#         return tags
    
#     def extract_keywords_and_concepts(self, text, domain=None):
#         """
#         Extract key terms and concepts from text, optionally customized by domain.
        
#         Args:
#             text: Text to analyze
#             domain: Optional domain to customize extraction
            
#         Returns:
#             Dictionary with keywords and concepts
#         """
#         # Simple implementation using frequency-based extraction
#         # In a real implementation, this would use NLP techniques
        
#         words = text.lower().split()
#         # Remove common stop words
#         stop_words = set(['the', 'and', 'is', 'in', 'to', 'of', 'that', 'for', 'with', 'as'])
#         filtered_words = [word for word in words if word not in stop_words]
        
#         # Count word frequencies
#         word_freq = {}
#         for word in filtered_words:
#             if len(word) > 3:  # Only consider words with more than 3 characters
#                 word_freq[word] = word_freq.get(word, 0) + 1
        
#         # Get top keywords by frequency
#         keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:20]
        
#         # Extract multi-word concepts using simple pattern matching
#         concepts = []
#         if domain and 'entity_patterns' in self.domain_specific_rules:
#             import re
#             for pattern in self.domain_specific_rules['entity_patterns']:
#                 matches = re.findall(pattern, text)
#                 concepts.extend(matches)
        
#         return {
#             'keywords': [keyword for keyword, _ in keywords],
#             'concepts': concepts
#         }
    
#     def perform_sentiment_analysis(self, text):
#         """
#         Analyze tone and sentiment of content sections.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Dictionary with sentiment analysis results
#         """
#         # Simple sentiment analysis implementation
#         # In a real implementation, this would use a proper sentiment analysis model
        
#         positive_words = ['good', 'excellent', 'positive', 'beneficial', 'advantage', 
#                           'effective', 'success', 'improve', 'best', 'important']
#         negative_words = ['bad', 'poor', 'negative', 'problem', 'disadvantage', 
#                           'ineffective', 'failure', 'worse', 'worst', 'difficult']
        
#         # Count occurrences of positive and negative words
#         text_lower = text.lower()
#         positive_count = sum(text_lower.count(' ' + word + ' ') for word in positive_words)
#         negative_count = sum(text_lower.count(' ' + word + ' ') for word in negative_words)
        
#         # Determine overall sentiment
#         if positive_count > negative_count * 1.5:
#             sentiment = 'positive'
#         elif negative_count > positive_count * 1.5:
#             sentiment = 'negative'
#         else:
#             sentiment = 'neutral'
        
#         # Check for academic/formal language
#         academic_markers = ['therefore', 'thus', 'consequently', 'furthermore', 'moreover']
#         academic_count = sum(text_lower.count(' ' + marker + ' ') for marker in academic_markers)
#         is_academic = academic_count > len(text.split()) / 200  # Arbitrary threshold
        
#         return {
#             'overall_sentiment': sentiment,
#             'positive_count': positive_count,
#             'negative_count': negative_count,
#             'is_academic_tone': is_academic
#         }
    
#     def generate_summaries(self, sections, max_length=200):
#         """
#         Create concise summaries of complex sections.
        
#         Args:
#             sections: Text or sections to summarize
#             max_length: Maximum length of the summary in characters
            
#         Returns:
#             Summary text
#         """
#         # Simple extractive summarization implementation
#         # In a real implementation, this would use more sophisticated NLP techniques
        
#         if isinstance(sections, str):
#             text = sections
#         else:
#             # If sections is a dictionary or list, concatenate all text
#             text = ' '.join([section if isinstance(section, str) else section.get('text', '')
#                              for section in sections])
        
#         # Split into sentences
#         sentences = text.replace('!', '.').replace('?', '.').split('.')
#         sentences = [s.strip() for s in sentences if s.strip()]
        
#         if not sentences:
#             return ""
        
#         # Simple scoring: prefer sentences with keywords and at the beginning/end
#         keywords = self.extract_keywords_and_concepts(text)['keywords'][:10]
        
#         scored_sentences = []
#         for i, sentence in enumerate(sentences):
#             # Score based on position (beginning and end sentences are important)
#             position_score = 1.0
#             if i < len(sentences) / 10:  # First 10%
#                 position_score = 2.0
#             elif i > len(sentences) * 0.9:  # Last 10%
#                 position_score = 1.5
                
#             # Score based on keyword presence
#             keyword_score = sum(1.0 for keyword in keywords if keyword in sentence.lower())
            
#             # Score based on sentence length (not too short, not too long)
#             length = len(sentence.split())
#             length_score = 1.0
#             if 5 <= length <= 20:
#                 length_score = 1.5
                
#             # Combine scores
#             total_score = position_score + keyword_score + length_score
#             scored_sentences.append((sentence, total_score))
        
#         # Sort by score and select top sentences
#         scored_sentences.sort(key=lambda x: x[1], reverse=True)
        
#         # Select top sentences up to max_length
#         summary = ""
#         for sentence, _ in scored_sentences:
#             if len(summary) + len(sentence) <= max_length:
#                 summary += sentence + ". "
#             else:
#                 break
                
#         return summary.strip()
    

In [ ]:
# class ContentAnalyzer(EnhancedPDFExtractor):
#     def __init__(self, language_model=None, domain=None, tesseract_path=None, 
#                 math_recognition=True, chem_recognition=True):
#         """
#         Initialize the comprehensive PDF analyzer with extraction and analysis capabilities
        
#         Args:
#             language_model: Optional pre-trained language model for advanced text analysis
#             domain: Optional domain specification to customize analysis for specific fields
#             tesseract_path: Path to Tesseract OCR executable
#             math_recognition: Boolean to enable mathematical expression recognition
#             chem_recognition: Boolean to enable chemical formula recognition
#         """
#         # Initialize the parent class (EnhancedPDFExtractor)
#         super().__init__(tesseract_path, math_recognition, chem_recognition)
        
#         # Initialize ContentAnalyzer components
#         self.language_model = language_model
#         self.domain = domain
#         self.domain_specific_rules = self._load_domain_rules(domain)

#     # Import methods from ContentAnalyzer
#     def _load_domain_rules(self, domain):
#         """
#         Load domain-specific rules and patterns for content analysis.
        
#         Args:
#             domain: The domain to load rules for
            
#         Returns:
#             Dictionary of rules and patterns specific to the domain
#         """
#         if domain is None:
#             return {}
            
#         domain_rules = {
#             'science': {
#                 'terminologies': ['experiment', 'hypothesis', 'theory', 'data'],
#                 'section_types': ['methodology', 'results', 'discussion', 'conclusion'],
#                 'entity_patterns': [r'[A-Z][a-z]+ [a-z]+ium', r'\d+\.\d+ [a-z]+']
#             },
#             'math': {
#                 'terminologies': ['theorem', 'proof', 'lemma', 'corollary'],
#                 'section_types': ['definition', 'theorem', 'proof', 'example'],
#                 'entity_patterns': [r'[A-Z][a-z]+ (theorem|formula|equation)', r'equation \(\d+\)']
#             },
#             'literature': {
#                 'terminologies': ['symbolism', 'metaphor', 'character', 'plot'],
#                 'section_types': ['exposition', 'climax', 'resolution', 'analysis'],
#                 'entity_patterns': [r'[A-Z][a-z]+ (symbolizes|represents)', r'theme of [a-z]+']
#             }
#         }
        
#         return domain_rules.get(domain.lower(), {})
    
        
#     def tag_content_blocks(self, blocks):
#         """
#         Classify content blocks into specific types.
        
#         Args:
#             blocks: List of content blocks from the extraction result
            
#         Returns:
#             List of content blocks with type classification added
#         """
#         tagged_blocks = []
        
#         for block in blocks:
#             block_copy = block.copy()
            
#             # Determine the block type based on content and structure
#             if 'text' in block:
#                 block_copy['block_type'] = self._identify_segment_type(block['text'])
                
#                 # Add additional semantic tags if possible
#                 block_copy['semantic_tags'] = self._generate_semantic_tags(block['text'])
            
#             tagged_blocks.append(block_copy)
            
#         return tagged_blocks
    
    
#     def _generate_semantic_tags(self, text):
#         """
#         Generate semantic tags for a piece of text.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             List of semantic tags
#         """
#         tags = []
        
#         # Check for domain-specific terminology
#         if self.domain and 'terminologies' in self.domain_specific_rules:
#             for term in self.domain_specific_rules['terminologies']:
#                 if term.lower() in text.lower():
#                     tags.append(term)
        
#         # Check if text contains numerical data
#         if any(char.isdigit() for char in text):
#             tags.append('numerical_data')
            
#         # Check if text contains questions
#         if '?' in text:
#             tags.append('question')
            
#         # Check if text contains lists
#         if any(line.strip().startswith(('-', '•', '*')) for line in text.split('\n')):
#             tags.append('list')
            
#         return tags
    
#     def extract_keywords_and_concepts(self, text, domain=None):
#         """
#         Extract key terms and concepts from text, optionally customized by domain.
        
#         Args:
#             text: Text to analyze
#             domain: Optional domain to customize extraction
            
#         Returns:
#             Dictionary with keywords and concepts
#         """
#         # Simple implementation using frequency-based extraction
#         # In a real implementation, this would use NLP techniques
        
#         words = text.lower().split()
#         # Remove common stop words
#         stop_words = set(['the', 'and', 'is', 'in', 'to', 'of', 'that', 'for', 'with', 'as'])
#         filtered_words = [word for word in words if word not in stop_words]
        
#         # Count word frequencies
#         word_freq = {}
#         for word in filtered_words:
#             if len(word) > 3:  # Only consider words with more than 3 characters
#                 word_freq[word] = word_freq.get(word, 0) + 1
        
#         # Get top keywords by frequency
#         keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:20]
        
#         # Extract multi-word concepts using simple pattern matching
#         concepts = []
#         if domain and 'entity_patterns' in self.domain_specific_rules:
#             import re
#             for pattern in self.domain_specific_rules['entity_patterns']:
#                 matches = re.findall(pattern, text)
#                 concepts.extend(matches)
        
#         return {
#             'keywords': [keyword for keyword, _ in keywords],
#             'concepts': concepts
#         }
    
#     def perform_sentiment_analysis(self, text):
#         """
#         Analyze tone and sentiment of content sections.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Dictionary with sentiment analysis results
#         """
#         # Simple sentiment analysis implementation
#         # In a real implementation, this would use a proper sentiment analysis model
        
#         positive_words = ['good', 'excellent', 'positive', 'beneficial', 'advantage', 
#                           'effective', 'success', 'improve', 'best', 'important']
#         negative_words = ['bad', 'poor', 'negative', 'problem', 'disadvantage', 
#                           'ineffective', 'failure', 'worse', 'worst', 'difficult']
        
#         # Count occurrences of positive and negative words
#         text_lower = text.lower()
#         positive_count = sum(text_lower.count(' ' + word + ' ') for word in positive_words)
#         negative_count = sum(text_lower.count(' ' + word + ' ') for word in negative_words)
        
#         # Determine overall sentiment
#         if positive_count > negative_count * 1.5:
#             sentiment = 'positive'
#         elif negative_count > positive_count * 1.5:
#             sentiment = 'negative'
#         else:
#             sentiment = 'neutral'
        
#         # Check for academic/formal language
#         academic_markers = ['therefore', 'thus', 'consequently', 'furthermore', 'moreover']
#         academic_count = sum(text_lower.count(' ' + marker + ' ') for marker in academic_markers)
#         is_academic = academic_count > len(text.split()) / 200  # Arbitrary threshold
        
#         return {
#             'overall_sentiment': sentiment,
#             'positive_count': positive_count,
#             'negative_count': negative_count,
#             'is_academic_tone': is_academic
#         }
    
#     def generate_summaries(self, sections, max_length=200):
#         """
#         Create concise summaries of complex sections.
        
#         Args:
#             sections: Text or sections to summarize
#             max_length: Maximum length of the summary in characters
            
#         Returns:
#             Summary text
#         """
#         # Simple extractive summarization implementation
#         # In a real implementation, this would use more sophisticated NLP techniques
        
#         if isinstance(sections, str):
#             text = sections
#         else:
#             # If sections is a dictionary or list, concatenate all text
#             text = ' '.join([section if isinstance(section, str) else section.get('text', '')
#                              for section in sections])
        
#         # Split into sentences
#         sentences = text.replace('!', '.').replace('?', '.').split('.')
#         sentences = [s.strip() for s in sentences if s.strip()]
        
#         if not sentences:
#             return ""
        
#         # Simple scoring: prefer sentences with keywords and at the beginning/end
#         keywords = self.extract_keywords_and_concepts(text)['keywords'][:10]
        
#         scored_sentences = []
#         for i, sentence in enumerate(sentences):
#             # Score based on position (beginning and end sentences are important)
#             position_score = 1.0
#             if i < len(sentences) / 10:  # First 10%
#                 position_score = 2.0
#             elif i > len(sentences) * 0.9:  # Last 10%
#                 position_score = 1.5
                
#             # Score based on keyword presence
#             keyword_score = sum(1.0 for keyword in keywords if keyword in sentence.lower())
            
#             # Score based on sentence length (not too short, not too long)
#             length = len(sentence.split())
#             length_score = 1.0
#             if 5 <= length <= 20:
#                 length_score = 1.5
                
#             # Combine scores
#             total_score = position_score + keyword_score + length_score
#             scored_sentences.append((sentence, total_score))
        
#         # Sort by score and select top sentences
#         scored_sentences.sort(key=lambda x: x[1], reverse=True)
        
#         # Select top sentences up to max_length
#         summary = ""
#         for sentence, _ in scored_sentences:
#             if len(summary) + len(sentence) <= max_length:
#                 summary += sentence + ". "
#             else:
#                 break
                
#         return summary.strip()
    
        
#     def segment_content(self, extraction_result):
#         """
#         Divide content into logical segments beyond just headings.
        
#         Args:
#             extraction_result: Dictionary containing the extracted content
            
#         Returns:
#             List of content segments with their type and content
#         """
#         segments = []
        
#         # If document is already structured by headings
#         if 'document_structure' in extraction_result:
#             structure = extraction_result['document_structure']
#             for section in structure:
#                 # Create a segment for the heading
#                 if 'heading' in section:
#                     segments.append({
#                         'type': 'heading',
#                         'level': section.get('level', 1),
#                         'content': section['heading']
#                     })
                
#                 # Process the content under this heading
#                 if 'content' in section:
#                     content_segments = self._segment_text_block(section['content'])
#                     segments.extend(content_segments)
        
#         # If no structure is available, segment the raw text
#         elif 'text_content' in extraction_result:
#             segments = self._segment_text_block(extraction_result['text_content'])
        
#         return segments
    
#     def _segment_text_block(self, text):
#         """
#         Break a block of text into logical segments.
        
#         Args:
#             text: Text content to segment
            
#         Returns:
#             List of segments with their identified type and content
#         """
#         # Simple implementation - split by paragraphs and try to determine type
#         segments = []
#         paragraphs = text.split('\n\n')
        
#         for paragraph in paragraphs:
#             if not paragraph.strip():
#                 continue
                
#             segment_type = self._identify_segment_type(paragraph)
#             segments.append({
#                 'type': segment_type,
#                 'content': paragraph.strip()
#             })
            
#         return segments
    
#     def _identify_segment_type(self, text):
#         """
#         Identify the type of a text segment.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Segment type as string
#         """
#         text = text.strip().lower()
        
#         # Check if it's a definition
#         if text.startswith('definition') or ': is defined as' in text or 'refers to' in text:
#             return 'definition'
            
#         # Check if it's an example
#         if text.startswith('example') or text.startswith('for example') or 'for instance' in text:
#             return 'example'
            
#         # Check if it's a theorem/proposition
#         if text.startswith('theorem') or text.startswith('proposition') or text.startswith('lemma'):
#             return 'theorem'
            
#         # Check if it's a proof
#         if text.startswith('proof') or text.endswith('q.e.d.') or text.endswith('□'):
#             return 'proof'
            
#         # Check domain-specific section types
#         if self.domain and 'section_types' in self.domain_specific_rules:
#             for section_type in self.domain_specific_rules['section_types']:
#                 if section_type.lower() in text[:20]:
#                     return section_type
        
#         # Default to paragraph
#         return 'paragraph'
    

#     def analyze_content(self, extraction_result):
#         """
#         Main method to analyze extracted content from a PDF.
        
#         Args:
#             extraction_result: Dictionary containing the extracted content
            
#         Returns:
#             Dictionary containing analysis results
#         """
#         analyzed_result = {}
        
#         # Add semantic analysis to the content blocks
#         if 'pages' in extraction_result:
#             all_blocks = []
#             for page in extraction_result['pages']:
#                 if 'blocks' in page:
#                     all_blocks.extend(page['blocks'])
#             analyzed_result['content_blocks'] = self.tag_content_blocks(all_blocks)
        
#         # Extract keywords and concepts from the full text
#         if 'full_text' in extraction_result:
#             analyzed_result['keywords_and_concepts'] = self.extract_keywords_and_concepts(
#                 extraction_result['full_text'], self.domain
#             )
        
#         # Generate summaries for sections
#         if 'pages' in extraction_result:
#             section_texts = {}
#             for page in extraction_result['pages']:
#                 for section in page.get('structured_content', []):
#                     heading = section.get('heading', 'Unnamed Section')
#                     content = '\n'.join(section.get('content', []))
#                     if heading:
#                         section_texts[heading] = section_texts.get(heading, '') + '\n' + content
            
#             analyzed_result['section_summaries'] = {
#                 section_id: self.generate_summaries(section_content)
#                 for section_id, section_content in section_texts.items()
#                 if section_content.strip()  # Only process non-empty sections
#             }
        
#         # Segment content into logical units
#         analyzed_result['content_segments'] = self.segment_content(extraction_result)
        
#         # Add sentiment analysis
#         if 'full_text' in extraction_result:
#             analyzed_result['sentiment_analysis'] = self.perform_sentiment_analysis(
#                 extraction_result['full_text']
#             )
        
#         return analyzed_result

#     def extract_analyze_and_save(self, pdf_path, output_path_base, output_dir=None, 
#                                extract_images=True, extract_tables=True, 
#                                perform_analysis=True, formats=None):
#         """
#         Comprehensive method that extracts, analyzes and saves PDF content
        
#         Args:
#             pdf_path: Path to the PDF file
#             output_path_base: Base path for output files (without extension)
#             output_dir: Directory to save extracted images (optional)
#             extract_images: Boolean indicating whether to extract images
#             extract_tables: Boolean indicating whether to extract tables
#             perform_analysis: Boolean indicating whether to perform content analysis
#             formats: List of formats to save ('json', 'text', 'markdown') or None for all
            
#         Returns:
#             Dictionary containing both extraction and analysis results
#         """
#         # Extract content with cleaning (from EnhancedPDFExtractor)
#         extraction_result = self.extract_content_with_cleaning(
#             pdf_path, output_dir, extract_images, extract_tables
#         )
        
#         # Perform content analysis if requested
#         if perform_analysis:
#             analysis_result = self.analyze_content(extraction_result)
#             # Combine the results
#             comprehensive_result = {**extraction_result, **analysis_result}
#         else:
#             comprehensive_result = extraction_result
        
#         # Save in all requested formats
#         output_files = self.save_extraction_result(comprehensive_result, output_path_base, formats)
        
#         # Add output files to result
#         comprehensive_result['output_files'] = output_files
        
#         return comprehensive_result

#     def save_extraction_result(self, result, output_path_base, formats=None):
#         """
#         Save extraction and analysis result in multiple formats with a single call

#         Args:
#             result: The extraction and analysis result dictionary
#             output_path_base: Base path for output files (without extension)
#             formats: List of formats to save ('json', 'text', 'markdown') or None for all

#         Returns:
#             Dictionary with paths to all generated files
#         """
#         if formats is None:
#             formats = ['json', 'text', 'markdown']

#         output_files = {}

#         # Handle JSON format
#         if 'json' in formats:
#             json_path = f"{output_path_base}.json"
#             try:
#                 with open(json_path, 'w', encoding='utf-8') as f:
#                     json.dump(result, f, indent=2, ensure_ascii=False)
#                 output_files['json'] = json_path
#                 self.logger.info(f"JSON result saved to {json_path}")
#             except Exception as e:
#                 self.logger.error(f"Error saving JSON: {e}")

#         # Handle text format
#         if 'text' in formats:
#             text_path = f"{output_path_base}.txt"
#             try:
#                 # Use full_text if available, otherwise compile from pages
#                 if "full_text" in result:
#                     full_text = result["full_text"]
#                 else:
#                     # Generate full text from pages
#                     text_blocks = []
#                     for page in result.get("pages", []):
#                         for block in page.get("blocks", []):
#                             if "text" in block:
#                                 text_blocks.append(block["text"])
#                     full_text = "\n\n".join(text_blocks)

#                 with open(text_path, 'w', encoding='utf-8') as f:
#                     f.write(full_text)
#                 output_files['text'] = text_path
#                 self.logger.info(f"Plain text saved to {text_path}")
#             except Exception as e:
#                 self.logger.error(f"Error saving text: {e}")

#         # Enhanced markdown that includes analysis results
#         if 'markdown' in formats:
#             markdown_path = f"{output_path_base}.md"
#             try:
#                 markdown_content = convert_to_enhanced_markdown(result, image_output_dir=output_path_base)
#                 with open(markdown_path, 'w', encoding='utf-8') as f:
#                     f.write(markdown_content)
#                 output_files['markdown'] = markdown_path
#                 self.logger.info(f"Enhanced markdown content saved to {markdown_path}")
#             except Exception as e:
#                 self.logger.error(f"Error saving markdown: {e}")

#         return output_files

# import os
# def convert_to_enhanced_markdown(extraction_result, image_output_dir=None):
#     """
#     Convert the extracted and analyzed PDF content to an enhanced markdown document
#     that includes semantic analysis information
    
#     Args:
#         extraction_result: The dictionary returned by extract_analyze_and_save
#         image_output_dir: Directory where images are stored (to create proper paths)
        
#     Returns:
#         String containing the formatted markdown content with analysis
#     """
#     # Start with basic markdown conversion
#     markdown_lines = []
    
#     # Add document title if available
#     title = extraction_result.get("metadata", {}).get("title")
#     if title:
#         markdown_lines.append(f"# {title}")
#         markdown_lines.append("")
    
#     # Process each page's structured content
#     for page in extraction_result.get("pages", []):
#         structured_content = page.get("structured_content", [])
        
#         for section in structured_content:
#             heading_text = section.get("heading")
#             heading_level = section.get("heading_level", 3)  # Default to level 3
#             content = section.get("content", [])
            
#             # Add section heading with proper markdown level
#             if heading_text:
#                 # Ensure heading level is between 1-6 (valid markdown heading levels)
#                 valid_level = max(1, min(heading_level, 6))
#                 heading_marker = "#" * valid_level
#                 markdown_lines.append(f"{heading_marker} {heading_text}")
#                 markdown_lines.append("")
            
#             # Add section content - fix for int vs string issue
#             for text in content:
#                 if text is None:
#                     continue
                    
#                 # Convert to string if not already
#                 if not isinstance(text, str):
#                     text = str(text)
                    
#                 if text.strip():
#                     # Check if this might be a list item
#                     if text.strip().startswith("•") or text.strip().startswith("-") or re.match(r"^\d+\.\s", text.strip()):
#                         # It's likely a list item, preserve it
#                         markdown_lines.append(text)
#                     else:
#                         # Regular paragraph
#                         markdown_lines.append(text)
#                     markdown_lines.append("")  # Add blank line after each content block
    
#     # Process tables if available
#     tables = extraction_result.get("tables", [])
#     if tables:
#         markdown_lines.append("## Tables")
#         markdown_lines.append("")
        
#         for i, table in enumerate(tables):
#             markdown_lines.append(f"### Table {i+1}")
#             markdown_lines.append("")
            
#             # Convert table data to markdown format
#             table_data = table.get("data", [])
#             if table_data:
#                 # Get all keys from all rows to ensure complete header
#                 all_keys = set()
#                 for row in table_data:
#                     all_keys.update(row.keys())
                
#                 # Create header row
#                 header = " | ".join(str(key) for key in all_keys)  # Convert keys to strings
#                 markdown_lines.append(f"| {header} |")
                
#                 # Create separator row
#                 separator = " | ".join(["---" for _ in all_keys])
#                 markdown_lines.append(f"| {separator} |")
                
#                 # Create data rows
#                 for row in table_data:
#                     values = [str(row.get(key, "")) for key in all_keys]  # Ensure all values are strings
#                     data_row = " | ".join(values)
#                     markdown_lines.append(f"| {data_row} |")
                
#                 markdown_lines.append("")
    
#     # Process images if available
#     images = extraction_result.get("images", [])
#     if images and any(img.get("file_name") for img in images):
#         markdown_lines.append("## Images")
#         markdown_lines.append("")
        
#         for i, image in enumerate(images):
#             file_name = image.get("file_name")
#             ocr_text = image.get("ocr_text")
            
#             if file_name:
#                 markdown_lines.append(f"### Image {i+1}")
#                 markdown_lines.append("")
                
#                 # Fix: Create proper relative path to the image
#                 if image_output_dir:
#                     # Use path relative to the markdown file
#                     img_path = os.path.join(os.path.basename(image_output_dir), file_name)
#                     # Replace backslashes with forward slashes for markdown compatibility
#                     img_path = img_path.replace("\\", "/")
#                 else:
#                     img_path = file_name
                    
#                 markdown_lines.append(f"![Image {i+1}]({img_path})")
                
#                 if ocr_text:
#                     markdown_lines.append("")
#                     markdown_lines.append("**OCR Text:**")
#                     markdown_lines.append("")
#                     markdown_lines.append(f"```\n{ocr_text}\n```")
                
#                 markdown_lines.append("")
    
#     # Process mathematical content if available
#     math_expressions = extraction_result.get("math_expressions", [])
#     if math_expressions:
#         markdown_lines.append("## Mathematical Content")
#         markdown_lines.append("")
        
#         for i, expr in enumerate(math_expressions):
#             latex = expr.get("latex")
#             if latex:
#                 markdown_lines.append(f"### Expression {i+1}")
#                 markdown_lines.append("")
#                 # Use double dollar signs for display math in markdown
#                 markdown_lines.append(f"$${latex}$$")
#                 markdown_lines.append("")
    
#     # Process chemical content if available
#     chemical_content = extraction_result.get("chemical_content", [])
#     if chemical_content:
#         markdown_lines.append("## Chemical Content")
#         markdown_lines.append("")
        
#         # Process reactions
#         reactions = []
#         for item in chemical_content:
#             if isinstance(item, dict) and "reactions" in item:
#                 reactions.extend(item["reactions"])
        
#         if reactions:
#             markdown_lines.append("### Chemical Reactions")
#             markdown_lines.append("")
            
#             for i, reaction in enumerate(reactions):
#                 reaction_text = reaction.get("text", "")
#                 if reaction_text:
#                     markdown_lines.append(f"{i+1}. `{reaction_text}`")
#                     markdown_lines.append("")
    
#     # Add semantic analysis sections if available
#     if 'keywords_and_concepts' in extraction_result:
#         markdown_lines.append("## Content Analysis")
#         markdown_lines.append("")
        
#         # Add keywords
#         if 'keywords' in extraction_result['keywords_and_concepts']:
#             markdown_lines.append("### Keywords")
#             markdown_lines.append("")
#             keywords = extraction_result['keywords_and_concepts']['keywords']
#             markdown_lines.append(", ".join(keywords))
#             markdown_lines.append("")
            
#         # Add concepts
#         if 'concepts' in extraction_result['keywords_and_concepts']:
#             markdown_lines.append("### Key Concepts")
#             markdown_lines.append("")
#             concepts = extraction_result['keywords_and_concepts']['concepts']
#             for concept in concepts:
#                 markdown_lines.append(f"- {concept}")
#             markdown_lines.append("")
    
#     # Add section summaries if available
#     if 'section_summaries' in extraction_result:
#         markdown_lines.append("## Section Summaries")
#         markdown_lines.append("")
        
#         for section_id, summary in extraction_result['section_summaries'].items():
#             markdown_lines.append(f"### {section_id}")
#             markdown_lines.append("")
#             markdown_lines.append(summary)
#             markdown_lines.append("")
    
#     # Add sentiment analysis if available
#     if 'sentiment_analysis' in extraction_result:
#         markdown_lines.append("## Document Sentiment")
#         markdown_lines.append("")
#         sentiment = extraction_result['sentiment_analysis']
#         markdown_lines.append(f"Overall sentiment: **{sentiment['overall_sentiment']}**")
#         markdown_lines.append("")
#         markdown_lines.append(f"- Positive indicators: {sentiment['positive_count']}")
#         markdown_lines.append(f"- Negative indicators: {sentiment['negative_count']}")
#         if 'is_academic_tone' in sentiment:
#             markdown_lines.append(f"- Academic tone: {'Yes' if sentiment['is_academic_tone'] else 'No'}")
#         markdown_lines.append("")
    
#     # Add metadata section at the end
#     metadata = extraction_result.get("metadata", {})
#     if metadata:
#         markdown_lines.append("## Document Metadata")
#         markdown_lines.append("")
#         markdown_lines.append("| Property | Value |")
#         markdown_lines.append("| --- | --- |")
        
#         for key, value in metadata.items():
#             if key == "table_of_contents":
#                 continue  # Skip TOC for the table
#             # Ensure values are strings
#             markdown_lines.append(f"| {key} | {str(value)} |")
        
#         markdown_lines.append("")
    
#     # Return the complete markdown document
#     return "\n".join(markdown_lines)


In [ ]:
# class ContentAnalyzer:
#     def __init__(self, language_model=None, domain=None, extractor=None):
#         """
#         Initialize with optional connection to an extractor
        
#         Args:
#             language_model: Optional pre-trained language model
#             domain: Optional domain specification
#             extractor: Optional PDFContentExtractor to use for segmentation
#         """
#         self.language_model = language_model
#         self.domain = domain
#         self.domain_specific_rules = self._load_domain_rules(domain)
#         self.extractor = extractor
#         self.stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 
#                       'by', 'this', 'with', 'i', 'you', 'it', 'not', 'or', 'be', 'are', 
#                       'from', 'at', 'as', 'your', 'have', 'has', 'had', 'was', 'were', 
#                       'they', 'their', 'them', 'we', 'our', 'us', 'he', 'she', 'his', 
#                       'her', 'him', 'an', 'can', 'will', 'just', 'do', 'does', 'did',
#                       'but', 'if', 'so', 'what', 'when', 'where', 'how', 'all', 'any',
#                       'both', 'each', 'few', 'more', 'most', 'some', 'such', 'no', 'nor',
#                       'too', 'very', 'one', 'every', 'least', 'less', 'many', 'now',
#                       'ever', 'never', 'say', 'says', 'said', 'also', 'like', 'time'}
    
#     def _load_domain_rules(self, domain):
#         """
#         Load domain-specific rules and patterns for content analysis.
        
#         Args:
#             domain: The domain to load rules for
            
#         Returns:
#             Dictionary of rules and patterns specific to the domain
#         """
#         if domain is None:
#             return {}
            
#         domain_rules = {
#             # Existing domains remain unchanged
#             'science': {
#                 'terminologies': ['experiment', 'hypothesis', 'theory', 'data'],
#                 'section_types': ['methodology', 'results', 'discussion', 'conclusion'],
#                 'entity_patterns': [r'[A-Z][a-z]+ [a-z]+ium', r'\d+\.\d+ [a-z]+']
#             },
#             'math': {
#                 'terminologies': ['theorem', 'proof', 'lemma', 'corollary'],
#                 'section_types': ['definition', 'theorem', 'proof', 'example'],
#                 'entity_patterns': [r'[A-Z][a-z]+ (theorem|formula|equation)', r'equation \(\d+\)']
#             },
#             'literature': {
#                 'terminologies': ['symbolism', 'metaphor', 'character', 'plot', 'theme', 'motif', 'foreshadowing', 'irony', 'allegory', 'setting'],
#                 'section_types': ['exposition', 'climax', 'resolution', 'analysis', 'chapter', 'prologue', 'epilogue', 'act', 'scene', 'soliloquy'],
#                 'entity_patterns': [
#                     r'[A-Z][a-z]+ (symbolizes|represents)', 
#                     r'theme of [a-z]+', 
#                     r'Chapter [IVXLCDM]+', 
#                     r'Chapter \d+',
#                     r'Act [IVXLCDM]+', 
#                     r'Scene [IVXLCDM]+', 
#                     r'Enter [A-Z][a-z]+', 
#                     r'\[Exit[^\]]*\]', 
#                     r'\[Aside\]'
#                 ],
#                 'subtypes': {
#                     'play': {
#                         'shakespeare': {
#                             'terminologies': ['soliloquy', 'aside', 'iambic pentameter', 'sonnet', 'blank verse', 'couplet', 'quatrain', 'folio', 'quarto'],
#                             'themes': ['power_corruption', 'love_romance', 'betrayal', 'appearance_reality', 'supernatural', 'order_chaos', 'revenge']
#                         },
#                         'modern': {
#                             'terminologies': ['dialogue', 'stage direction', 'monologue', 'act', 'scene']
#                         }
#                     },
#                     'novel': {
#                         'terminologies': ['narrator', 'protagonist', 'antagonist', 'setting', 'plot', 'chapter']
#                     },
#                     'poem': {
#                         'terminologies': ['stanza', 'verse', 'rhyme', 'meter', 'rhythm', 'sonnet', 'ode', 'elegy']
#                     }
#                 }
#             }
#         }
        
#         return domain_rules.get(domain.lower(), {})
    
#     def analyze_content(self, extraction_result):
#         """
#         Main method to analyze extracted content from a PDF.
        
#         Args:
#             extraction_result: Dictionary containing the extracted content from PDFContentExtractor
            
#         Returns:
#             Dictionary containing the original content enriched with semantic annotations
#         """
#         analyzed_result = extraction_result.copy()
        
#         # Add semantic analysis to the content
#         if 'content_blocks' in analyzed_result:
#             analyzed_result['content_blocks'] = self.tag_content_blocks(analyzed_result['content_blocks'])
        
#         # Extract keywords and concepts
#         if 'text_content' in analyzed_result:
#             analyzed_result['keywords_and_concepts'] = self.extract_keywords_and_concepts(
#                 analyzed_result['text_content'], self.domain
#             )
        
#         # Generate summaries for sections
#         if 'sections' in analyzed_result:
#             analyzed_result['section_summaries'] = {
#                 section_id: self.generate_summaries(section_content)
#                 for section_id, section_content in analyzed_result['sections'].items()
#             }
        
#         # Segment content into logical units
#         analyzed_result['content_segments'] = self.segment_content(extraction_result)
        
#         # Add sentiment analysis if text is available
#         if 'text_content' in analyzed_result:
#             analyzed_result['sentiment_analysis'] = self.perform_sentiment_analysis(
#                 analyzed_result['text_content']
#             )
        
#         # Add literary work analysis if domain suggests it
#         if self.domain == 'literature':
#             # Try to determine if this is a play, specifically Shakespeare
#             text_content = extraction_result.get('text_content', '')

#             # Check for play-like structure (character names followed by dialogue)
#             import re
#             dialogue_patterns = re.findall(r'([A-Z]{2,}[A-Z\s]*)[\.:]\s', text_content)

#             work_subtype = None

#             # Determine the work subtype
#             if len(dialogue_patterns) > 10:
#                 # This looks like a play
#                 work_subtype = 'play'

#                 # Check for Shakespeare-specific markers
#                 shakespeare_markers = ['thee', 'thou', 'thy', 'hath', 'doth', 'prithee', 'forsooth', 'anon']
#                 shakespeare_count = sum(text_content.lower().count(' ' + marker + ' ') for marker in shakespeare_markers)

#                 if shakespeare_count > 5:
#                     work_subtype += ':shakespeare'
#             elif re.search(r'Chapter [IVXLCDM]+|Chapter \d+', text_content):
#                 work_subtype = 'novel'
#             elif self._detect_poetic_structure(text_content):  # You'd need to implement this method
#                 work_subtype = 'poem'

#             if work_subtype:
#                 try:
#                     # Create literary analyzer
#                     literary_analyzer = LiteraryWorkAnalyzer(work_subtype)

#                     # Add literary analysis
#                     analyzed_result['literary_analysis'] = {
#                         'work_type': work_subtype,
#                         'characters': literary_analyzer.analyze_characters(text_content),
#                     }

#                     # Add structure analysis based on work type
#                     if work_subtype.startswith('play'):
#                         analyzed_result['literary_analysis']['structure'] = literary_analyzer.identify_scenes(text_content)
#                         analyzed_result['literary_analysis']['verse_patterns'] = literary_analyzer.analyze_verse_patterns(text_content)

#                         # For Shakespeare plays specifically
#                         if work_subtype.endswith('shakespeare'):
#                             analyzed_result['literary_analysis']['themes'] = literary_analyzer.detect_shakespearean_themes(text_content)
#                     elif work_subtype == 'novel':
#                         analyzed_result['literary_analysis']['structure'] = literary_analyzer.identify_scenes(text_content)

#                     # Add literary devices analysis for all literature
#                     analyzed_result['literary_analysis']['literary_devices'] = literary_analyzer.detect_literary_devices(text_content)

#                 except Exception as e:
#                     # Fail gracefully if the literary analysis can't be performed
#                     analyzed_result['literary_analysis_error'] = str(e)

#         return analyzed_result
    
#     def segment_content(self, extraction_result):
#         """
#         Divides content into logical segments using extractor if available,
#         otherwise uses built-in segmentation
        
#         Args:
#             extraction_result: Dictionary with extracted content
            
#         Returns:
#             List of content segments with semantic annotations
#         """
#         segments = []
        
#         # Use extractor for segmentation if available
#         if self.extractor and hasattr(self.extractor, 'segment_text'):
#             # If we have structured content already
#             if 'pages' in extraction_result:
#                 for page in extraction_result['pages']:
#                     for block in page.get('blocks', []):
#                         # Get base segmentation from extractor
#                         if 'text' in block:
#                             text_segments = self.extractor.segment_text(block['text'])
#                             for text in text_segments:
#                                 # Add semantic annotation 
#                                 segment_type = self._identify_semantic_segment_type(text)
#                                 segments.append({
#                                     'type': segment_type,
#                                     'content': text,
#                                     'semantic_tags': self._generate_semantic_tags(text)
#                                 })
#             # If we only have raw text
#             elif 'text_content' in extraction_result:
#                 text_segments = self.extractor.segment_text(extraction_result['text_content'])
#                 for text in text_segments:
#                     segment_type = self._identify_semantic_segment_type(text)
#                     segments.append({
#                         'type': segment_type,
#                         'content': text,
#                         'semantic_tags': self._generate_semantic_tags(text)
#                     })
#         else:
#             # Fall back to built-in segmentation
#             if 'document_structure' in extraction_result:
#                 structure = extraction_result['document_structure']
#                 for section in structure:
#                     # Create a segment for the heading
#                     if 'heading' in section:
#                         segments.append({
#                             'type': 'heading',
#                             'level': section.get('level', 1),
#                             'content': section['heading'],
#                             'semantic_tags': self._generate_semantic_tags(section['heading'])
#                         })
                    
#                     # Process the content under this heading
#                     if 'content' in section:
#                         content_segments = self._segment_text_block(section['content'])
#                         segments.extend(content_segments)
            
#             # If no structure is available, segment the raw text
#             elif 'text_content' in extraction_result:
#                 segments = self._segment_text_block(extraction_result['text_content'])
        
#         return segments
    
#     def _segment_text_block(self, text):
#         """
#         Break a block of text into logical segments.
        
#         Args:
#             text: Text content to segment
            
#         Returns:
#             List of segments with their identified type and content
#         """
#         # Simple implementation - split by paragraphs and try to determine type
#         segments = []
#         paragraphs = text.split('\n\n')
        
#         for paragraph in paragraphs:
#             if not paragraph.strip():
#                 continue
                
#             segment_type = self._identify_segment_type(paragraph)
#             segments.append({
#                 'type': segment_type,
#                 'content': paragraph.strip()
#             })
            
#         return segments
    
#     def _identify_segment_type(self, text):
#         """
#         Identify the type of a text segment.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Segment type as string
#         """
#         text = text.strip().lower()
        
#         # Check if it's a definition
#         if text.startswith('definition') or ': is defined as' in text or 'refers to' in text:
#             return 'definition'
            
#         # Check if it's an example
#         if text.startswith('example') or text.startswith('for example') or 'for instance' in text:
#             return 'example'
            
#         # Check if it's a theorem/proposition
#         if text.startswith('theorem') or text.startswith('proposition') or text.startswith('lemma'):
#             return 'theorem'
            
#         # Check if it's a proof
#         if text.startswith('proof') or text.endswith('q.e.d.') or text.endswith('□'):
#             return 'proof'
            
#         # Check domain-specific section types
#         if self.domain and 'section_types' in self.domain_specific_rules:
#             for section_type in self.domain_specific_rules['section_types']:
#                 if section_type.lower() in text[:20]:
#                     return section_type
        
#         # Default to paragraph
#         return 'paragraph'
    
#     def tag_content_blocks(self, blocks):
#         """
#         Classify content blocks into specific types.
        
#         Args:
#             blocks: List of content blocks from the extraction result
            
#         Returns:
#             List of content blocks with type classification added
#         """
#         tagged_blocks = []
        
#         for block in blocks:
#             block_copy = block.copy()
            
#             # Determine the block type based on content and structure
#             if 'text' in block:
#                 block_copy['block_type'] = self._identify_segment_type(block['text'])
                
#                 # Add additional semantic tags if possible
#                 block_copy['semantic_tags'] = self._generate_semantic_tags(block['text'])
            
#             tagged_blocks.append(block_copy)
            
#         return tagged_blocks
    
#     def _generate_semantic_tags(self, text):
#         """
#         Generate semantic tags for a piece of text.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             List of semantic tags
#         """
#         tags = []
        
#         # Check for domain-specific terminology
#         if self.domain and 'terminologies' in self.domain_specific_rules:
#             for term in self.domain_specific_rules['terminologies']:
#                 if term.lower() in text.lower():
#                     tags.append(term)
        
#         # Check if text contains numerical data
#         if any(char.isdigit() for char in text):
#             tags.append('numerical_data')
            
#         # Check if text contains questions
#         if '?' in text:
#             tags.append('question')
            
#         # Check if text contains lists
#         if any(line.strip().startswith(('-', '•', '*')) for line in text.split('\n')):
#             tags.append('list')
            
#         return tags
    
#     def extract_keywords_and_concepts(self, text, domain=None, max_terms=20, min_word_length=3):
#         """
#         Core method to extract key terms and concepts from text, optionally customized by domain.

#         Args:
#             text: Text to analyze
#             domain: Optional domain to customize extraction
#             max_terms: Maximum number of terms to return
#             min_word_length: Minimum length of words to consider

#         Returns:
#             Dictionary with keywords and concepts
#         """
#         # Split text into words and clean them
#         if not text:
#             return {'keywords': [], 'concepts': []}

#         words = text.lower().split()
#         words = [word.strip('.,;:()[]{}"\'-') for word in words if word.strip('.,;:()[]{}"\'-')]

#         # Remove stop words and short words
#         filtered_words = [word for word in words if word not in self.stop_words and len(word) > min_word_length]

#         # Count word frequencies
#         word_freq = {}
#         for word in filtered_words:
#             word_freq[word] = word_freq.get(word, 0) + 1

#         # Get top keywords by frequency
#         keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:max_terms]

#         # Extract multi-word concepts (bigrams)
#         bigrams = []
#         for i in range(len(filtered_words) - 1):
#             if filtered_words[i] and filtered_words[i+1]:
#                 bigram = filtered_words[i] + ' ' + filtered_words[i+1]
#                 bigrams.append(bigram)

#         bigram_counts = {}
#         for bigram in bigrams:
#             bigram_counts[bigram] = bigram_counts.get(bigram, 0) + 1

#         # Get top bigrams
#         top_bigrams = sorted(bigram_counts.items(), key=lambda x: x[1], reverse=True)[:max_terms//2]

#         # Extract domain-specific concepts if available
#         domain_concepts = []
#         if domain and hasattr(self, 'domain_specific_rules') and self.domain_specific_rules:
#             if 'entity_patterns' in self.domain_specific_rules:
#                 import re
#                 for pattern in self.domain_specific_rules['entity_patterns']:
#                     matches = re.findall(pattern, text)
#                     domain_concepts.extend(matches)

#         return {
#             'keywords': [keyword for keyword, _ in keywords],
#             'bigrams': [bigram for bigram, _ in top_bigrams],
#             'domain_concepts': domain_concepts,
#             'word_frequencies': word_freq,
#             'raw_keywords': keywords,  # Include frequency information
#             'raw_bigrams': top_bigrams  # Include frequency information
#         }
    
#     def perform_sentiment_analysis(self, text):
#         """
#         Analyze tone and sentiment of content sections.
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Dictionary with sentiment analysis results
#         """
#         # Simple sentiment analysis implementation
#         # In a real implementation, this would use a proper sentiment analysis model
        
#         positive_words = ['good', 'excellent', 'positive', 'beneficial', 'advantage', 
#                           'effective', 'success', 'improve', 'best', 'important']
#         negative_words = ['bad', 'poor', 'negative', 'problem', 'disadvantage', 
#                           'ineffective', 'failure', 'worse', 'worst', 'difficult']
        
#         # Count occurrences of positive and negative words
#         text_lower = text.lower()
#         positive_count = sum(text_lower.count(' ' + word + ' ') for word in positive_words)
#         negative_count = sum(text_lower.count(' ' + word + ' ') for word in negative_words)
        
#         # Determine overall sentiment
#         if positive_count > negative_count * 1.5:
#             sentiment = 'positive'
#         elif negative_count > positive_count * 1.5:
#             sentiment = 'negative'
#         else:
#             sentiment = 'neutral'
        
#         # Check for academic/formal language
#         academic_markers = ['therefore', 'thus', 'consequently', 'furthermore', 'moreover']
#         academic_count = sum(text_lower.count(' ' + marker + ' ') for marker in academic_markers)
#         is_academic = academic_count > len(text.split()) / 200  # Arbitrary threshold
        
#         return {
#             'overall_sentiment': sentiment,
#             'positive_count': positive_count,
#             'negative_count': negative_count,
#             'is_academic_tone': is_academic
#         }
    
#     def generate_summaries(self, sections, max_length=200):
#         """
#         Create concise summaries of complex sections.
        
#         Args:
#             sections: Text or sections to summarize
#             max_length: Maximum length of the summary in characters
            
#         Returns:
#             Summary text
#         """
#         # Simple extractive summarization implementation
#         # In a real implementation, this would use more sophisticated NLP techniques
        
#         if isinstance(sections, str):
#             text = sections
#         else:
#             # If sections is a dictionary or list, concatenate all text
#             text = ' '.join([section if isinstance(section, str) else section.get('text', '')
#                              for section in sections])
        
#         # Split into sentences
#         sentences = text.replace('!', '.').replace('?', '.').split('.')
#         sentences = [s.strip() for s in sentences if s.strip()]
        
#         if not sentences:
#             return ""
        
#         # Simple scoring: prefer sentences with keywords and at the beginning/end
#         keywords = self.extract_keywords_and_concepts(text)['keywords'][:10]
        
#         scored_sentences = []
#         for i, sentence in enumerate(sentences):
#             # Score based on position (beginning and end sentences are important)
#             position_score = 1.0
#             if i < len(sentences) / 10:  # First 10%
#                 position_score = 2.0
#             elif i > len(sentences) * 0.9:  # Last 10%
#                 position_score = 1.5
                
#             # Score based on keyword presence
#             keyword_score = sum(1.0 for keyword in keywords if keyword in sentence.lower())
            
#             # Score based on sentence length (not too short, not too long)
#             length = len(sentence.split())
#             length_score = 1.0
#             if 5 <= length <= 20:
#                 length_score = 1.5
                
#             # Combine scores
#             total_score = position_score + keyword_score + length_score
#             scored_sentences.append((sentence, total_score))
        
#         # Sort by score and select top sentences
#         scored_sentences.sort(key=lambda x: x[1], reverse=True)
        
#         # Select top sentences up to max_length
#         summary = ""
#         for sentence, _ in scored_sentences:
#             if len(summary) + len(sentence) <= max_length:
#                 summary += sentence + ". "
#             else:
#                 break
                
#         return summary.strip()
    
#     def extract_key_terms_with_importance(self, text, max_terms=20, min_word_length=3):
#         """
#         Extract key terms with importance scores.
        
#         Args:
#             text: Text to analyze
#             max_terms: Maximum number of terms to extract
#             min_word_length: Minimum length of words to consider
            
#         Returns:
#             List of (term, importance) tuples sorted by importance
#         """
#         result = self.extract_keywords_and_concepts(text, self.domain, max_terms*2, min_word_length)
#         term_importance = {}
        
#         # Add single words with their importance
#         for term, count in result['raw_keywords']:
#             # Calculate importance using frequency
#             term_importance[term] = count / max(1, len(text.split())) * 0.8  # Single words get slightly lower weight
        
#         # Add bigrams with higher weight
#         for bigram, count in result['raw_bigrams']:
#             if count > 1:  # Only include bigrams that occur more than once
#                 term_importance[bigram] = count / max(1, len(result.get('bigrams', []))) * 1.2  # Bigrams get higher weight
        
#         # Add domain-specific concepts with highest weight
#         for concept in result.get('domain_concepts', []):
#             # Domain concepts get highest weight
#             term_importance[concept] = 1.5
        
#         # Sort by importance and return top terms
#         sorted_terms = sorted(term_importance.items(), key=lambda x: x[1], reverse=True)
        
#         return sorted_terms[:max_terms]
    
#     def _identify_semantic_segment_type(self, text):
#         """
#         Focuses solely on semantic classification, not structural
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Semantic segment type as string
#         """
#         text = text.strip().lower()
        
#         # Check if it's a definition
#         if text.startswith('definition') or ': is defined as' in text or 'refers to' in text:
#             return 'definition'
            
#         # Check if it's an example
#         if text.startswith('example') or text.startswith('for example') or 'for instance' in text:
#             return 'example'
            
#         # Check if it's a theorem/proposition
#         if text.startswith('theorem') or text.startswith('proposition') or text.startswith('lemma'):
#             return 'theorem'
            
#         # Check if it's a proof
#         if text.startswith('proof') or text.endswith('q.e.d.') or text.endswith('□'):
#             return 'proof'
            
#         # Check domain-specific section types
#         if self.domain and 'section_types' in self.domain_specific_rules:
#             for section_type in self.domain_specific_rules['section_types']:
#                 if section_type.lower() in text[:20]:
#                     return section_type
        
#         # Default to paragraph
#         return 'paragraph'
    
#     def extract_keywords_with_positions(self, text, domain=None, max_terms=20, min_word_length=3):
#         """
#         Extract keywords with their positions in the text.

#         Args:
#             text: Text to analyze
#             domain: Optional domain to customize extraction
#             max_terms: Maximum number of terms to return
#             min_word_length: Minimum length of words to consider

#         Returns:
#             Dictionary with keywords, concepts and their positions
#         """
#         # Get base extraction results
#         base_results = self.extract_keywords_and_concepts(text, domain, max_terms, min_word_length)

#         # Add position information
#         terms_with_positions = {}

#         # Find positions for keywords
#         for keyword in base_results['keywords']:
#             positions = self._find_term_positions(text, keyword)
#             if positions:
#                 terms_with_positions[keyword] = positions

#         # Find positions for bigrams
#         for bigram in base_results['bigrams']:
#             positions = self._find_term_positions(text, bigram)
#             if positions:
#                 terms_with_positions[bigram] = positions

#         # Find positions for domain concepts
#         for concept in base_results['domain_concepts']:
#             positions = self._find_term_positions(text, concept)
#             if positions:
#                 terms_with_positions[concept] = positions

#         # Add position information to results
#         base_results['term_positions'] = terms_with_positions

#         return base_results

#     def _find_term_positions(self, text, term):
#         """
#         Find all positions of a term in text.

#         Args:
#             text: Text to search in
#             term: Term to find

#         Returns:
#             List of starting positions
#         """
#         import re
#         positions = []
#         pattern = r'\b' + re.escape(term) + r'\b'

#         for match in re.finditer(pattern, text, re.IGNORECASE):
#             positions.append(match.start())

#         return positions
    
#     def analyze_literary_work(self, extraction_result, work_type='play'):
#         """
#         Special method for analyzing literary works like Shakespeare plays

#         Args:
#             extraction_result: Dictionary containing the extracted content
#             work_type: Type of literary work ('play', 'novel', 'poem')

#         Returns:
#             Dictionary with literary analysis results
#         """
#         # Create literary work analyzer if not already available
#         literary_analyzer = LiteraryWorkAnalyzer(work_type)

#         # Extract full text content
#         text_content = extraction_result.get('text_content', '')

#         # Start with basic content analysis
#         analyzed_result = self.analyze_content(extraction_result)

#         # Add literary-specific analysis
#         analyzed_result['literary_analysis'] = {
#             'characters': literary_analyzer.analyze_characters(text_content),
#             'structure': literary_analyzer.identify_scenes(text_content),
#             'literary_devices': literary_analyzer.detect_literary_devices(text_content)
#         }

#         # For plays and poetry, add verse analysis
#         if work_type in ['play', 'poem']:
#             analyzed_result['literary_analysis']['verse_patterns'] = literary_analyzer.analyze_verse_patterns(text_content)

#         return analyzed_result
    
#     def _detect_poetic_structure(self, text):
#         """
#         Detect if text has poetic structure based on line length patterns
        
#         Args:
#             text: Text to analyze
            
#         Returns:
#             Boolean indicating if text appears to be poetry
#         """
#         lines = [line.strip() for line in text.split('\n') if line.strip()]
        
#         if not lines:
#             return False
        
#         # Count lines with similar length (potential verse structure)
#         line_lengths = [len(line) for line in lines]
        
#         # Get average line length
#         avg_length = sum(line_lengths) / len(line_lengths)
        
#         # Count lines within 50% of average length (consistent structure)
#         consistent_lines = sum(1 for length in line_lengths if 0.5*avg_length <= length <= 1.5*avg_length)
        
#         # If at least 60% of lines have consistent length, likely poetry
#         return (consistent_lines / len(line_lengths)) > 0.6
